### Black Box Optimization Benchmarks
Given the results of TT vs. Bayesian Optimization for the HNNs, it makes sense to see how well the Optimizers compare as the size of problem grows in dimensionality. For example, we are looking at < 10 hyperparameter ranges, how well do the optimizers scale if we look at larger problem sizes? theoretically TT should be more performant due to the reduction in complexity. 

So, the goal is now to look at a few benchmarking algorithms (Ackleys, Rastrigins, etc.) and see if we can ascertain differences in:
* minima (the best solution found after a number of calls within a budget)
* expected run times (the total number of function calls)
* wall clock times for the optimization procedure

TODOS
1. we want set up a benchmarking suite of tests(define or reuse ackleys etc)
2. we need to configure the optimizers to work with the test suite
3. Run the optimizers, see if there is any difference with respect to scaling
4. Report on results

Optproblems https://pypi.org/project/optproblems/ seems pretty straightforward and has support for continuous optimization. We would use this or write functions from scratch. We will also need to reconfigure the optimizers to work with this or use functions that we define


In [1]:
! pip install optproblems

In [2]:
import optproblems
help(optproblems)

Help on package optproblems:

NAME
    optproblems - A collection of test problems for black-box optimization.

PACKAGE CONTENTS
    base
    binary
    cec2005 (package)
    cec2007
    continuous
    dtlz
    mpm
    multiobjective
    realworld
    wfg
    zdt

VERSION
    1.3

FILE
    /Users/davidvondollen/opt/anaconda3/envs/phd-fundamentals-3.8/lib/python3.8/site-packages/optproblems/__init__.py




In [3]:
from optproblems import continuous
help(continuous)

Help on module optproblems.continuous in optproblems:

NAME
    optproblems.continuous

DESCRIPTION
    This module contains miscellaneous test problems with continuous/real-valued
    search space. The problems are mostly from the early days of research on
    optimization.

CLASSES
    builtins.list(builtins.object)
        DixonSzegoe
    builtins.object
        EllipsoidFunction
        ModifiedRastriginFunction
        RastriginFunction
        SequenceChecker
        WeierstrassFunction
    optproblems.base.TestProblem(optproblems.base.Problem)
        Ackley
        Branin
        DoubleSum
        Ellipsoid
        FletcherPowell
        GoldsteinPrice
        Griewank
        Hartman3
        Hartman6
        Himmelblau
        LunacekTwoSpheres
            LunacekTwoRastrigins
        ModifiedRastrigin
        Rastrigin
        Rosenbrock
        Schaffer6
        Schaffer7
        Schwefel
        Shekel
        SixHumpCamelback
        Sphere
        Vincent
        Weierst

In [3]:
from optproblems.continuous import *

for i in [Schwefel,FletcherPowell, Vincent,Rastrigin]:
    f = i(num_variables=5)
    print(f([1,2,1,1,1]))
    #print(f.max_bounds)

-5.341415831217057
231803.03307553256
-0.12076428542337381
8.0


In [5]:
help(optproblems.continuous)

Help on module optproblems.continuous in optproblems:

NAME
    optproblems.continuous

DESCRIPTION
    This module contains miscellaneous test problems with continuous/real-valued
    search space. The problems are mostly from the early days of research on
    optimization.

CLASSES
    builtins.list(builtins.object)
        DixonSzegoe
    builtins.object
        EllipsoidFunction
        ModifiedRastriginFunction
        RastriginFunction
        SequenceChecker
        WeierstrassFunction
    optproblems.base.TestProblem(optproblems.base.Problem)
        Ackley
        Branin
        DoubleSum
        Ellipsoid
        FletcherPowell
        GoldsteinPrice
        Griewank
        Hartman3
        Hartman6
        Himmelblau
        LunacekTwoSpheres
            LunacekTwoRastrigins
        ModifiedRastrigin
        Rastrigin
        Rosenbrock
        Schaffer6
        Schaffer7
        Schwefel
        Shekel
        SixHumpCamelback
        Sphere
        Vincent
        Weierst

In [6]:
###Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Imports

In [4]:
#TODO Debug, or rewrite algorithm from scratch
import numpy as np
#from hyperopt import hp, tpe, fmin

from optproblems.continuous import Ackley, Rastrigin, Branin, FletcherPowell, Rosenbrock, GoldsteinPrice, Griewank, Vincent,Himmelblau,SixHumpCamelback

#from bayes_opt import BayesianOptimization as BO
from sklearn.gaussian_process import GaussianProcessRegressor
from scipy.stats import norm

test_suite = [Ackley, Rastrigin,Rosenbrock]
bounds_suite = []


#Vanilla BO with GPR as surrogate model
def ucb_acquisition(mean, std, x):
    
    mean=mean.flatten()

    #UCB
    upper=mean+std
    #argmax
    print(upper)
    best_ix=np.argmax(upper)
    return x[best_ix]

def ei_acquisition(best,mean, std, x):
    #print(best)
    #mean= mean.flatten()

    probs = norm.cdf((mean - best) / (std))
    #print("probs, ", probs)
    best_ix = np.argmax(probs)
    imp = mean - best
    Z = imp / std
    ei = (mean- best) * norm.cdf(Z) + std * norm.pdf(Z)
    best_ix = np.argmax(ei)
    #print(ei)
    return x[best_ix]




    





So some intial thoughts. In looking at the BO, the bottleneck seems to be in fitting the GPR, which makes sense as the complexity is $O(n^3)$ (and this is why people have investigated using tpe, other surrogate models). 

We see tt optimizer working fast (makes sense due the complexity being $O(ndr^2)$ however the drawbacks are that we must discretize the input, also I cannot seem to find if there is a way to limit the total number of objective function calls (set a budget).

With these aspects (discretization vs. continuous), I wonder if it makes sense to compare the two. Still, it is probably worthwhile to still investigate the scaling.

#### Implement Quantum-Enhanced Sparse Gaussian process regression

In [5]:
! pip install dimod dwave-neal

In [6]:
import neal

In [7]:
def rbf_kernel(x, y, l2):
    sqdist = np.sum(x**2,1).reshape(-1,1) + \
             np.sum(y**2,1) - 2*np.dot(x, y.T)
    return np.exp(-.5 * (1/l2) * sqdist)

In [8]:
noise_var=1e-9
import dimod
import greedy
from scipy import stats
import random

def qesgpr_fit(X, y, noise_variance, k):
    """
    gaussian process regression with sparse quantum-enhanced regularization
    X, y: data points
    noise_variance : noise parameter
    k: size that we want to reduce the set X by, for now we increment this to a ratio of |X|/2
    
    Normally GPR would be O(n^3) with this sparse version
    we could potentially have  an upper bound dominated by O(k^3) where k< n.
    Here we use simulated annealing to find the regularization coefficients, but could use
    Quantum annealing, QAOA etc. The QUBO formulation is a first pass, essentially using the euclidean distance 
    of the members of X to create a QUBO with the k constraint
    
    In this version we only regularize the test data
    """
    l2=0.2
    N=len(X)
    #set up prior
    K = rbf_kernel(X, X,l2=l2)
    i_k =noise_var*np.eye(N)
    L = np.linalg.cholesky(K + i_k)
    
    return L

def gpr_subset_fit(X, y, noise_variance, k):
    """
    gaussian process regression with sparse quantum-enhanced regularization
    X, y: data points
    noise_variance : noise parameter
    k: size that we want to reduce the set X by, for now we increment this to a ratio of |X|/2
    
    Normally GPR would be O(n^3) with this sparse version
    we could potentially have  an upper bound dominated by O(k^3) where k< n.
    Here we use simulated annealing to find the regularization coefficients, but could use
    Quantum annealing, QAOA etc. The QUBO formulation is a first pass, essentially using the euclidean distance 
    of the members of X to create a QUBO with the k constraint
    
    In this version we only regularize the test data
    """
    l2=0.2
    N=len(X)
    
    K_idx = random.sample(list(range(N)),k)
    X_star = X[K_idx]
    y_star = y[K_idx]
    #set up prior
    K = rbf_kernel(X_star, X_star,l2=l2)
    i_k =noise_var*np.eye(k)
    L = np.linalg.cholesky(K + i_k)
    
    return L, X_star, y_star, K_idx

def qesgpr_predict_vanilla(L, Xtest, y, noise_variance, k):
    """
    Given a prior and some test data, predict mean and std from posterior
    

    """
    l2=0.2
    K_ = rbf_kernel(Xtest, Xtest,l2=l2)
    
    
    
    Lk = np.linalg.solve(L, rbf_kernel(Xtest, Xtest,l2=l2))
    #print(L.shape, y.shape, Lk.shape)
    #return mean and std
    mu = np.dot(Lk.T, np.linalg.solve(L, y))
    std = np.sqrt(np.diag(K_) - np.sum(Lk**2, axis=0))
    
    return mu, std


def qesgpr_predict(L, Xtest, y, noise_variance, k):
    """
    Given a prior and some test data, predict mean and std from posterior
    
    In this version, we only filter the test data
    """
    l2=0.2
    K_ = rbf_kernel(Xtest, Xtest,l2=l2)
     #construct BQM
    bqm = dimod.BinaryQuadraticModel.empty(dimod.BINARY)
    for i in range(len(K_)):
        for j in range(len(K_)):
            if i == j:
                bqm.add_variable(i, -np.abs(K_[i, j]))
            elif i > j:
                #bqm.add_interaction(i, j, np.abs(K_[i, j]))
                bqm.add_interaction(i, j, -np.abs(np.linalg.norm(Xtest[i] - Xtest[j])))

    bqm.update(dimod.generators.combinations(list(range(len(K_))), k, strength=1000))
    
    #Sample solution from BQM
    #sampler = neal.SimulatedAnnealingSampler()
    sampler = greedy.SteepestDescentSampler()
    sampler
    sample_ = sampler.sample(bqm)
    sample= sample_.first.sample
    
    #filter data by coefficients
    K_idx = np.where(np.array(list(sample.values())) ==1)
    
    #Calculate posterior with filtered data
    X_t_star = Xtest[K_idx]
    #print(X.shape)
    K_star = rbf_kernel(X_t_star, X_t_star,l2=l2)
    
    Lk = np.linalg.solve(L, rbf_kernel(X_t_star, X_t_star,l2=l2))
    #print(L.shape, y.shape, Lk.shape)
    #return mean and std
    mu = np.dot(Lk.T, np.linalg.solve(L, y))
    std = np.sqrt(np.diag(K_star) - np.sum(Lk**2, axis=0))
    
    return mu, std

def qesgpr_fit2(X, y, noise_variance, k):
    """
    gaussian process regression with sparse quantum-enhanced regularization
    X, y: data points
    noise_variance : noise parameter
    k: size that we want to reduce the set X by, for now we increment this to a ratio of |X|/2
    
    Normally GPR would be O(n^3) with this sparse version
    we could potentially have  an upper bound dominated by O(k^3) where k< n.
    Here we use simulated annealing to find the regularization coefficients, but could use
    Quantum annealing, QAOA etc. The QUBO formulation is a first pass, essentially using the euclidean distance 
    of the members of X to create a QUBO with the k constraint
    
    In this version we filter both train and test data
    """
    l2=0.2
    N=len(X)
    
    #set up prior -Here we filter down the initial samples as well as xtest
    K = rbf_kernel(X, X,l2=l2)
    bqm = dimod.BinaryQuadraticModel.empty(dimod.BINARY)
    for i in range(len(K)):
        for j in range(len(K)):
            if i == j:
                bqm.add_variable(i, np.abs(y[i]))
            elif i > j:
                #bqm.add_interaction(i, j, -np.abs(K[i, j]))
                bqm.add_interaction(i, j, -np.abs(np.linalg.norm(X[i] - X[j])))
                #print(np.abs(stats.pearsonr(X[i] ,X[j]))
                #bqm.add_interaction(i, j, np.abs(stats.pearsonr(X[i] ,X[j])))
    bqm.update(dimod.generators.combinations(list(range(len(K))), k, strength=10000))            
    sampler = neal.SimulatedAnnealingSampler()
    sample_ = sampler.sample(bqm)
    sample= sample_.first.sample
    
    #filter data by coefficients
    K_idx = np.where(np.array(list(sample.values())) ==1)
    
    #Calculate posterior with filtered data
    X_star = X[K_idx]
    y_star = y[K_idx]
    K_star = rbf_kernel(X_star, X_star,l2=l2)


    i_k =noise_var*np.eye(len(X_star))
    L = np.linalg.cholesky(K_star + i_k)
    
    return L, X_star, y_star, K_idx

    
    

def qesgpr_predict2(L, X_star, Xtest, y_star, noise_variance, k):
    """
    predict mean and std
    
    In this version, we filter on both train and test data
    """
    l2=0.2
    K_ = rbf_kernel(Xtest, Xtest,l2=l2)
     #construct BQM
    bqm = dimod.BinaryQuadraticModel.empty(dimod.BINARY)
    for i in range(len(K_)):
        for j in range(len(K_)):
            if i == j:
                bqm.add_variable(i, np.abs(K_[i, j]))
            elif i > j:
                #bqm.add_interaction(i, j, -np.abs(K_[i, j]))
                bqm.add_interaction(i, j, -np.abs(np.linalg.norm(Xtest[i] - Xtest[j])))
                #bqm.add_interaction(i, j, -np.abs(stats.pearsonr(Xtest[i], Xtest[j])))

    bqm.update(dimod.generators.combinations(list(range(len(K_))), k, strength=10000))
    
    #Sample solution from BQM
    sampler = neal.SimulatedAnnealingSampler()
    #sampler = greedy.SteepestDescentSampler()
    sample_ = sampler.sample(bqm)
    sample= sample_.first.sample
    
    #filter data by coefficients
    K_idx = np.where(np.array(list(sample.values())) ==1)
    
    #Calculate posterior with filtered data
    X_t_star = Xtest[K_idx]
    #print(X_t_star.shape)
    K_star = rbf_kernel(X_t_star, X_t_star,l2=l2)
    
    Lk = np.linalg.solve(L, rbf_kernel(X_star, X_t_star,l2=l2))
    #print(L, rbf_kernel(X_star, X_t_star,l2=l2))
    #print(L.shape, y.shape, Lk.shape)
    #return mean and std
   # print(L, y_star)
    mu = np.dot(Lk.T, np.linalg.solve(L, y_star))
    std = np.sqrt(np.diag(K_star) - np.sum(Lk**2, axis=0))
    
    return mu, std 
    
def qesgpr_predict3(L, X_star, Xtest, y_star, noise_variance, k):
    """
    predict mean and std
    
    In this version, we filter on both train and test data
    """
    l2=0.2
    K_ = rbf_kernel(Xtest, Xtest,l2=l2)
     #construct BQM
    bqm = dimod.BinaryQuadraticModel.empty(dimod.BINARY)
    for i in range(len(K_)):
        for j in range(len(K_)):
            if i == j:
                bqm.add_variable(i, np.abs(K_[i, j]))
            elif i > j:
                #bqm.add_interaction(i, j, -np.abs(K_[i, j]))
                bqm.add_interaction(i, j, -np.abs(np.linalg.norm(Xtest[i] - Xtest[j])))
                #bqm.add_interaction(i, j, -np.abs(stats.pearsonr(Xtest[i], Xtest[j])))

    bqm.update(dimod.generators.combinations(list(range(len(K_))), k, strength=10000))
    
    #Sample solution from BQM
    sampler = neal.SimulatedAnnealingSampler()
    #sampler = greedy.SteepestDescentSampler()
    sample_ = sampler.sample(bqm)
    sample= sample_.first.sample
    
    #filter data by coefficients
    K_idx = np.where(np.array(list(sample.values())) ==1)
    
    #Calculate posterior with filtered data
    X_t_star = Xtest[K_idx]
    #print(X_t_star.shape)
    K_star = rbf_kernel(X_t_star, X_t_star,l2=l2)
    
    Lk = np.linalg.solve(L, rbf_kernel(X_star, X_t_star,l2=l2))
    #print(L, rbf_kernel(X_star, X_t_star,l2=l2))
    #print(L.shape, y.shape, Lk.shape)
    #return mean and std
   # print(L, y_star)
    mu = np.dot(Lk.T, np.linalg.solve(L, y_star))
    std = np.sqrt(np.diag(K_star) - np.sum(Lk**2, axis=0))
    
    return mu, std, X_t_star


def select_features(X, y, k):
    """
    predict mean and std
    
    In this version, we filter on both train and test data
    """
    l2=0.2
  
     #construct BQM
    bqm = dimod.BinaryQuadraticModel.empty(dimod.BINARY)
    for i in range(len(K_)):
        for j in range(len(K_)):
            if i == j:
                bqm.add_variable(i, np.abs(y[i]))
            elif i > j:
                #bqm.add_interaction(i, j, -np.abs(K_[i, j]))
                #bqm.add_interaction(i, j, -np.abs(np.linalg.norm(X[:, i] - Xtest[j])))
                bqm.add_interaction(i, j, -np.abs(stats.pearsonr(X[:, i], X[:, j])))

    bqm.update(dimod.generators.combinations(list(range(len(K_))), k, strength=10000))
    
    #Sample solution from BQM
    sampler = neal.SimulatedAnnealingSampler()
    #sampler = greedy.SteepestDescentSampler()
    sample_ = sampler.sample(bqm)
    sample= sample_.first.sample
    
    #filter data by coefficients
    s_idx = np.where(np.array(list(sample.values())) ==1)
    
    #Calculate posterior with filtered data
    X_filtered = X[:, K_idx]
    #print(X_t_star.shape)
    
    
    return X_filtered
    
    

In [9]:
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error

class GaussianProcessRegression:
    def __init__(self, kernel_function, noise_variance=1e-4):
        self.kernel_function = kernel_function
        self.noise_variance = noise_variance
        self.alpha = None
        self.L = None
        self.X_train = None

    def fit(self, X, y):
        self.X_train = X
        K = self.kernel_function(X, X) + self.noise_variance * np.eye(len(X))
        #print(K.shape)
        self.L = np.linalg.cholesky(K + 1e-10 * np.eye(len(X)))  # Add a small jitter for numerical stability
        self.alpha = np.linalg.solve(self.L.T, np.linalg.solve(self.L, y))
        
   
    def predict(self, X):
        K_s = self.kernel_function(self.X_train, X)
        #print(K_s.shape)
        mu_s = K_s.T @ self.alpha

        v = np.linalg.solve(self.L, K_s)
        cov_s = self.kernel_function(X, X) - v.T @ v

        return mu_s, np.diag(cov_s)
    



def rbf_kernel_(X1, X2, length_scale=0.2, sigma_f=0.2):
    sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    return sigma_f**2 * np.exp(-0.5 / length_scale**2 * sqdist)

# Example usage:
#X_train = np.array([[1.], [2.], [3.]])
#y_train = np.array([1., 3., 2.])

X_train = np.random.rand(100, 5)  
y_train = np.sum(np.sin(X_train[:] * 4) + np.cos(X_train[:] * 4),axis=1).reshape(-1,1) # Sample target values
X_test = np.random.rand(50, 5)
y_test = np.sum(np.sin(X_test[:] * 4) + np.cos(X_test[:] * 4),axis=1).reshape(-1,1) 

gp = GaussianProcessRegression(kernel_function=rbf_kernel_)
gp.fit(X_train, y_train)

#X_test = np.array([[1.5], [2.5]])
y_pred, sigma = gp.predict(X_test)

print(y_test.shape)
#gp = GaussianProcessRegression(kernel_function=plq_kernel)
#gp.fit_qk(X_train, y_train)
#y_pred, sigma = gp.predict_qk(X_test)

print("Predictions:", y_pred)
print("Standard Deviations:", sigma)

print(r2_score(y_pred, y_test),mean_squared_error(y_pred, y_test, squared=False) )

(50, 1)
Predictions: [[-3.84643782e-01]
 [-6.61889342e-01]
 [ 3.58265295e+00]
 [ 7.58682561e-01]
 [ 4.36050634e-01]
 [ 8.16874145e-01]
 [ 1.73990481e+00]
 [ 2.18517640e+00]
 [-1.36918011e+00]
 [-1.16897959e+00]
 [ 1.47693450e+00]
 [ 1.38923188e+00]
 [ 2.38857649e+00]
 [ 1.19396942e+00]
 [ 2.56099978e+00]
 [-1.54590417e-01]
 [ 5.57129056e-01]
 [ 8.86998813e-02]
 [-4.84324864e-01]
 [-1.71450863e-01]
 [ 4.49763220e-01]
 [-2.60010755e-01]
 [ 1.36876745e+00]
 [ 2.57736923e-02]
 [-3.26784529e-01]
 [ 1.34940504e+00]
 [ 3.44882540e-01]
 [-1.11543269e-01]
 [-1.67792443e-01]
 [-1.30804697e-01]
 [ 2.69213519e+00]
 [ 1.85512972e+00]
 [-3.65946766e-03]
 [ 3.19363596e+00]
 [ 2.40103582e+00]
 [ 1.06393260e+00]
 [-5.07985291e-01]
 [-3.15568323e-01]
 [ 1.73009642e+00]
 [ 8.66480999e-01]
 [ 4.07634625e-01]
 [ 4.34903378e+00]
 [ 4.96229962e-02]
 [-3.14640430e-01]
 [ 1.12696572e+00]
 [ 2.81640214e-01]
 [ 4.54507079e+00]
 [ 2.59662371e-01]
 [-1.03121775e+00]
 [-8.97926177e-01]]
Standard Deviations: [0.0369

In [10]:
def bo_optimize(maxmin_bounds, num_calls, f, num_start, dimension):
    surrogate = GaussianProcessRegression(kernel_function=rbf_kernel_)
    
    #generate intial samples to warm start optimizer, given by num_start
    try:
        xsamples=np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (num_start, dimension))
    except:
        xsamples=np.random.uniform(maxmin_bounds[0], maxmin_bounds[1], size = (num_start, dimension))
        #print(f.min_bounds)
    #print(f(xsamples[0]))

    #Get obj function values to train surrogate
    ysamples=np.array([f(x) for x in xsamples])
    #print(ysamples)
    #print(ysamples)
    for i in range(num_calls):
        #Fit the surrogate function.
        surrogate.fit(xsamples, ysamples)
        yhat, _ = surrogate.predict(xsamples)
    
        best = np.max(yhat)
        #print(best)
        
        
        #predict mean, std from xtest
        
        try:
            xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (10, dimension))
        except:
            xtest = np.random.uniform(maxmin_bounds[0], maxmin_bounds[1], size = (10, dimension))
        mean, std = surrogate.predict(xtest)
        #print(mean, std)

        #Find new x using acquisition function.
        new_x= ei_acquisition(best, np.mean(mean), np.std(std), xtest)
        #new_x= ucb_acquisition(mean, std, xtest)
        #print(new_x)
        new_y= f(new_x)
        #print(new_y)

        #update
        xsamples=np.vstack((xsamples, new_x))
        ysamples=np.hstack((ysamples, new_y))
        
    print("bo: ",ysamples[np.argmin(ysamples)])
    return ysamples[np.argmin(ysamples)], xsamples[np.argmin(ysamples)]
     
 
    
problem_dimension = 3    
maxmin_bounds = (5.,-5.)
for black_box_function in test_suite:
    print(black_box_function)
   
    
    try:
        f=black_box_function(num_variables=problem_dimension)
    except:
        f=black_box_function()
        
    #print(f([10,10,10,10]))
    res = bo_optimize(maxmin_bounds,100,f, num_start=10,dimension=problem_dimension)
    print(res)
    

<class 'optproblems.continuous.Ackley'>
bo:  4.45225821379702
(4.45225821379702, array([-0.39532712,  1.19487812, -0.17745177]))
<class 'optproblems.continuous.Rastrigin'>
bo:  18.024757563485494
(18.024757563485494, array([-1.01800136, -0.98798413,  3.98165816]))
<class 'optproblems.continuous.Rosenbrock'>
bo:  172.37605825217722
(172.37605825217722, array([-0.89897396,  2.10202184,  4.38026622]))


In [11]:
"""from qiskit import Aer

from qiskit.circuit.library import NLocal, TwoLocal
from qiskit.utils import QuantumInstance
import numpy as np
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.circuit.library import RYGate, RZGate, CXGate

class QuantumKernelRegression:
    def __init__(self, feature_map, backend=None, noise_variance=1e-1):
        self.feature_map = feature_map
        self.backend = backend if backend else Aer.get_backend('statevector_simulator')
        self.noise_variance = noise_variance

        self.qi = QuantumInstance(self.backend)
        self.X_train = None

    def fit(self, X, y):
        self.X_train = X
        K = self.kernel_function(X, X) + self.noise_variance * np.eye(len(X))
        self.L = np.linalg.cholesky(K + 1e-10 * np.eye(len(X)))
        self.alpha = np.linalg.solve(self.L.T, np.linalg.solve(self.L, y))

    def predict(self, X):
        k_s = self.kernel_function(self.X_train, X)
        return k_s.T @ self.alpha

    def kernel_function(self, X1, X2):
        from qiskit_machine_learning.kernels import QuantumKernel

        qkernel = QuantumKernel(feature_map=self.feature_map, quantum_instance=self.qi)
        return qkernel.evaluate(X1, X2)

# Test with n-dimensional data
n_dimensions = 3  # For example
# Define rotation blocks
num_qubits = 3

feature_map = TwoLocal(num_qubits, ['ry', 'rz'], 'cx', reps=1, entanglement='linear', insert_barriers=True)
backend = Aer.get_backend('qasm_simulator')

qkr = QuantumKernelRegression(feature_map=feature_map, backend=backend)
#X_train = np.random.rand(10, n_dimensions)  # 10 data points with n_dimensions features each
#y_train = np.random.rand(10,1)

qkr.fit(X_train, y_train)

#X_test = np.array([[1.5, 1.5], [2.5, 2.5]])
y_pred, sigma = qkr.predict(X_test)

print("Predictions:", y_pred)
print("Standard Deviations:", sigma)
print(r2_score(y_pred, y_test),mean_squared_error(y_pred, y_test, squared=False) )
"""

'from qiskit import Aer\n\nfrom qiskit.circuit.library import NLocal, TwoLocal\nfrom qiskit.utils import QuantumInstance\nimport numpy as np\nfrom qiskit.circuit import QuantumCircuit, Parameter\nfrom qiskit.circuit.library import RYGate, RZGate, CXGate\n\nclass QuantumKernelRegression:\n    def __init__(self, feature_map, backend=None, noise_variance=1e-1):\n        self.feature_map = feature_map\n        self.backend = backend if backend else Aer.get_backend(\'statevector_simulator\')\n        self.noise_variance = noise_variance\n\n        self.qi = QuantumInstance(self.backend)\n        self.X_train = None\n\n    def fit(self, X, y):\n        self.X_train = X\n        K = self.kernel_function(X, X) + self.noise_variance * np.eye(len(X))\n        self.L = np.linalg.cholesky(K + 1e-10 * np.eye(len(X)))\n        self.alpha = np.linalg.solve(self.L.T, np.linalg.solve(self.L, y))\n\n    def predict(self, X):\n        k_s = self.kernel_function(self.X_train, X)\n        return k_s.T @ se

In [52]:
from qiskit import Aer
from qiskit.circuit.library import ZZFeatureMap
from qiskit.utils import QuantumInstance
from qiskit_machine_learning.kernels import QuantumKernel

class QuantumKernelRegression:
    def __init__(self, feature_map, backend=None, noise_variance=1e-4):
        self.kernel_function = self._quantum_kernel_function(feature_map, backend)
        self.noise_variance = noise_variance
        self.alpha = None
        self.L = None
        self.X_train = None

    def _quantum_kernel_function(self, feature_map, backend):
        # Setup quantum kernel using provided feature map and backend
        qkernel = QuantumKernel(feature_map=feature_map, quantum_instance=backend)
        
        def kernel_function(X1, X2):
            return qkernel.evaluate(X1, X2)
        
        return kernel_function
    
    def get_kernel(self, X):
        K = self.kernel_function(X, X)
        return K
        

    def fit(self, X, y):
        self.X_train = X
        K = self.kernel_function(X, X) + self.noise_variance * np.eye(len(X))
        self.L = np.linalg.cholesky(K + 1e-10 * np.eye(len(X)))
        self.alpha = np.linalg.solve(self.L.T, np.linalg.solve(self.L, y))

    def predict(self, X):
        K_s = self.kernel_function(self.X_train, X)
        mu_s = K_s.T @ self.alpha

        v = np.linalg.solve(self.L, K_s)
        cov_s = self.kernel_function(X, X) - v.T @ v

        return mu_s, np.diag(cov_s)

# Define a quantum feature map
feature_dim = 2  # assuming 2-dimensional data for simplicity
feature_map = ZZFeatureMap(feature_dimension=feature_dim)

# Define quantum instance
backend = QuantumInstance(Aer.get_backend('statevector_simulator'))

# Example usage
#X_train = np.array([[1., 1.], [2., 2.], [3., 3.]])
#y_train = np.array([1., 3., 2.])

qkr = QuantumKernelRegression(feature_map=feature_map, backend=backend)
qkr.fit(X_train, y_train)

#X_test = np.array([[1.5, 1.5], [2.5, 2.5]])
y_pred, sigma = qkr.predict(X_test)

print("Predictions:", y_pred)
print("Standard Deviations:", sigma)


Predictions: [[ 0.5046586 ]
 [-0.47632251]
 [ 0.95081323]
 [ 0.26886628]
 [ 0.58526532]
 [ 0.8243942 ]
 [ 0.60562021]
 [ 2.52758462]
 [ 0.13064695]
 [ 0.16442507]
 [ 0.9512384 ]
 [ 1.5646475 ]
 [ 1.53855829]
 [ 1.21248696]
 [-0.15005991]
 [-1.36817958]
 [-0.10662524]
 [ 0.43213558]
 [ 0.97936181]
 [ 1.24234197]
 [ 0.48743223]
 [ 0.88725434]
 [ 1.98520758]
 [ 0.98016751]
 [ 0.55859689]
 [-0.52657567]
 [-0.00862735]
 [ 1.31312556]
 [-0.28122175]
 [ 1.53585287]
 [ 0.46377899]
 [ 0.5816445 ]
 [ 0.36745682]
 [ 1.7183915 ]
 [ 2.78050525]
 [ 1.01465788]
 [ 0.35953292]
 [-0.51203222]
 [ 1.56608129]
 [ 0.40942268]
 [ 1.59321332]
 [ 2.22154002]
 [ 0.61244308]
 [-0.42535679]
 [-0.1897187 ]
 [ 1.5635457 ]
 [ 0.56306083]
 [ 0.24170881]
 [ 1.0458033 ]
 [-0.88964614]]
Standard Deviations: [0.89457511 0.68720894 0.87463509 0.82391172 0.80003095 0.8203
 0.89436792 0.77516795 0.77694022 0.89811526 0.88461333 0.68184958
 0.85531587 0.89404109 0.79282166 0.77010472 0.86060137 0.76898185
 0.75467801 0.8473

In [53]:
#Random fourier features
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, Kernel, RBF
from sklearn.metrics import r2_score, mean_squared_error

class RandomFourierFeatures:
    def __init__(self, output_dim, input_dim, kernel_scale=0.2):
        """
        Initialize Random Fourier Features
        
        :param output_dim: Dimension of the transformed feature space.
        :param input_dim: Dimension of the input data.
        :param kernel_scale: A hyperparameter that scales the random weights.
        """
        self.output_dim = output_dim
        self.input_dim = input_dim
        self.kernel_scale = kernel_scale
        self.random_weights = None
        self.random_offsets = None

    def fit(self, X):
        """
        Generate random weights and offsets for the transformation.
        
        :param X: Input data. Shape: (num_samples, input_dim)
        :return: self
        """
        self.random_weights = np.random.randn(self.input_dim, self.output_dim) * self.kernel_scale
        self.random_offsets = 2 * np.pi * np.random.rand(self.output_dim)
        return self

    def transform(self, X):
        """
        Transform the input data using Random Fourier Features.
        
        :param X: Input data. Shape: (num_samples, input_dim)
        :return: Transformed data. Shape: (num_samples, output_dim)
        """
        if self.random_weights is None or self.random_offsets is None:
            raise ValueError("The fit method must be called before transform.")
        
        # Project data
        projected_data = X @ self.random_weights + self.random_offsets
        return np.sqrt(2 / self.output_dim) * np.cos(projected_data)

    
def rff_predict(X_train, X_test, y_train, output_dim, k):
    rff = RandomFourierFeatures(output_dim=output_dim, input_dim=X_train.shape[1])
    rff.fit(X_train)
    X_train_transformed = rff.transform(X_train)
    X_test_transformed = rff.transform(X_test)
    

    num_inducing_points = k
    
    inducing_indices = np.random.choice(X_train_transformed.shape[0], num_inducing_points, replace=False)
    X_inducing = X_train_transformed[inducing_indices]

    # Use Gaussian Process Regression with inducing points
    
    # Adding a white kernel to account for noise
    #gpr = GaussianProcessRegressor(kernel=RBF(), random_state=42)
    #gpr.fit(X_inducing, y_train[inducing_indices])
    #y_pred= gpr.predict(X_test_transformed)
    #L = qesgpr_fit(X_inducing, y_train[inducing_indices], noise_variance=0.2, k=k)
    #y_pred = qesgpr_predict_vanilla(L,  X_test_transformed, y_train[inducing_indices], noise_variance=0.2, k=k)
    
    gp = GaussianProcessRegression(kernel_function=rbf_kernel_)
    gp.fit(X_inducing, y_train[inducing_indices])
    
    #gp.fit(X_train_transformed, y_train)

    #X_test = np.array([[1.5], [2.5]])
    y_pred, sigma = gp.predict(X_test_transformed)
    
    return y_pred, sigma, inducing_indices


def rff_predict_qbo(X_train, X_test, y_train, output_dim, k):
    rff = RandomFourierFeatures(output_dim=output_dim, input_dim=X_train.shape[1])
    rff.fit(X_train)
    X_train_transformed = rff.transform(X_train)
    X_test_transformed = rff.transform(X_test)
    

    K = rbf_kernel(X_train, X_train,l2=0.2)
    #print(K, y_train)
    bqm = dimod.BinaryQuadraticModel.empty(dimod.BINARY)
    for i in range(len(K)):
        for j in range(len(K)):
            if i == j:
                #bqm.add_variable(i, -np.abs(y[i]))
                #bqm.add_variable(i, np.abs(K[i, i]))
                bqm.add_variable(i, (y_train[i]))
                
            elif i > j:
                bqm.add_interaction(i, j, 10 *np.abs(K[i, j] ))
                #bqm.add_interaction(i, j, -np.abs(np.linalg.norm(X[i] - X[j])))
                #print(np.abs(stats.pearsonr(X[i] ,X[j]))
                #bqm.add_interaction(i, j, np.abs(stats.pearsonr(X[i] ,X[j])))
    bqm.update(dimod.generators.combinations(list(range(len(K))), k, strength=1000))            
    sampler = neal.SimulatedAnnealingSampler()
    sample_ = sampler.sample(bqm)
    sample= sample_.first.sample
    
    #filter data by coefficients
    inducing_indices = np.where(np.array(list(sample.values())) ==1)
    #print(len(K_idx[0]))

    # Use Gaussian Process Regression with inducing points
    
    # Adding a white kernel to account for noise
    #gpr = GaussianProcessRegressor(kernel=RBF(), random_state=42)
    #gpr.fit(X_inducing, y_train[inducing_indices])
    #y_pred= gpr.predict(X_test_transformed)
    #L = qesgpr_fit(X_inducing, y_train[inducing_indices], noise_variance=0.2, k=k)
    #y_pred = qesgpr_predict_vanilla(L,  X_test_transformed, y_train[inducing_indices], noise_variance=0.2, k=k)
    
    X_inducing = X_train_transformed[inducing_indices]
    gp = GaussianProcessRegression(kernel_function=rbf_kernel_)
    gp.fit(X_inducing, y_train[inducing_indices])
    
    #gp.fit(X_train_transformed, y_train)

    #X_test = np.array([[1.5], [2.5]])
    y_pred, sigma = gp.predict(X_test_transformed)
    
    return y_pred, sigma, inducing_indices

# Example usage:
k =20
output_dim = 500
#X_train = np.random.rand(100, 1)  
#y_train = np.sin(X_train[:] * 4) + np.cos(X_train[:] * 4) # Sample target values
#X_test = np.random.rand(20, 1)
#y_test = np.sin(X_test[:] * 4) + np.cos(X_test[:] * 4) 



y_pred, s, inducing_indices = rff_predict_qbo(X_train, X_test, y_train, output_dim, k)

y_pred
print(y_pred)
# Predict and score



from sklearn.metrics import mean_squared_error, r2_score


print(r2_score(y_pred, y_test),mean_squared_error(y_pred, y_test, squared=False) )

#print(r2_score(y_pred[0], y_test),mean_squared_error(y_pred[0], y_test, squared=False) )

[[-1.89662262]
 [-1.75646008]
 [ 4.65220286]
 [ 2.00178555]
 [ 3.03196862]
 [ 0.93235963]
 [ 5.07894817]
 [ 3.67604667]
 [-2.90247129]
 [-4.40695171]
 [ 5.43985659]
 [ 1.04908331]
 [ 2.67977077]
 [ 2.29564302]
 [ 3.29815861]
 [ 0.34322357]
 [ 1.27954074]
 [ 0.15056746]
 [-0.40901011]
 [-0.39473736]
 [-0.58774521]
 [-0.75614052]
 [ 4.21534488]
 [-0.11480971]
 [-0.86729507]
 [ 1.90034217]
 [ 3.88121164]
 [ 0.78045364]
 [ 0.16653714]
 [ 1.01547031]
 [ 3.14144018]
 [ 3.08275297]
 [-0.25509327]
 [ 4.87141835]
 [ 3.88410315]
 [ 3.74162255]
 [-0.86699832]
 [-1.92862574]
 [ 4.48101931]
 [ 3.29297918]
 [ 0.96799866]
 [ 4.53670905]
 [ 0.38351454]
 [ 0.2980211 ]
 [ 1.4345857 ]
 [ 0.45699804]
 [ 5.27847536]
 [ 0.11439903]
 [-0.34158319]
 [-1.88674326]]
0.9041445440198779 0.7260146459817404


In [54]:
def rff_qubo_optimize(maxmin_bounds,num_calls, f, num_start, dimension, output_dim, k):
    noise_var=1e-9
    
    #generate intial samples to warm start optimizer, given by num_start
    #xsamples=np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (num_start, dimension))
   # print(xsamples)
    try:
        xsamples=np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (num_start, dimension))
    except:
        xsamples=np.random.uniform(maxmin_bounds[0], maxmin_bounds[1], size = (num_start, dimension))

    #Get obj function values to train surrogate
    ysamples=np.array([f(x) for x in xsamples])
    print(ysamples)
   
    for i in range(num_calls):
        #Fit the surrogate model.
        #k = int(len(xsamples) * 0.5)
        #mean, std = qesgpr_fit_predict(xsamples, ysamples, noise_variance=1e-6, k=k)
        #Use fit2 and predict2 functions to use quantum filters for both data, x, xtest
        #fit prior
        k = int(len(xsamples) * 0.5)
        #L, x_star, y_star = qesgpr_fit2(xsamples, ysamples, noise_var, k)
        #L = qesgpr_fit(xsamples, ysamples, noise_var, k)
        #print(x_star.shape, y_star.shape)
        #yhat, _ = qesgpr_predict2(L, x_star, xsamples, y_star, noise_var, k)
        #yhat, _ = qesgpr_predict(L,xsamples, ysamples, noise_var, k)
        mean, std, inducing_indices = rff_predict_qbo(xsamples, xsamples, ysamples, output_dim,k)
        
        #predict from posterior
        best = np.max(mean)
        #xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (300, dimension))
        try:
            xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (10, dimension))
        except:
            xtest = np.random.uniform(maxmin_bounds[0], maxmin_bounds[1],size = (10, dimension))
        #mean, std = qesgpr_predict2(L,x_star, xtest, y_star, noise_var, k)
        #mean, std = qesgpr_predict(L, xtest, yhat, noise_var, k)
       # print(xtest)
        mean, std, inducing_indices = rff_predict(xsamples, xtest, ysamples, output_dim, k)
        #best = np.max(mean)
        mean, std = np.mean(mean), np.mean(std)
        #print(mean, std)

        #Find new x using acquisition function.
        new_x= ei_acquisition(best, mean, std, xtest)
        #new_x= ucb_acquisition(mean, std, xtest)
        new_y= f(new_x)
        #print(new_y)

        #update
        xsamples=np.vstack((xsamples, new_x))
        ysamples=np.hstack((ysamples, new_y))
    print("rff-qubo: ", ysamples[np.argmin(ysamples)])
    return ysamples[np.argmin(ysamples)], xsamples[np.argmin(ysamples)]


def rff_optimize(maxmin_bounds,num_calls, f, num_start, dimension, output_dim, k):
    noise_var=1e-9
    
    #generate intial samples to warm start optimizer, given by num_start
    #xsamples=np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (num_start, dimension))
   # print(xsamples)
    try:
        xsamples=np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (num_start, dimension))
    except:
        xsamples=np.random.uniform(maxmin_bounds[0], maxmin_bounds[1], size = (num_start, dimension))

    #Get obj function values to train surrogate
    ysamples=np.array([f(x) for x in xsamples])
    print(ysamples)
   
    for i in range(num_calls):
        #Fit the surrogate model.
        #k = int(len(xsamples) * 0.5)
        #mean, std = qesgpr_fit_predict(xsamples, ysamples, noise_variance=1e-6, k=k)
        #Use fit2 and predict2 functions to use quantum filters for both data, x, xtest
        #fit prior
        k = int(len(xsamples) * 0.5)
        #L, x_star, y_star = qesgpr_fit2(xsamples, ysamples, noise_var, k)
        #L = qesgpr_fit(xsamples, ysamples, noise_var, k)
        #print(x_star.shape, y_star.shape)
        #yhat, _ = qesgpr_predict2(L, x_star, xsamples, y_star, noise_var, k)
        #yhat, _ = qesgpr_predict(L,xsamples, ysamples, noise_var, k)
        mean, std, inducing_indices = rff_predict(xsamples, xsamples, ysamples, output_dim,k)
        
        #predict from posterior
        best = np.max(mean)
        #xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (300, dimension))
        try:
            xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (10, dimension))
        except:
            xtest = np.random.uniform(maxmin_bounds[0], maxmin_bounds[1],size = (10, dimension))
        #mean, std = qesgpr_predict2(L,x_star, xtest, y_star, noise_var, k)
        #mean, std = qesgpr_predict(L, xtest, yhat, noise_var, k)
       # print(xtest)
        mean, std, inducing_indices = rff_predict(xsamples, xtest, ysamples, output_dim, k)
        #best = np.max(mean)
        mean, std = np.mean(mean), np.mean(std)
        #print(mean, std)

        #Find new x using acquisition function.
        new_x= ei_acquisition(best, mean, std, xtest)
        #new_x= ucb_acquisition(mean, std, xtest)
        new_y= f(new_x)
        #print(new_y)

        #update
        xsamples=np.vstack((xsamples, new_x))
        ysamples=np.hstack((ysamples, new_y))
    print("rff: ", ysamples[np.argmin(ysamples)])
    return ysamples[np.argmin(ysamples)], xsamples[np.argmin(ysamples)]



problem_dimension = 3    
maxmin_bounds = (5.,-5.)
for black_box_function in test_suite:
    print(black_box_function)
   
    
    try:
        f=black_box_function(num_variables=problem_dimension)
    except:
        f=black_box_function()
        
    #print(f([10,10,10,10]))
    res = rff_qubo_optimize(maxmin_bounds,100,f, num_start=10,dimension=problem_dimension, output_dim=output_dim, k=k)
    print(res[0])
    res2 = rff_optimize(maxmin_bounds,100,f, num_start=10,dimension=problem_dimension, output_dim=output_dim, k=k)
    print(res2[0])
    
     

<class 'optproblems.continuous.Ackley'>
[ 8.59455606 10.44594612  9.01392264 13.07610894  6.29598642 11.94090808
  9.59952651  8.78414084 11.95801726 10.8232874 ]
rff-qubo:  5.124358239136551
5.124358239136551
[10.5947146   3.57160421  8.92564602 10.49353719  8.67435263  9.70249918
 10.94671959 12.3229168  12.17397873  8.34256379]
rff:  3.5716042127295786
3.5716042127295786


In [55]:
def qbo_predict(X_train, X_test, y_train, k):

#set up prior -Here we filter down the initial samples as well as xtest
    K = rbf_kernel(X_train, X_train,l2=0.2)
    #print(K, y_train)
    bqm = dimod.BinaryQuadraticModel.empty(dimod.BINARY)
    for i in range(len(K)):
        for j in range(len(K)):
            if i == j:
                #bqm.add_variable(i, -np.abs(y[i]))
                #bqm.add_variable(i, np.abs(K[i, i]))
                bqm.add_variable(i, y_train[i])
                
            elif i > j:
                bqm.add_interaction(i, j, 10 *np.abs(K[i, j] ))
                #bqm.add_interaction(i, j, -np.abs(np.linalg.norm(X[i] - X[j])))
                #print(np.abs(stats.pearsonr(X[i] ,X[j]))
                #bqm.add_interaction(i, j, np.abs(stats.pearsonr(X[i] ,X[j])))
    bqm.update(dimod.generators.combinations(list(range(len(K))), k, strength=1000))            
    sampler = neal.SimulatedAnnealingSampler()
    sample_ = sampler.sample(bqm)
    sample= sample_.first.sample
    
    #filter data by coefficients
    K_idx = np.where(np.array(list(sample.values())) ==1)
    #print(len(K_idx[0]))
    #print(K_idx)
    
    X_train_inducing, y_train_inducing = X_train[K_idx], y_train[K_idx]
    
    gp = GaussianProcessRegression(kernel_function=rbf_kernel_)
    gp.fit(X_train_inducing, y_train_inducing)

    y_pred, sigma = gp.predict(X_test)
    
    return y_pred, sigma, K_idx


In [56]:
K = rbf_kernel(X_train, X_train,l2=0.2)
K

array([[1.        , 0.16015667, 0.14717596, ..., 0.17788292, 0.11876194,
        0.10191283],
       [0.16015667, 1.        , 0.62533056, ..., 0.10895947, 0.8302571 ,
        0.0843225 ],
       [0.14717596, 0.62533056, 1.        , ..., 0.08967157, 0.37430578,
        0.08475756],
       ...,
       [0.17788292, 0.10895947, 0.08967157, ..., 1.        , 0.12821896,
        0.43980873],
       [0.11876194, 0.8302571 , 0.37430578, ..., 0.12821896, 1.        ,
        0.05149539],
       [0.10191283, 0.0843225 , 0.08475756, ..., 0.43980873, 0.05149539,
        1.        ]])

In [57]:
def sd_predict(X_train, X_test, y_train, k):

#set up prior -Here we filter down the initial samples as well as xtest
    
    
    #filter data 
    N=len(X_train)
    
    K_idx = random.sample(list(range(N)),k)
    
    X_train_inducing, y_train_inducing = X_train[K_idx], y_train[K_idx]
    
    gp = GaussianProcessRegression(kernel_function=rbf_kernel_)
    gp.fit(X_train_inducing, y_train_inducing)

    #X_test = np.array([[1.5], [2.5]])
    y_pred, sigma = gp.predict(X_test)
    
    return y_pred, sigma, K_idx

 # Example usage:
k =20

#X_train = np.random.rand(100, 1)  
#y_train = np.sin(X_train[:] * 4) + np.cos(X_train[:] * 4) # Sample target values
#X_test = np.random.rand(100, 1)
#y_test = np.sin(X_test[:] * 4) + np.cos(X_test[:] * 4) 



y_pred, s, inducing_indices = sd_predict(X_train, X_test, y_train,  k)

y_pred
#print(y_pred)
# Predict and score



from sklearn.metrics import mean_squared_error, r2_score


print(r2_score(y_pred, y_test),mean_squared_error(y_pred, y_test, squared=False) )

#print(r2_score(y_pred[0], y_test),mean_squared_error(y_pred[0], y_test, squared=False) )   
    

-4.382144827149144 1.9758471602981917


In [58]:
#n_qubits = len(X_train[0])
#n_qubits
#dev_kernel = qml.device("lightning.qubit", wires=n_qubits)

#projector = np.zeros((2**n_qubits, 2**n_qubits))
#projector[0, 0] = 1

#@qml.qnode(dev_kernel, interface="autograd")
#def plq_kernel(x1, x2):
    
    #"""The quantum kernel."""
   # AngleEmbedding(x1, wires=range(n_qubits))
    #qml.adjoint(AngleEmbedding)(x2, wires=range(n_qubits))
   # return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))

#
#def qk_predict(X_train, X_test, y_train,  k):
    
   
   
    
#    inducing_indices = np.random.choice(X_train.shape[0], k, replace=False)
#    X_inducing = X_train[inducing_indices]
#    y_inducing = y_train[inducing_indices]
    
#    X_test_inducing = X_test
    # Use Gaussian Process Regression with inducing points
    
    # Adding a white kernel to account for noise
    
    # Define a quantum feature map
#    feature_dim = X_inducing.shape[0]  # assuming 2-dimensional data for simplicity
#    feature_map = ZZFeatureMap(feature_dimension=feature_dim)

    # Define quantum instance
#    backend = QuantumInstance(Aer.get_backend('statevector_simulator'))

    # Example usage

#    qkr = QuantumKernelRegression(feature_map=feature_map, backend=backend)
#    qkr.fit(X_inducing, y_inducing)

    
#    y_pred, sigma = qkr.predict(X_test_inducing)
    
#    return y_pred, sigma, inducing_indices

#y_pred_qk, sigma_qk, inducing_indices = qk_predict(X_train, X_test, y_train,  k)   

#y_pred_qk

In [59]:
#mean_squared_error(y_pred_qk, y_test, squared=False)


In [60]:
gp = GaussianProcessRegression(kernel_function=rbf_kernel_)
gp.fit(X_train, y_train)

#X_test = np.array([[1.5], [2.5]])
y_pred2, sigma = gp.predict(X_test)
mean_squared_error(y_pred2, y_test, squared=False)

1.375993327465161

In [61]:
"""
rff_out = []
qbo_out = []
sd_out = []

output_dim =100
for i in range(20):
    print(i)
    k =50

    X_train = np.random.rand(100, 1)  
    y_train = np.sin(X_train[:] * 4) + np.cos(X_train[:] * 4) # Sample target values
    X_test = np.random.rand(100, 1)
    y_test = np.sin(X_test[:] * 4) + np.cos(X_test[:] * 4) 
    
    y_pred_sd, s,  inducing_indices = sd_predict(X_train, X_test, y_train,  k)
    y_pred_qbo, _, inducing_indices = qbo_predict(X_train, X_test, y_train,  k)
    y_pred_rff, s,  inducing_indices = rff_predict(X_train, X_test, y_train,output_dim,  k)
    
    rff_out.append(mean_squared_error(y_pred_rff, y_test, squared=False))
    qbo_out.append(mean_squared_error(y_pred_qbo, y_test, squared=False))
    sd_out.append(mean_squared_error(y_pred_sd, y_test, squared=False))
    
    
    
print(np.mean(np.array(qbo_out)))  
print(np.mean(np.array(rff_out)))   
print(np.mean(np.array(sd_out))) 
"""

'\nrff_out = []\nqbo_out = []\nsd_out = []\n\noutput_dim =100\nfor i in range(20):\n    print(i)\n    k =50\n\n    X_train = np.random.rand(100, 1)  \n    y_train = np.sin(X_train[:] * 4) + np.cos(X_train[:] * 4) # Sample target values\n    X_test = np.random.rand(100, 1)\n    y_test = np.sin(X_test[:] * 4) + np.cos(X_test[:] * 4) \n    \n    y_pred_sd, s,  inducing_indices = sd_predict(X_train, X_test, y_train,  k)\n    y_pred_qbo, _, inducing_indices = qbo_predict(X_train, X_test, y_train,  k)\n    y_pred_rff, s,  inducing_indices = rff_predict(X_train, X_test, y_train,output_dim,  k)\n    \n    rff_out.append(mean_squared_error(y_pred_rff, y_test, squared=False))\n    qbo_out.append(mean_squared_error(y_pred_qbo, y_test, squared=False))\n    sd_out.append(mean_squared_error(y_pred_sd, y_test, squared=False))\n    \n    \n    \nprint(np.mean(np.array(qbo_out)))  \nprint(np.mean(np.array(rff_out)))   \nprint(np.mean(np.array(sd_out))) \n'

In [62]:
def qbo_optimize(maxmin_bounds,num_calls, f, num_start, dimension, k):
    noise_var=1e-9
    
    #generate intial samples to warm start optimizer, given by num_start
    #xsamples=np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (num_start, dimension))
   # print(xsamples)
    try:
        xsamples=np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (num_start, dimension))
    except:
        xsamples=np.random.uniform(maxmin_bounds[0], maxmin_bounds[1], size = (num_start, dimension))

    #Get obj function values to train surrogate
    ysamples=np.array([f(x) for x in xsamples])
    print(ysamples)
   
    for i in range(num_calls):
        #Fit the surrogate model.
        #k = int(len(xsamples) * 0.5)
        #mean, std = qesgpr_fit_predict(xsamples, ysamples, noise_variance=1e-6, k=k)
        #Use fit2 and predict2 functions to use quantum filters for both data, x, xtest
        #fit prior
        k = int(len(xsamples) * 0.5)
        #L, x_star, y_star = qesgpr_fit2(xsamples, ysamples, noise_var, k)
        #L = qesgpr_fit(xsamples, ysamples, noise_var, k)
        #print(x_star.shape, y_star.shape)
        #yhat, _ = qesgpr_predict2(L, x_star, xsamples, y_star, noise_var, k)
        #yhat, _ = qesgpr_predict(L,xsamples, ysamples, noise_var, k)
        mean, std, inducing_indices = qbo_predict(xsamples, xsamples, ysamples, k)
        
        #predict from posterior
        best = np.max(mean)
        #xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (300, dimension))
        try:
            xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (10, dimension))
        except:
            xtest = np.random.uniform(maxmin_bounds[0], maxmin_bounds[1],size = (10, dimension))
        #mean, std = qesgpr_predict2(L,x_star, xtest, y_star, noise_var, k)
        #mean, std = qesgpr_predict(L, xtest, yhat, noise_var, k)
       # print(xtest)
        mean, std, inducing_indices = qbo_predict(xsamples, xtest, ysamples, k)
        #best = np.max(mean)
        mean, std = np.mean(mean), np.mean(std)
        #print(mean, std)

        #Find new x using acquisition function.
        new_x= ei_acquisition(best, mean, std, xtest)
        #new_x= ucb_acquisition(mean, std, xtest)
        new_y= f(new_x)
        #print(new_y)

        #update
        xsamples=np.vstack((xsamples, new_x))
        ysamples=np.hstack((ysamples, new_y))
        print("qbo: ", ysamples[np.argmin(ysamples)])
    return ysamples[np.argmin(ysamples)], xsamples[np.argmin(ysamples)]
     

In [63]:
#n_qubits = len(X_train[0])
#n_qubits
#dev_kernel = qml.device("lightning.qubit", wires=n_qubits)

#projector = np.zeros((2**n_qubits, 2**n_qubits))
#projector[0, 0] = 1

#@qml.qnode(dev_kernel, interface="autograd")
#def plq_kernel(x1, x2):
    
    #"""The quantum kernel."""
   # AngleEmbedding(x1, wires=range(n_qubits))
    #qml.adjoint(AngleEmbedding)(x2, wires=range(n_qubits))
   # return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))


def qk_predict(X_train, X_test, y_train,  k):
    
    km =3
    
    
    
    
    #filter data by coefficients
    K_idx = np.random.choice(X_train.shape[1], km, replace=False)
   
    #print(list(K_idx))
    inducing_indices = np.random.choice(X_train.shape[0], k, replace=False)
    #print(X_train[inducing_indices][:, K_idx].reshape(k, km))
    X_inducing = X_train[inducing_indices][:, K_idx].reshape(k, km)
    y_inducing = y_train[inducing_indices]
    
    X_test_inducing = X_test[ :, list(K_idx)].reshape(len(X_test), km)
    # Use Gaussian Process Regression with inducing points
    
    # Adding a white kernel to account for noise
    #gpr = GaussianProcessRegressor(kernel=RBF(), random_state=42)
    #gpr.fit(X_inducing, y_train[inducing_indices])
    #y_pred= gpr.predict(X_test_transformed)
    #L = qesgpr_fit(X_inducing, y_train[inducing_indices], noise_variance=0.2, k=k)
    #y_pred = qesgpr_predict_vanilla(L,  X_test_transformed, y_train[inducing_indices], noise_variance=0.2, k=k)
    
    # Define a quantum feature map
    feature_dim = X_inducing.shape[1] 
    feature_map = ZZFeatureMap(feature_dimension=feature_dim)

    # Define quantum instance
    backend = QuantumInstance(Aer.get_backend('statevector_simulator'))

    # Example usage
    #X_train = np.array([[1., 1.], [2., 2.], [3., 3.]])
    #y_train = np.array([1., 3., 2.])

    qkr = QuantumKernelRegression(feature_map=feature_map, backend=backend)
    qkr.fit(X_inducing, y_inducing)

    #X_test = np.array([[1.5, 1.5], [2.5, 2.5]])
    y_pred, sigma = qkr.predict(X_test_inducing)
    
    return y_pred, sigma, inducing_indices

y_pred_qk, sigma_qk, inducing_indices = qk_predict(X_train, X_test, y_train,  k)   

y_pred_qk

array([[-0.36838867],
       [ 1.53516536],
       [ 2.79677821],
       [ 2.54886678],
       [ 6.40840671],
       [-0.06761185],
       [ 4.73506144],
       [ 1.19819461],
       [-2.70330423],
       [ 0.31397152],
       [ 2.11986989],
       [-2.11150714],
       [ 2.77190636],
       [-1.60094264],
       [ 5.13871104],
       [ 5.39180489],
       [ 2.18778874],
       [-0.28876119],
       [-2.35071251],
       [ 2.03675108],
       [ 0.27186119],
       [ 0.27022916],
       [-0.31745762],
       [ 2.09680737],
       [ 4.00609326],
       [-2.66872401],
       [ 4.47418147],
       [ 3.28704016],
       [ 0.32247199],
       [-0.70171   ],
       [ 0.29532028],
       [ 2.30082504],
       [ 1.63067521],
       [ 2.86181906],
       [-3.06004333],
       [-2.66580686],
       [ 0.52659724],
       [ 1.04239306],
       [-0.666716  ],
       [-0.70151477],
       [ 0.69294423],
       [ 5.44174097],
       [ 1.4645033 ],
       [-1.57184659],
       [ 2.12456436],
       [ 3

In [64]:
#n_qubits = len(X_train[0])
#n_qubits
#dev_kernel = qml.device("lightning.qubit", wires=n_qubits)

#projector = np.zeros((2**n_qubits, 2**n_qubits))
#projector[0, 0] = 1

#@qml.qnode(dev_kernel, interface="autograd")
#def plq_kernel(x1, x2):
    
    #"""The quantum kernel."""
   # AngleEmbedding(x1, wires=range(n_qubits))
    #qml.adjoint(AngleEmbedding)(x2, wires=range(n_qubits))
   # return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))


def qk_predict_qbo(X_train, X_test, y_train,  k):
    
    km =3
    
    K = rbf_kernel(X_train, X_train,l2=0.2)
    #print(K, y_train)
    bqm = dimod.BinaryQuadraticModel.empty(dimod.BINARY)
    for i in range(len(K)):
        for j in range(len(K)):
            if i == j:
                #bqm.add_variable(i, -np.abs(y[i]))
                #bqm.add_variable(i, np.abs(K[i, i]))
                bqm.add_variable(i, y_train[i])
                
            elif i > j:
                bqm.add_interaction(i, j, 10 * np.abs(K[i, j] ))
                #bqm.add_interaction(i, j, -np.abs(np.linalg.norm(X[i] - X[j])))
                #print(np.abs(stats.pearsonr(X[i] ,X[j]))
                #bqm.add_interaction(i, j, np.abs(stats.pearsonr(X[i] ,X[j])))
    bqm.update(dimod.generators.combinations(list(range(len(K))), k, strength=10000))            
    sampler = neal.SimulatedAnnealingSampler()
    sample_ = sampler.sample(bqm)
    sample= sample_.first.sample
    
    #filter data by coefficients
    inducing_indices = np.where(np.array(list(sample.values())) ==1)
    bqm = dimod.BinaryQuadraticModel.empty(dimod.BINARY)
    
    for i in range(len(X_train[0])):
        for j in range(len(X_train[0])):
            if i == j:
                #bqm.add_variable(i, -np.abs(y[i]))
                #bqm.add_variable(i, np.abs(K[i, i]))
                bqm.add_variable(i, np.abs(stats.pearsonr(y_train.flatten(),X_train[:,i].flatten())[0]))
                
            elif i > j:
                bqm.add_interaction(i, j, np.abs(stats.pearsonr(X_train[:,i] ,X_train[:,j])[0]))
                #bqm.add_interaction(i, j, -np.abs(np.linalg.norm(X[i] - X[j])))
                #print(np.abs(stats.pearsonr(X[i] ,X[j]))
                #bqm.add_interaction(i, j, np.abs(stats.pearsonr(X[i] ,X[j])))
                
    bqm.update(dimod.generators.combinations(list(range(len(X_train[0]))), km, strength=10000))          
    sampler = neal.SimulatedAnnealingSampler()
    sample_2 = sampler.sample(bqm)
    sample1= sample_2.first.sample
    
    
    K_idx  = np.where(np.array(list(sample1.values())) ==1)[:km]
    inducing_indices =inducing_indices[:k]
    #print(len(K_idx[0]), len(inducing_indices[0]))
    #filter data by coefficients
    #K_idx = np.random.choice(X_train.shape[1], km, replace=False)
   
    #print(list(K_idx))
    #inducing_indices = np.random.choice(X_train.shape[0], k, replace=False)
    #print(X_train[inducing_indices][:, K_idx].reshape(k, km))
    X_inducing = X_train[inducing_indices][:, K_idx].reshape(len(inducing_indices[0]), len(K_idx[0]))
    y_inducing = y_train[inducing_indices]
    
    X_test_inducing = X_test[ :, list(K_idx)].reshape(len(X_test), len(K_idx[0]))
    # Use Gaussian Process Regression with inducing points
    
    # Adding a white kernel to account for noise
    #gpr = GaussianProcessRegressor(kernel=RBF(), random_state=42)
    #gpr.fit(X_inducing, y_train[inducing_indices])
    #y_pred= gpr.predict(X_test_transformed)
    #L = qesgpr_fit(X_inducing, y_train[inducing_indices], noise_variance=0.2, k=k)
    #y_pred = qesgpr_predict_vanilla(L,  X_test_transformed, y_train[inducing_indices], noise_variance=0.2, k=k)
    
    # Define a quantum feature map
    feature_dim = X_inducing.shape[0] 
    feature_map = ZZFeatureMap(feature_dimension=feature_dim)

    # Define quantum instance
    backend = QuantumInstance(Aer.get_backend('statevector_simulator'))

    # Example usage
    #X_train = np.array([[1., 1.], [2., 2.], [3., 3.]])
    #y_train = np.array([1., 3., 2.])

    qkr = QuantumKernelRegression(feature_map=feature_map, backend=backend)
    qkr.fit(X_inducing, y_inducing)

    #X_test = np.array([[1.5, 1.5], [2.5, 2.5]])
    y_pred, sigma = qkr.predict(X_test_inducing)
    
    return y_pred, sigma, inducing_indices

y_pred_qk, sigma_qk, inducing_indices = qk_predict_qbo(X_train, X_test, y_train,  k=5)   

y_pred_qk

array([[ 2.02446267],
       [ 1.47119832],
       [ 1.62341107],
       [ 0.32754876],
       [ 0.89692977],
       [ 1.47763442],
       [ 0.12076916],
       [ 0.42991499],
       [ 3.76097787],
       [ 0.60249473],
       [ 3.26101293],
       [ 2.23983884],
       [ 2.05541732],
       [ 1.58757687],
       [-0.11138529],
       [-0.47985863],
       [ 1.56353544],
       [ 2.44063821],
       [ 3.62215343],
       [ 0.44537805],
       [ 1.99947192],
       [ 0.27467551],
       [ 0.53031797],
       [-0.20928027],
       [ 0.41302915],
       [-0.29800965],
       [-0.15945969],
       [ 0.30295568],
       [ 1.97587871],
       [ 1.02258187],
       [ 0.55037417],
       [ 0.19993276],
       [ 0.40675887],
       [ 0.54438905],
       [ 5.24742169],
       [ 3.61491217],
       [-0.90381537],
       [ 0.50931786],
       [ 1.22355305],
       [ 2.69866979],
       [ 0.30591747],
       [ 2.67214741],
       [-0.06463453],
       [-0.70294219],
       [ 0.11539644],
       [-1

In [65]:
from sklearn.svm import SVC, SVR
from sklearn.cluster import SpectralClustering
from sklearn.metrics import normalized_mutual_info_score

import qiskit
from qiskit import BasicAer
from qiskit.circuit.library import ZZFeatureMap
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.datasets import ad_hoc_data



In [66]:
from sklearn.ensemble import GradientBoostingRegressor as GBR

def qkgprbo_optimize(maxmin_bounds, num_calls, f, num_start, dimension, k):
    #surrogate = SVR(kernel=q_kernel.evaluate)
    #surrogate = GBR()
    try:
        xsamples=np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (num_start, dimension))
    except:
        xsamples=np.random.uniform(maxmin_bounds[0], maxmin_bounds[1],size = (num_start, dimension))
    #generate intial samples to warm start optimizer, given by num_start
    #xsamples=np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (num_start, dimension))
    #print(f.min_bounds)
    #print(f(xsamples[0]))

    #Get obj function values to train surrogate
    ysamples=np.array([f(x) for x in xsamples])
    #print(ysamples)
    #print(ysamples)
    for i_ in range(num_calls):
        #Fit the surrogate function.
        #surrogate.fit(xsamples, ysamples)
        #yhat = surrogate.predict(xsamples)
        k = int(len(xsamples)* .5)
        
        mean, std, inducing_indices =qk_predict(xsamples, xsamples, ysamples,  k)
    
        best = np.max(mean)
        #print(best)
        
        
        #predict mean, std from xtest
        #xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (300, dimension))
        try:
            xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (10, dimension))
        except:
            xtest = np.random.uniform(maxmin_bounds[0], maxmin_bounds[1],size = (10, dimension))
        #mean, std = surrogate.predict(xtest, return_std=True)
        #y_bar = surrogate.predict(xtest)
        mean, std, inducing_indices = qk_predict(X_train, X_test, y_train,  k)
        
        mean, std = np.mean(mean), np.mean(std)
        #print(mean, std)

        #Find new x using acquisition function.
        new_x= ei_acquisition(best, mean, std, xtest)
        #new_x= ucb_acquisition(mean, std, xtest)
        #print(new_x)
        new_y= f(new_x)
        #print(new_y)

        #update
        xsamples=np.vstack((xsamples, new_x))
        ysamples=np.hstack((ysamples, new_y))
        
        print("qkgpr_bo: ",ysamples[np.argmin(ysamples)])
    return ysamples[np.argmin(ysamples)], xsamples[np.argmin(ysamples)]

In [67]:
from sklearn.ensemble import GradientBoostingRegressor as GBR

def qkgprbo_qubo_optimize(maxmin_bounds, num_calls, f, num_start, dimension, k):
    #surrogate = SVR(kernel=q_kernel.evaluate)
    #surrogate = GBR()
    try:
        xsamples=np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (num_start, dimension))
    except:
        xsamples=np.random.uniform(maxmin_bounds[0], maxmin_bounds[1],size = (num_start, dimension))
    #generate intial samples to warm start optimizer, given by num_start
    #xsamples=np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (num_start, dimension))
    #print(f.min_bounds)
    #print(f(xsamples[0]))

    #Get obj function values to train surrogate
    ysamples=np.array([f(x) for x in xsamples])
    #print(ysamples)
    #print(ysamples)
    for i_ in range(num_calls):
        #Fit the surrogate function.
        #surrogate.fit(xsamples, ysamples)
        #yhat = surrogate.predict(xsamples)
        k = int(len(xsamples)* .5)
        
        mean, std, inducing_indices =qk_predict_qbo(xsamples, xsamples, ysamples,  k)
    
        best = np.max(mean)
        #print(best)
        
        
        #predict mean, std from xtest
        #xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (300, dimension))
        try:
            xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (10, dimension))
        except:
            xtest = np.random.uniform(maxmin_bounds[0], maxmin_bounds[1],size = (10, dimension))
        #mean, std = surrogate.predict(xtest, return_std=True)
        #y_bar = surrogate.predict(xtest)
        mean, std, inducing_indices = qk_predict_qbo(X_train, X_test, y_train,  k)
        
        mean, std = np.mean(mean), np.mean(std)
        #print(mean, std)

        #Find new x using acquisition function.
        new_x= ei_acquisition(best, mean, std, xtest)
        #new_x= ucb_acquisition(mean, std, xtest)
        #print(new_x)
        new_y= f(new_x)
        #print(new_y)

        #update
        xsamples=np.vstack((xsamples, new_x))
        ysamples=np.hstack((ysamples, new_y))
        
        print("qkgpr_bo: ",ysamples[np.argmin(ysamples)])
    return ysamples[np.argmin(ysamples)], xsamples[np.argmin(ysamples)]

In [68]:
 xsamples=np.random.uniform(maxmin_bounds[0], maxmin_bounds[1],size = (10, 25))
list(xsamples[0])

[-0.5744602123721929,
 -3.2984180394545515,
 3.2322778713240274,
 3.5174168389997815,
 -4.375930413205321,
 -3.578537848797364,
 0.04462369575823377,
 -3.5559118297585606,
 2.496614980808878,
 -1.1556597928220107,
 2.5436986358626203,
 -0.86942581273561,
 1.1901391159394468,
 -3.1092927395155314,
 -4.201645455478365,
 -2.0552151455947785,
 -3.9631012093049502,
 3.55548935093579,
 -0.029729268312561175,
 2.68252573296228,
 1.9853791825344071,
 1.0021440163474482,
 -4.662372101094871,
 0.8495913495504057,
 -2.7211792930995937]

In [69]:
test_suite = [Ackley]

problem_dimension = 50  
maxmin_bounds = (5.,-5.)
k = 3
for black_box_function in test_suite:
    print(black_box_function)
   
    
    try:
        f=black_box_function(num_variables=problem_dimension)
    except:
        f=black_box_function()
        
    #print(f([10,10,10,10]))
    res = qkgprbo_optimize(maxmin_bounds,100,f, num_start=10,dimension=problem_dimension, k=k)
print(res)



<class 'optproblems.continuous.Ackley'>
qkgpr_bo:  10.085815919616984
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qkgpr_bo:  9.518695811321264
qk

In [70]:


def sd_optimize(maxmin_bounds,num_calls, f, num_start, dimension, k):
    noise_var=1e-9
    
    
    #generate intial samples to warm start optimizer, given by num_start
    #xsamples=np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (num_start, dimension))
   # print(xsamples)
    try:
        xsamples=np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (num_start, dimension))
    except:
        xsamples=np.random.uniform(maxmin_bounds[0], maxmin_bounds[1], size = (num_start, dimension))

    #Get obj function values to train surrogate
    ysamples=np.array([f(x) for x in xsamples])
    #print(ysamples)
  
    
    for i in range(num_calls):
        #Fit the surrogate model.
        #k = int(len(xsamples) * 0.5)
        #mean, std = qesgpr_fit_predict(xsamples, ysamples, noise_variance=1e-6, k=k)
        #Use fit2 and predict2 functions to use quantum filters for both data, x, xtest
        #fit prior
        k = int(len(xsamples) * 0.5)
        num_start = len(xsamples)
        
        y_pred, s, inducing_indices = sd_predict(X_train, X_train, y_train,  k)
        best = np.max(y_pred)
        #xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (300, dimension))
        try:
            xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (10, dimension))
        except:
            xtest = np.random.uniform(maxmin_bounds[0], maxmin_bounds[1],size = (10, dimension))
        
        y_pred, std, inducing_indices = sd_predict(X_train, X_test, y_train,  k)
        #Find new x using acquisition function.
        new_x= ei_acquisition(best, np.mean(y_pred), np.mean(std), xtest)
        #new_x= ucb_acquisition(Y_star_mean, Y_star_std, xtest)
        new_y= f(new_x)
        #print(new_y)

        #update
        xsamples=np.vstack((xsamples, new_x))
        ysamples=np.hstack((ysamples, new_y))
        print("sd: ", ysamples[np.argmin(ysamples)])
    return ysamples[np.argmin(ysamples)], xsamples[np.argmin(ysamples)]

In [71]:
import sparse_GP as sGP
def vgpr_predict(X_train, X_test, y_train,  k):
    L = None
    num_start= len(X_train)
    noise_var=0.001
        
    while L == None:

        (L, sigma, K, C, inv_C, Xs, Ys, LB_best, elapsed_time) = sGP.train(0.2, noise_var,
                                                           X_train, y_train, num_start, k,
                                                           5)


       # Make predictions
    N_Star = len(X_test)               # No. points where we make predictions
    Y_star_mean = np.zeros(N_Star)     # Initialise GP mean predictions
    Y_star_std = np.zeros(N_Star) 

 

    #predict from posterior
    #best = np.max(Y_star_mean)
    #dimension = X_train.shape[1]
    #xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (300, dimension))
    #try:
    #    xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (10, dimension))
    #except:
    #    xtest = np.random.uniform(maxmin_bounds[0], maxmin_bounds[1],size = (10, dimension))
    for indx, x_star_ in enumerate(X_test):

        try:
            Y_star_mean[indx], Y_star_std[indx] = sGP.predict(Xs, x_star_, L, sigma,
                                                    Ys, K, C, inv_C, k)
        except:
            pass

    
    
    
    
    return Y_star_mean, Y_star_std

y_pred_vgpr, sigma_vgpr = vgpr_predict(X_train, X_test, y_train,  k)   

y_pred_vgpr

array([-3.05493811e-01,  2.56502674e-02,  1.01299412e-02,  1.97357118e-01,
        1.98508093e-01, -5.71489652e-05,  6.22844190e-02,  2.84788368e-01,
       -1.83472280e-03, -1.26113205e-04,  8.86236022e-03,  6.64873072e-02,
       -2.43951155e-03,  7.69893845e-02,  8.27305131e-01, -1.60719609e-02,
        7.58800970e-03,  1.93601230e-03,  9.52582512e-04, -7.51365914e-02,
       -3.58811385e-02, -4.51436568e-02,  4.90086191e-04,  1.65019980e-02,
       -4.92917779e-03,  1.39298135e-02,  4.52176383e-03, -9.24184861e-02,
        1.57390348e-02, -4.25512807e-05, -1.28923638e-02,  1.89187655e-02,
        1.88730105e-04,  4.94323899e-01,  1.16859419e-01,  5.46620640e-01,
        4.58667261e-04,  1.44816558e-03,  1.36938709e-01,  7.91276028e-02,
       -7.54592145e-04,  2.35931477e+00, -3.24981458e-02, -1.09098826e-02,
        7.61852120e-02,  6.72799318e-04,  3.21462802e-01, -2.53502714e-01,
       -2.10895457e-02, -3.92622313e-05])

In [72]:
import sparse_GP as sGP

def vgpr_optimize(maxmin_bounds,num_calls, f, num_start, dimension, k):
    noise_var=1e-2
    
    
    #generate intial samples to warm start optimizer, given by num_start
    #xsamples=np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (num_start, dimension))
   # print(xsamples)
    try:
        xsamples=np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (num_start, dimension))
    except:
        xsamples=np.random.uniform(maxmin_bounds[0], maxmin_bounds[1], size = (num_start, dimension))

    #Get obj function values to train surrogate
    ysamples=np.array([f(x) for x in xsamples])
    #print(ysamples)
  
    
    for i in range(num_calls):
        #Fit the surrogate model.
        #k = int(len(xsamples) * 0.5)
        #mean, std = qesgpr_fit_predict(xsamples, ysamples, noise_variance=1e-6, k=k)
        #Use fit2 and predict2 functions to use quantum filters for both data, x, xtest
        #fit prior
        #k = int(len(xsamples) * 0.5)
        num_start = len(xsamples)
        
        L = None
        
        while L == None:
            
            (L, sigma, K, C, inv_C, Xs, Ys, LB_best, elapsed_time) = sGP.train(0.2, noise_var,
                                                               xsamples, ysamples, num_start, k,
                                                               1)
        

           # Make predictions
        N_Star = len(xsamples)               # No. points where we make predictions
        Y_star_mean = np.zeros(N_Star)     # Initialise GP mean predictions
        Y_star_std = np.zeros(N_Star) 
        
        for indx, x_star_ in enumerate(xsamples):
            try:
                Y_star_mean[indx], Y_star_std[indx] = sGP.predict(Xs, x_star_, L, sigma,
                                                        Ys, K, C, inv_C, k)
            except:
                pass

        #predict from posterior
        best = np.max(Y_star_mean)
        #xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (300, dimension))
        try:
            xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (10, dimension))
        except:
            xtest = np.random.uniform(maxmin_bounds[0], maxmin_bounds[1],size = (10, dimension))
        for indx, x_star_ in enumerate(xtest):
            
            try:
                Y_star_mean[indx], Y_star_std[indx] = sGP.predict(Xs, x_star_, L, sigma,
                                                        Ys, K, C, inv_C, k)
            except:
                pass

        #Find new x using acquisition function.
        try:
            new_x= ei_acquisition(best,  Y_star_mean, Y_star_std, xtest)
        except:
            pass
        #new_x= ucb_acquisition(Y_star_mean, Y_star_std, xtest)
        new_y= f(new_x)
        #print(new_y)

        #update
        xsamples=np.vstack((xsamples, new_x))
        ysamples=np.hstack((ysamples, new_y))
        print("vgpr-bo: ", ysamples[np.argmin(ysamples)])
    return ysamples[np.argmin(ysamples)], xsamples[np.argmin(ysamples)]

In [ ]:
import pyGPs


def fitc_gpr_optimize(maxmin_bounds,num_calls, f, num_start, dimension, k):
    noise_var=1e-9
    
    
    #generate intial samples to warm start optimizer, given by num_start
    #xsamples=np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (num_start, dimension))
   # print(xsamples)
    try:
        xsamples=np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (num_start, dimension))
    except:
        xsamples=np.random.uniform(maxmin_bounds[0], maxmin_bounds[1], size = (num_start, dimension))

    #Get obj function values to train surrogate
    ysamples=np.array([f(x) for x in xsamples])
    #print(ysamples)
    pygps_m = pyGPs.GPR_FITC()
    pygps_m.useLikelihood('Laplace')
    pygps_m.useInference('EP')
    
    for i in range(num_calls):
        #Fit the surrogate model.
        #k = int(len(xsamples) * 0.5)
        #mean, std = qesgpr_fit_predict(xsamples, ysamples, noise_variance=1e-6, k=k)
        #Use fit2 and predict2 functions to use quantum filters for both data, x, xtest
        #fit prior
        #k = int(len(xsamples) * 0.5)
        num_start = len(xsamples)
        pygps_m.setData(xsamples,ysamples, value_per_axis=k)
        pygps_m.optimize(numIterations=1)

        y_hat = pygps_m.predict(xsamples)
        y_hat_mean = y_hat[0]
        y_hat_std = y_hat[1]
        
        
        #predict from posterior
        best = np.max(y_hat_mean)
        #print(best)
        #xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (300, dimension))
        try:
            xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (10, dimension))
        except:
            xtest = np.random.uniform(maxmin_bounds[0], maxmin_bounds[1],size = (10, dimension))
        
        y_star = pygps_m.predict(xtest)
        y_star_mean = np.mean(y_star[0].reshape(-1, 1))
        y_star_std = np.mean(y_star[1].reshape(-1, 1))
        #print(y_star_std, y_star_mean)

        #Find new x using acquisition function.
        new_x= ei_acquisition(best,  y_star_mean, y_star_std, xtest)
        #new_x= ucb_acquisition(y_star_mean, y_star_std, xtest)
        new_y= f(new_x)
        #print(new_y)

        #update
        xsamples=np.vstack((xsamples, new_x))
        ysamples=np.hstack((ysamples, new_y))
        print("fitc-bo: ", ysamples[np.argmin(ysamples)])
    return ysamples[np.argmin(ysamples)], xsamples[np.argmin(ysamples)]

In [ ]:
import sparse_GP as sGP
test_suite = [Ackley]
k=5
problem_dimension = 20    
maxmin_bounds = (5.,-5.)
for black_box_function in test_suite:
    print(black_box_function)
   
    
    try:
        f=black_box_function(num_variables=problem_dimension)
    except:
        f=black_box_function()
        
    #print(f([10,10,10,10]))
    res = sd_optimize(maxmin_bounds,100,f, num_start=10,dimension=problem_dimension, k=k)
    #print(res)

# Tests

In [ ]:

"""
import numpy as np
from scipy.optimize import minimize

class GaussianProcessRegression:
    def __init__(self, kernel_function, noise_variance=1e-10):
        self.kernel_function = kernel_function
        self.noise_variance = noise_variance

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        K = self.kernel_function(X, X) + self.noise_variance * np.eye(len(X))
        self.L = np.linalg.cholesky(K + 1e-10 * np.eye(len(X)))
        self.alpha = np.linalg.solve(self.L.T, np.linalg.solve(self.L, y))

    def predict(self, X):
        K_trans = self.kernel_function(X, self.X_train)
        mean = np.dot(K_trans, self.alpha)
        v = np.linalg.solve(self.L, K_trans.T)
        variance = self.kernel_function(X, X) - np.dot(K_trans, v)
        return mean, variance

class BayesianOptimization:
    def __init__(self, func, bounds, kernel_function, n_pre_samples=3):
        self.func = func
        self.bounds = bounds
        self.gp = GaussianProcessRegression(kernel_function)
        self.X_train = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_pre_samples, bounds.shape[0]))
        self.y_train = [self.func(x) for x in self.X_train]
        self.gp.fit(self.X_train, self.y_train)

    def acquisition_function(self, x_new):
        mean, variance = self.gp.predict(x_new.reshape(1, -1))
        return -(mean - np.min(self.y_train))

    def optimize(self, n_iterations=100):
        for iteration in range(n_iterations):
            result = minimize(fun=self.acquisition_function, x0=np.mean(self.bounds, axis=1), bounds=self.bounds,method='L-BFGS-B')
            x_new = result.x
            y_new = self.func(x_new)

            self.X_train = np.vstack([self.X_train, x_new])
            self.y_train = np.append(self.y_train, y_new)
            self.gp.fit(self.X_train, self.y_train)
            idx_best = np.argmin(self.y_train)
            print(self.y_train[idx_best])

        #idx_best = np.argmin(self.y_train)
        return self.X_train[idx_best], self.y_train[idx_best]

# Kernel definition
def rbf_kernel_(X1, X2, l=1.0, sigma_f=1.0):
    sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    return sigma_f**2 * np.exp(-0.5 / l**2 * sqdist)

# Generic n-dimensional objective function
def objective_function(X):
    return (np.sin(5 * np.pi * X)).sum(axis=1)


# Specify bounds for n-dimensional data.
n_dimensions = 3  # Change this to your desired number of dimensions
bounds = np.array([[-2, 2]] * n_dimensions)
f = Ackley(num_variables=n_dimensions)
bo = BayesianOptimization(func=f, bounds=bounds, kernel_function=rbf_kernel_)
optimal_x, optimal_y = bo.optimize()

print("Optimal X:", optimal_x)
print("Optimal Y:", optimal_y)
"""

In [ ]:
"""
def rff_predict(X_train, X_test, y_train, k):

#set up prior -Here we filter down the initial samples as well as xtest
    K = rbf_kernel(X_train, X_train,l2=0.2)
    #print(K, y_train)
    bqm = dimod.BinaryQuadraticModel.empty(dimod.BINARY)
    for i in range(len(K)):
        for j in range(len(K)):
            if i == j:
                #bqm.add_variable(i, -np.abs(y[i]))
                #bqm.add_variable(i, np.abs(K[i, i]))
                bqm.add_variable(i, -np.abs(y_train[i]))
                
            elif i > j:
                bqm.add_interaction(i, j, np.abs(K[i, j] ))
                #bqm.add_interaction(i, j, -np.abs(np.linalg.norm(X[i] - X[j])))
                #print(np.abs(stats.pearsonr(X[i] ,X[j]))
                #bqm.add_interaction(i, j, np.abs(stats.pearsonr(X[i] ,X[j])))
    bqm.update(dimod.generators.combinations(list(range(len(K))), k, strength=1000))            
    sampler = neal.SimulatedAnnealingSampler()
    sample_ = sampler.sample(bqm)
    sample= sample_.first.sample
    
    #filter data by coefficients
    K_idx = np.where(np.array(list(sample.values())) ==1)
    #print(len(K_idx[0]))
    #print(K_idx)
    
    X_train_inducing, y_train_inducing = X_train[K_idx], y_train[K_idx]
    
    gp = GaussianProcessRegression(kernel_function=rbf_kernel_)
    gp.fit(X_train_inducing, y_train_inducing)

    y_pred, sigma = gp.predict(X_test)
    
    return y_pred, sigma, K_idx

 # Example usage:
k =20

#X_train = np.random.rand(100, 1)  
#y_train = np.sin(X_train[:] * 4) + np.cos(X_train[:] * 4) # Sample target values
#X_test = np.random.rand(50, 1)
#y_test = np.sin(X_test[:] * 4) + np.cos(X_test[:] * 4) 



y_pred, sigma, inducing_indices = qbo_predict(X_train, X_test, y_train,  k)

y_pred
print(y_pred)
# Predict and score



from sklearn.metrics import mean_squared_error, r2_score


print(r2_score(y_pred, y_test),mean_squared_error(y_pred, y_test, squared=False) )

#print(r2_score(y_pred, y_test),mean_squared_error(y_pred, y_test, squared=False) )   
 """   

In [ ]:
"""

# Example usage:
X_train = np.random.rand(100, 2)  
y_train = np.sin(X_train[:, 0] * 4) + np.cos(X_train[:, 1] * 4)  # Sample target values
X_test = np.random.rand(50, 2)

L, X_star, y_star, K_idx = gpr_subset_fit(X_train, y_train, noise_variance=0.2, k=k)

#print(L, X_star, y_star, K_idx)
y_pred, _ = qesgpr_predict_vanilla(L, X_test, y_star, noise_variance=0.2, k=k)

y_pred
"""

In [ ]:
"""
import numpy as np
rand = np.random.randint(low=1, high=10000)
print(rand)
np.random.seed(rand)
num_calls =50
problem_dimension = 3
num_start=20
k = int(num_start * .2)

seed=1234
adhoc_feature_map = ZZFeatureMap(feature_dimension=problem_dimension, reps=2, entanglement="linear")

adhoc_backend = QuantumInstance(
    BasicAer.get_backend("qasm_simulator"), shots=1024, seed_simulator=seed, seed_transpiler=seed
)

q_kernel = QuantumKernel(feature_map=adhoc_feature_map, quantum_instance=adhoc_backend)


for black_box_function in test_suite:
    print(black_box_function)
   
    
    f=black_box_function(num_variables=problem_dimension)
    #print(f([10,10,10,10]))
    
    
    res = bo_optimize(num_calls,f=f, num_start=num_start,dimension=problem_dimension)
    print(res)
    
    res = qbo_optimize(num_calls,f=f, num_start=num_start,dimension=problem_dimension, k=k)
    print(res)
    
    #res = qsvmbo_optimize(num_calls,f=f, num_start=num_start,dimension=problem_dimension)
    #print(res)
    
"""



So, this seems to work. What we have done is used regularization terms 
in the kernel matrices to select a subset X to approximate a sparse GPR. This reduces the
complexity by our approximation ratio of $|X|/2$
There may be a trade off for accuracy in regards to reduction in complexity. Additionally, as the size of the $Xtest$ set grows, the quantum-enhanced slows down, I believe this is due to the annealing time for simulated annealing. In practice currently this would take longer due to calls to a QPU. However it would be advantageous to run more tests, as well as apply to HPO and see if there are some small performance gains.

~TODO: Create distributions from randoms seeds for test suite for GBO and BO, an see if there is significance in the difference of the distributions~
TODO: Test both on HPO with HQNN and Classical NN


# Experiments

In [73]:
# this is the main training loop- alter the experiment parameters to save results 
from optproblems.continuous import *
import warnings
warnings.filterwarnings('ignore')


from optproblems.continuous import Weierstrass, Schaffer7, LunacekTwoRastrigins, Ackley, Rastrigin, Branin, FletcherPowell, GoldsteinPrice, Griewank, Vincent,Himmelblau,SixHumpCamelback
import pickle
#from mpl_toolkits.mplot3d import Axes3D
#%matplotlib inline



fs = [FletcherPowell, Griewank, Vincent,LunacekTwoRastrigins,Weierstrass, Ackley, Rastrigin ]
#fs = [Ackley, Rastrigin,Griewank, Vincent,LunacekTwoRastrigins,Weierstrass]
#fs = [Weierstrass]
#fs2 = [Branin,GoldsteinPrice,Himmelblau,SixHumpCamelback]

num_experiments = 20
num_calls =90
problem_dimension = 3
num_start=10
k = int(num_start * .5)
output_dim =problem_dimension * 3
rand_seeds = np.random.random_integers(low=1, high=10000, size=(num_experiments, 1))

results = {}

for black_box_function in fs:
    try:
        f =black_box_function(num_variables=problem_dimension, name=str(func.__name__))
    
    except:
       
        f = black_box_function(num_variables=problem_dimension)
        print("no bounds")
    #step_size = 0.1

    #r_min, r_max = 0., 10.
    try:

        r_min, r_max = f.min_bounds[0], f.max_bounds[0]

    except:
        r_min, r_max = -5, 5.
    
    maxmin_bounds = (r_min, r_max)
    print(black_box_function.__name__)
    results[str(black_box_function.__name__)] = {}
    results[str(black_box_function.__name__)]['qbo'] = []
    results[str(black_box_function.__name__)]['bo'] = []
    results[str(black_box_function.__name__)]['sd'] = []
    results[str(black_box_function.__name__)]['qkgpr'] = []
    results[str(black_box_function.__name__)]['qkgpr-qubo'] = []
    results[str(black_box_function.__name__)]['rff'] = []
    results[str(black_box_function.__name__)]['rff-qubo'] = []
    results[str(black_box_function.__name__)]['vgpr-bo'] = []
    
    
    #results[str(black_box_function.__name__)]['bo'] = []
    for idx, seed_ in enumerate(rand_seeds):
        print(idx)
        np.random.seed(seed_)
        f=black_box_function(num_variables=problem_dimension)
        vgpr_bo_res = vgpr_optimize(maxmin_bounds,num_calls=num_calls,f=f,dimension=problem_dimension, num_start=num_start, k=k)
        
        bo_res = bo_optimize(maxmin_bounds,num_calls=num_calls,f=f,dimension=problem_dimension, num_start=num_start)
        qbo_res = qbo_optimize(maxmin_bounds,num_calls=num_calls,f=f,dimension=problem_dimension, num_start=num_start, k=k)
        rff_res = rff_optimize(maxmin_bounds,num_calls=num_calls,f=f,dimension=problem_dimension, num_start=num_start, output_dim=500,k=k)
        rff_qubo_res = rff_qubo_optimize(maxmin_bounds,num_calls=num_calls,f=f,dimension=problem_dimension, num_start=num_start, output_dim=500,k=k)
        sd_res = sd_optimize(maxmin_bounds,num_calls=num_calls,f=f,dimension=problem_dimension, num_start=num_start, k=k)
        qkgpr_res = qkgprbo_optimize(maxmin_bounds,num_calls=num_calls,f=f,dimension=problem_dimension, num_start=num_start, k=k)
        qkgpr_qubo_res = qkgprbo_qubo_optimize(maxmin_bounds,num_calls=num_calls,f=f,dimension=problem_dimension, num_start=num_start, k=k)
        
        
        vgpr_bo_res = vgpr_optimize(maxmin_bounds,num_calls=num_calls,f=f,dimension=problem_dimension, num_start=num_start, k=k)
        
        #fitc_bo_res = fitc_gpr_optimize(maxmin_bounds,num_calls=num_calls,f=f,dimension=problem_dimension, num_start=num_start, k=k)
        
        results[str(black_box_function.__name__)]['qbo'].append(qbo_res[0])
        results[str(black_box_function.__name__)]['bo'].append(bo_res[0])
        results[str(black_box_function.__name__)]['rff'].append(rff_res[0])
        results[str(black_box_function.__name__)]['rff-qubo'].append(rff_qubo_res[0])
        results[str(black_box_function.__name__)]['sd'].append(sd_res[0])
        results[str(black_box_function.__name__)]['qkgpr'].append(qkgpr_res[0])
        results[str(black_box_function.__name__)]['qkgpr-qubo'].append(qkgpr_qubo_res[0])
        
        
        #results[str(black_box_function.__name__)]['fitc-bo'].append(fitc_bo_res[0])
        results[str(black_box_function.__name__)]['vgpr-bo'].append(vgpr_bo_res[0])

        
        
    
print(results)    

no bounds
FletcherPowell
0
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814970978
vgpr-bo:  4125.513814

qkgpr_bo:  2446.1564458220264
qkgpr_bo:  2446.1564458220264
qkgpr_bo:  2446.1564458220264
qkgpr_bo:  2446.1564458220264
qkgpr_bo:  2446.1564458220264
qkgpr_bo:  2446.1564458220264
qkgpr_bo:  2446.1564458220264
qkgpr_bo:  2446.1564458220264
qkgpr_bo:  2446.1564458220264
qkgpr_bo:  2446.1564458220264
qkgpr_bo:  2446.1564458220264
qkgpr_bo:  2446.1564458220264
qkgpr_bo:  2446.1564458220264
qkgpr_bo:  2446.1564458220264
qkgpr_bo:  2446.1564458220264
qkgpr_bo:  2446.1564458220264
qkgpr_bo:  2446.1564458220264
qkgpr_bo:  2446.1564458220264
qkgpr_bo:  2446.1564458220264
qkgpr_bo:  2446.1564458220264
qkgpr_bo:  2446.1564458220264
qkgpr_bo:  1406.4540946226593
qkgpr_bo:  1406.4540946226593
qkgpr_bo:  1406.4540946226593
qkgpr_bo:  1406.4540946226593
qkgpr_bo:  1406.4540946226593
qkgpr_bo:  1406.4540946226593
qkgpr_bo:  1406.4540946226593
qkgpr_bo:  1406.4540946226593
qkgpr_bo:  1406.4540946226593
qkgpr_bo:  1406.4540946226593
qkgpr_bo:  1406.4540946226593
qkgpr_bo:  1406.4540946226593
qkgpr_bo: 

vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
vgpr-bo:  6683.990024117782
bo:  3869.4193426178
[ 63957.95255855 164612.07752275  29372.96578978 127000.77236467
 131011.90031623   9172.3179543  157994.29480714  71459.05647441
  55699.6016341   26278.70794452]
qbo

qkgpr_bo:  195.15716504130626
qkgpr_bo:  195.15716504130626
qkgpr_bo:  195.15716504130626
qkgpr_bo:  195.15716504130626
qkgpr_bo:  195.15716504130626
qkgpr_bo:  195.15716504130626
qkgpr_bo:  195.15716504130626
qkgpr_bo:  195.15716504130626
qkgpr_bo:  15491.506615920765
qkgpr_bo:  15491.506615920765
qkgpr_bo:  15491.506615920765
qkgpr_bo:  15491.506615920765
qkgpr_bo:  8230.092756816157
qkgpr_bo:  8230.092756816157
qkgpr_bo:  8230.092756816157
qkgpr_bo:  8230.092756816157
qkgpr_bo:  8230.092756816157
qkgpr_bo:  8230.092756816157
qkgpr_bo:  8230.092756816157
qkgpr_bo:  8230.092756816157
qkgpr_bo:  8230.092756816157
qkgpr_bo:  8230.092756816157
qkgpr_bo:  8230.092756816157
qkgpr_bo:  8230.092756816157
qkgpr_bo:  8230.092756816157
qkgpr_bo:  8230.092756816157
qkgpr_bo:  8230.092756816157
qkgpr_bo:  8230.092756816157
qkgpr_bo:  8230.092756816157
qkgpr_bo:  8230.092756816157
qkgpr_bo:  8230.092756816157
qkgpr_bo:  8230.092756816157
qkgpr_bo:  8230.092756816157
qkgpr_bo:  8230.092756816157
qk

qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.349040167584
qbo:  2342.34904

...

qkgpr_bo:  7.99341741471167
qkgpr_bo:  7.99341741471167
qkgpr_bo:  7.99341741471167
qkgpr_bo:  7.99341741471167
qkgpr_bo:  7.99341741471167
qkgpr_bo:  7.99341741471167
qkgpr_bo:  7.99341741471167
qkgpr_bo:  7.99341741471167
qkgpr_bo:  7.99341741471167
qkgpr_bo:  7.99341741471167
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.89992400037702
qkgpr_bo:  22.8999240003

qbo:  29.753158968416837
qbo:  29.753158968416837
qbo:  29.753158968416837
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.85445177407182
qbo:  19.8544

qkgpr_bo:  19.01397974458057
qkgpr_bo:  19.01397974458057
qkgpr_bo:  16.262282581495374
qkgpr_bo:  16.262282581495374
qkgpr_bo:  16.262282581495374
qkgpr_bo:  16.262282581495374
qkgpr_bo:  16.262282581495374
qkgpr_bo:  16.262282581495374
qkgpr_bo:  16.262282581495374
qkgpr_bo:  16.262282581495374
qkgpr_bo:  16.262282581495374
qkgpr_bo:  16.262282581495374
qkgpr_bo:  16.262282581495374
qkgpr_bo:  16.262282581495374
qkgpr_bo:  9.562455901508951
qkgpr_bo:  9.562455901508951
qkgpr_bo:  9.562455901508951
qkgpr_bo:  9.562455901508951
qkgpr_bo:  9.562455901508951
qkgpr_bo:  9.562455901508951
qkgpr_bo:  9.562455901508951
qkgpr_bo:  9.562455901508951
qkgpr_bo:  9.562455901508951
qkgpr_bo:  9.562455901508951
qkgpr_bo:  9.562455901508951
qkgpr_bo:  9.562455901508951
qkgpr_bo:  9.562455901508951
qkgpr_bo:  9.562455901508951
qkgpr_bo:  9.562455901508951
qkgpr_bo:  9.562455901508951
qkgpr_bo:  9.562455901508951
qkgpr_bo:  9.562455901508951
qkgpr_bo:  9.562455901508951
qkgpr_bo:  9.562455901508951
qk

qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
qbo:  16.232224030574656
[46.22781907 84.60308615 65.79946872 42.00608729 49.460476   61.24159525
 4

qkgpr_bo:  9.321931159882988
qkgpr_bo:  9.321931159882988
qkgpr_bo:  9.321931159882988
qkgpr_bo:  9.321931159882988
qkgpr_bo:  9.321931159882988
qkgpr_bo:  9.321931159882988
qkgpr_bo:  9.321931159882988
qkgpr_bo:  9.321931159882988
qkgpr_bo:  9.321931159882988
qkgpr_bo:  9.321931159882988
qkgpr_bo:  9.321931159882988
qkgpr_bo:  9.321931159882988
qkgpr_bo:  9.321931159882988
qkgpr_bo:  9.321931159882988
vgpr-bo:  28.921504163856405
vgpr-bo:  28.921504163856405
vgpr-bo:  28.921504163856405
vgpr-bo:  28.921504163856405
vgpr-bo:  28.921504163856405
vgpr-bo:  28.921504163856405
vgpr-bo:  28.921504163856405
vgpr-bo:  28.921504163856405
vgpr-bo:  28.921504163856405
vgpr-bo:  28.921504163856405
vgpr-bo:  28.921504163856405
vgpr-bo:  28.921504163856405
vgpr-bo:  28.921504163856405
vgpr-bo:  28.921504163856405
vgpr-bo:  28.921504163856405
vgpr-bo:  28.921504163856405
vgpr-bo:  28.921504163856405
vgpr-bo:  28.921504163856405
vgpr-bo:  28.921504163856405
vgpr-bo:  28.921504163856405
vgpr-bo:  28.9

In [ ]:
#np.array(results[str(black_box_function.__name__)]['vgpr-bo']).mean()

In [76]:
from scipy import stats
import pandas as pd
fs = [Ackley, Rastrigin,Griewank, Vincent,LunacekTwoRastrigins,Weierstrass]
#fs = [Weierstrass]
out =[]
for black_box_function in fs:
    fs_list = []
    
    print(black_box_function.__name__, ":")
    print("QBO mean fitnes: ",np.array(results[str(black_box_function.__name__)]['qbo']).mean(), np.array(results[str(black_box_function.__name__)]['qbo']).std() )
    print("BO mean fitness: ",np.array(results[str(black_box_function.__name__)]['bo']).mean(),  np.array(results[str(black_box_function.__name__)]['bo']).std() )
    print("qkgprBO mean fitness: ",np.array(results[str(black_box_function.__name__)]['qkgpr']).mean(), np.array(results[str(black_box_function.__name__)]['qkgpr']).std() )
    print("qkgpr-quboBO mean fitness: ",np.array(results[str(black_box_function.__name__)]['qkgpr-qubo']).mean(), np.array(results[str(black_box_function.__name__)]['qkgpr-qubo']).std() )
    print("RFF mean fitness: ",np.array(results[str(black_box_function.__name__)]['rff']).mean(),  np.array(results[str(black_box_function.__name__)]['rff']).std() )
    print("Q-RFF mean fitness: ",np.array(results[str(black_box_function.__name__)]['rff-qubo']).mean(),  np.array(results[str(black_box_function.__name__)]['rff-qubo']).std() )
    print("SD mean fitness: ",np.array(results[str(black_box_function.__name__)]['sd']).mean(),  np.array(results[str(black_box_function.__name__)]['sd']).std() )
    print(stats.ttest_ind(results[str(black_box_function.__name__)]['sd'], results[str(black_box_function.__name__)]['bo'], alternative='less'))
    #print("FITC mean fitnes: ",np.array(results[str(black_box_function.__name__)]['fitc-bo']).mean(), np.array(results[str(black_box_function.__name__)]['fitc-bo']).std() )
    print("VGPR mean fitness: ",np.array(results[str(black_box_function.__name__)]['vgpr-bo']).mean(),  np.array(results[str(black_box_function.__name__)]['vgpr-bo']).std() )


    fs_list.append(np.array(results[str(black_box_function.__name__)]['qbo']).mean())
    fs_list.append(np.array(results[str(black_box_function.__name__)]['bo']).mean()) 
    fs_list.append(np.array(results[str(black_box_function.__name__)]['qkgpr']).mean()) 
    fs_list.append(np.array(results[str(black_box_function.__name__)]['qkgpr-qubo']).mean()) 
    fs_list.append(np.array(results[str(black_box_function.__name__)]['rff']).mean()) 
    fs_list.append(np.array(results[str(black_box_function.__name__)]['rff-qubo']).mean()) 
    fs_list.append(np.array(results[str(black_box_function.__name__)]['sd']).mean()) 
    fs_list.append(np.array(results[str(black_box_function.__name__)]['vgpr-bo']).mean()) 


   
    out.append(fs_list)
    
print(out)
np.set_printoptions(precision=3)
out =np.array(out)
print([str(black_box_function.__name__) for black_box_function in fs])
df = pd.DataFrame(out, index=[str(black_box_function.__name__) for black_box_function in fs], columns = ['Q-RFF'])  
#df['p-value'] = scipy.stats.ttest_ind(df[])

pd.set_option('styler.format.precision', 3)

df
print(df.to_latex(caption="Average results over 20 trials over black-box function benchmarks "))

Ackley :
QBO mean fitnes:  3.9182395912816688 0.928117740782386
BO mean fitness:  4.016475735359625 0.9003820557784695
qkgprBO mean fitness:  4.080059025227824 0.9814001059402031
qkgpr-quboBO mean fitness:  4.209457827712095 0.8536459095413057
RFF mean fitness:  4.092583509382366 0.8678925779131237
Q-RFF mean fitness:  4.057849583322076 0.9380609134716246
SD mean fitness:  4.298954974471838 0.8326088202284435
Ttest_indResult(statistic=1.0040381236179232, pvalue=0.8391444019378181)
VGPR mean fitness:  5.674677088842924 1.3790901747125328
Rastrigin :
QBO mean fitnes:  14.541667405823574 4.609451321414787
BO mean fitness:  14.649233500983204 6.147790344624791
qkgprBO mean fitness:  14.716557206165836 4.0709088559048405
qkgpr-quboBO mean fitness:  13.521537632739419 4.151880130827067
RFF mean fitness:  14.81259922589616 4.06433338482763
Q-RFF mean fitness:  13.254046849949848 4.184595035891148
SD mean fitness:  15.076732522453437 4.15481254350046
Ttest_indResult(statistic=0.251132271331971

ValueError: Shape of passed values is (6, 8), indices imply (6, 1)

In [75]:
import pickle 
with open('./all_3d_10test_10numstart_90calls_20trials_122123.pkl', 'wb') as f:
    pickle.dump(results, f)

In [ ]:
### Read data back in below for visualization. Use this method for running tests with larger d

In [62]:
import pickle
with open('all_tests_50d_10test_10_numstart_20calls_20trials_101523.pkl', "rb") as input_file:
     results = pickle.load(input_file)

In [ ]:
#import pickle
#with open('./qsvm_bo_tests_3d_100test_10_numstart_100calls_20trials.pkl', "rb") as input_file:
#     qsvm_bo_results = pickle.load(input_file)
        
#with open('./bo_tests_3d_100test_10_numstart_100calls_20trials.pkl', "rb") as input_file:
#     bo_results = pickle.load(input_file)

In [ ]:
#print(bo_results['FletcherPowell'])

#with open('./vpr_bo_tests_3d_100test_10_numstart_100calls_20trials.pkl', "rb") as input_file:
#     vgpr_results = pickle.load(input_file)
        
#with open('./fitc_vpr_bo_tests_3d_100test_10_numstart_100calls_20trials.pkl', "rb") as input_file:
#     fitc_vgpr_results = pickle.load(input_file)

In [ ]:
from optproblems.continuous import *
import numpy as np
import pandas as pd

from scipy import stats

from optproblems.continuous import Weierstrass, Schaffer7, LunacekTwoRastrigins, Ackley, Rastrigin, Branin, FletcherPowell, GoldsteinPrice, Griewank, Vincent,Himmelblau,SixHumpCamelback
import pickle

out = []
#fs = [FletcherPowell, Griewank, Vincent,LunacekTwoRastrigins,Weierstrass, Ackley, Rastrigin ]
fs = [Ackley, Rastrigin,Griewank, Vincent,LunacekTwoRastrigins,Weierstrass]
for black_box_function in fs:
    fs_list = []
#for black_box_function in bo_results.keys():
    #try:
    print(black_box_function.__name__, ":")
    print("QBO mean fitnes: ",np.array(bo_results[str(black_box_function.__name__)]['qbo']).mean(), np.array(bo_results[str(black_box_function.__name__)]['qbo']).std() )
    print("BO mean fitness: ",np.array(bo_results[str(black_box_function.__name__)]['bo']).mean(),  np.array(bo_results[str(black_box_function.__name__)]['bo']).std() )
    print("qsvmBO mean fitness: ",np.array(qsvm_bo_results[str(black_box_function.__name__)]['qsvm-bo']).mean(), np.array(qsvm_bo_results[str(black_box_function.__name__)]['qsvm-bo']).std() )
    
    print(stats.ttest_ind(bo_results[str(black_box_function.__name__)]['qbo'], bo_results[str(black_box_function.__name__)]['bo']))
    print(stats.ttest_ind(qsvm_bo_results[str(black_box_function.__name__)]['qsvm-bo'], bo_results[str(black_box_function.__name__)]['bo']))
    for i in bo_results[str(black_box_function.__name__)].keys():
        print(i)
        fs_list.append(np.array(bo_results[str(black_box_function.__name__)][i]).mean())
    fs_list.append(np.array(qsvm_bo_results[str(black_box_function.__name__)]['qsvm-bo']).mean())
    fs_list.append(np.array(fitc_vgpr_results[str(black_box_function.__name__)]['vgpr-bo']).mean())
    fs_list.append(np.array(fitc_vgpr_results[str(black_box_function.__name__)]['fitc-bo']).mean())
    fs_list.append(np.round(stats.ttest_ind(bo_results[str(black_box_function.__name__)]['qbo'], bo_results[str(black_box_function.__name__)]['bo'], alternative='less')[1], 4))
    #except:
        #pass
    out.append(fs_list)
print(out)
np.set_printoptions(precision=3)
print([str(black_box_function.__name__) for black_box_function in fs])
df = pd.DataFrame(out, index=[str(black_box_function.__name__) for black_box_function in fs], columns = ['Q-BO', 'BO', 'Q-SVM-BO', 'VSGPR-BO', 'FITC-BO', 'p-value'])  



print(df.to_latex(caption="Average results over 20 trials over black-box function benchmarks "))

In [ ]:
with open('./bo_tests_20d_20trials.pkl', "rb") as input_file:
     bo_results = pickle.load(input_file)
    
with open('./bo_tests_20d_100test_10_numstart_100calls_20trials.pkl', "rb") as input_file:
     bo_results1 = pickle.load(input_file)
        
        
with open('./vpr_bo_tests_20d_100test_10_numstart_100calls_20trials.pkl', "rb") as input_file:
     vpr_bo_results = pickle.load(input_file)
        
print(bo_results.keys())

In [ ]:
out = []
#fs = [FletcherPowell, Griewank, Vincent,LunacekTwoRastrigins,Weierstrass, Ackley, Rastrigin ]
#fs1 = [Ackley, Rastrigin,Griewank, Vincent,LunacekTwoRastrigins,Weierstrass]
fs1 = [Ackley, Griewank,LunacekTwoRastrigins]
fs = [Rastrigin, Vincent, Weierstrass]
for black_box_function in fs:
    fs_list = []
#for black_box_function in bo_results.keys():
    #try:
    print(black_box_function.__name__, ":")
    print("QBO mean fitnes: ",np.array(bo_results[str(black_box_function.__name__)]['qbo']).mean(), np.array(bo_results[str(black_box_function.__name__)]['qbo']).std() )
    print("BO mean fitness: ",np.array(bo_results[str(black_box_function.__name__)]['bo']).mean(),  np.array(bo_results[str(black_box_function.__name__)]['bo']).std() )
    #print("qsvmBO mean fitness: ",np.array(qsvm_bo_results[str(black_box_function.__name__)]['qsvm-bo']).mean(), np.array(qsvm_bo_results[str(black_box_function.__name__)]['qsvm-bo']).std() )
    print(stats.ttest_ind(bo_results[str(black_box_function.__name__)]['qbo'], bo_results[str(black_box_function.__name__)]['bo']))
    #print(stats.ttest_ind(qsvm_bo_results[str(black_box_function.__name__)]['qsvm-bo'], bo_results[str(black_box_function.__name__)]['bo']))
    for i in bo_results[str(black_box_function.__name__)].keys():
        print(i)
        fs_list.append(np.array(bo_results[str(black_box_function.__name__)][i]).mean())
    fs_list.append(np.array(vpr_bo_results[str(black_box_function.__name__)]['vgpr-bo']).mean())
    
    fs_list.append(np.round(stats.ttest_ind(bo_results[str(black_box_function.__name__)]['qbo'], bo_results[str(black_box_function.__name__)]['bo'], alternative='less')[1], 4))
    #except:
    #except:
        #pass
    out.append(fs_list)

for black_box_function in fs1:
    fs_list = []
#for black_box_function in bo_results.keys():
    #try:
    print(black_box_function.__name__, ":")
    print("QBO mean fitnes: ",np.array(bo_results1[str(black_box_function.__name__)]['qbo']).mean(), np.array(bo_results1[str(black_box_function.__name__)]['qbo']).std() )
    print("BO mean fitness: ",np.array(bo_results1[str(black_box_function.__name__)]['bo']).mean(),  np.array(bo_results1[str(black_box_function.__name__)]['bo']).std() )
    #print("qsvmBO mean fitness: ",np.array(qsvm_bo_results[str(black_box_function.__name__)]['qsvm-bo']).mean(), np.array(qsvm_bo_results[str(black_box_function.__name__)]['qsvm-bo']).std() )
    print(stats.ttest_ind(bo_results1[str(black_box_function.__name__)]['qbo'], bo_results1[str(black_box_function.__name__)]['bo'], alternative='less'))
    #print(stats.ttest_ind(qsvm_bo_results[str(black_box_function.__name__)]['qsvm-bo'], bo_results[str(black_box_function.__name__)]['bo']))
    for i in bo_results1[str(black_box_function.__name__)].keys():
        print(i)
        fs_list.append(np.array(bo_results1[str(black_box_function.__name__)][i]).mean())
    fs_list.append(np.array(vpr_bo_results[str(black_box_function.__name__)]['vgpr-bo']).mean())
    fs_list.append(np.round(stats.ttest_ind(bo_results1[str(black_box_function.__name__)]['qbo'], bo_results1[str(black_box_function.__name__)]['bo'], alternative='less')[1], 4))
    #except:
    #except:
        #pass
    out.append(fs_list)
print(out)

idx = [str(black_box_function.__name__) for black_box_function in fs]
idx1 = [str(black_box_function.__name__) for black_box_function in fs1]
idx2 = idx +idx1
df = pd.DataFrame(out, index=idx2, columns = ['Q-BO', 'BO', 'VSGPR-BO','p-value']) 
df = df.reindex(['Ackley', 'Rastrigin','Griewank', 'Vincent','LunacekTwoRastrigins','Weierstrass'])
print(df.to_latex(caption="Average results over 20 trials over black-box function benchmarks with problem dimension=20 "))

In [ ]:
with open('./bo_tests_50d_100test_10_numstart_100calls_20trials.pkl', "rb") as input_file:
     bo_results = pickle.load(input_file)

In [ ]:
        
with open('./vpr_bo_tests_50d_100test_10_numstart_100calls_20trials.pkl', "rb") as input_file:
     vpr_bo_results = pickle.load(input_file)

In [ ]:
out = []
#fs = [FletcherPowell, Griewank, Vincent,LunacekTwoRastrigins,Weierstrass, Ackley, Rastrigin ]
fs = [Ackley, Rastrigin,Griewank, Vincent,LunacekTwoRastrigins,Weierstrass]
for black_box_function in fs:
    fs_list = []
#for black_box_function in bo_results.keys():
    #try:
    print(black_box_function.__name__, ":")
    print("QBO mean fitnes: ",np.array(bo_results[str(black_box_function.__name__)]['qbo']).mean(), np.array(bo_results[str(black_box_function.__name__)]['qbo']).std() )
    print("BO mean fitness: ",np.array(bo_results[str(black_box_function.__name__)]['bo']).mean(),  np.array(bo_results[str(black_box_function.__name__)]['bo']).std() )
    #print("qsvmBO mean fitness: ",np.array(qsvm_bo_results[str(black_box_function.__name__)]['qsvm-bo']).mean(), np.array(qsvm_bo_results[str(black_box_function.__name__)]['qsvm-bo']).std() )
    print(stats.ttest_ind(bo_results[str(black_box_function.__name__)]['qbo'], bo_results[str(black_box_function.__name__)]['bo']))
    #print(stats.ttest_ind(qsvm_bo_results[str(black_box_function.__name__)]['qsvm-bo'], bo_results[str(black_box_function.__name__)]['bo']))
    for i in bo_results[str(black_box_function.__name__)].keys():
        print(i)
        fs_list.append(np.array(bo_results[str(black_box_function.__name__)][i]).mean())
    fs_list.append(np.array(vpr_bo_results[str(black_box_function.__name__)]['vgpr-bo']).mean())
    fs_list.append(np.round(stats.ttest_ind(bo_results[str(black_box_function.__name__)]['qbo'], bo_results[str(black_box_function.__name__)]['bo'])[1], 4))
    #except:
        #pass
    out.append(fs_list)
print(out)
df = pd.DataFrame(out, index=[str(black_box_function.__name__) for black_box_function in fs], columns = ['Q-BO', 'BO', 'VSGPR-BO', 'p-value'])  
print(df.to_latex(caption="Average results over 20 trials over black-box function benchmarks with problem dimension=50"))

#### Test HPO

In [ ]:
from yahpo_gym import local_config
local_config.init_config()
local_config.set_data_path("../qa-hyp/yahpo_data")

In [ ]:
from yahpo_gym import benchmark_set
import yahpo_gym.benchmarks.lcbench
# Select a Benchmark
bench = benchmark_set.BenchmarkSet("lcbench")
# List available instances
print(bench.instances)
# Set an instance
bench.set_instance('3945')
# Sample a point from the configspace (containing parameters for the instance and budget)
value = bench.config_space.sample_configuration(1).get_dictionary()
# Evaluate
print(bench.config_space)
print(bench.objective_function(value))

value = bench.config_space.sample_configuration().get_dictionary()
# Evaluate
print(value.keys())

import struct
def binary(num):
    return ''.join('{:0>8b}'.format(c) for c in struct.pack('!f', num))

ranges = {'batch_size': [16, 512, 'int'],
          'epoch': [1, 50, 'int'],
          'learning_rate': [0.00010000000000000009, 0.10000000000000002, 'float'],
          'max_dropout': [0.0, 1.0,'float'],
          'max_units': [64, 1024,'int'],
          'momentum': [0.1, 0.99,'float'],
          'num_layers': [1, 5,'int'],
          'weight_decay': [0.00001, 0.1,'float']}




def get_random_candidate(ranges, taskid):
    settings = {}
    settings['OpenML_task_id'] =taskid

    for ix, i in enumerate(ranges.keys()):
        if i == 'OpenML_task_id':
            pass
        try:
            #print(ranges[i][2])
            if ranges[i][2] == 'float':
                l, h = ranges[i][0], ranges[i][1] 
                settings[i] = np.random.uniform(low=l, high=h, size=(1,))[0]
            else:
                l, h = ranges[i][0], ranges[i][1] 
                settings[i] = np.random.randint(low=l, high=h, size=(1,))[0]
        except:
            pass
    return settings
taskid='3945'
settings =get_random_candidate(ranges, taskid='3945')
print(len(settings.keys()))


def generate_x(ranges, taskid='3945'):
    output = []
    output_dict = {}
    output_dict['OpenML_task_id'] = taskid
    for ix, i in enumerate(ranges.keys()):
        
        try:
            #print(ranges[i][2])
            if ranges[i][2] == 'float':
                l, h = ranges[i][0], ranges[i][1] 
                val = np.random.uniform(low=l, high=h, size=(1,))[0]
                output.append(val)
                output_dict[i] = val
            else:
                l, h = ranges[i][0], ranges[i][1]
                val = np.random.randint(low=l, high=h, size=(1,))[0]
                output.append(val)
                output_dict[i] = val
        except:
            pass
    return np.array(output), output_dict



def convert_x(x, taskid):
    
    output_dict = {}
    output_dict['OpenML_task_id'] = taskid
    for ix, i in enumerate(ranges.keys()):
        if ranges[i][2] == 'float':
            output_dict[i] = x[ix]
        else:
            output_dict[i] = int(x[ix])
            
        
    return output_dict
            
        
def objective(candidate):
    eval_ = bench.objective_function(candidate)[0]['val_accuracy']
    return eval_


x_arr, x_dict = generate_x(ranges, taskid)
print(x_arr, x_dict)

print("converted", convert_x(x_arr, taskid), x_dict)
#objective(x_dict)

In [ ]:
def qkgpr_qubo_hpo_optimize(maxmin_bounds,num_calls, f, ranges, num_start, num_test, k):
    noise_var=1e-9
    
    #generate intial samples to warm start optimizer, given by num_start
    #xsamples=np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (num_start, dimension))
   # print(xsamples)
    xsamples= []
    x_dict_samples = []
    fitness = []
    taskid = f.config_space.sample_configuration(1).get_dictionary()['OpenML_task_id']
    dimension = len(f.config_space.sample_configuration(1).get_dictionary().keys()) -1
    #try:
    for i in range(num_start):
        x_arr, x_dict = generate_x(ranges)
        xsamples.append(x_arr)
        x_dict_samples.append(x_dict)
   # print(xsamples)
    xsamples = np.array(xsamples)        
   # except:
        #print('except loop failed')
        #xsamples.append(np.random.uniform(maxmin_bounds[0], maxmin_bounds[1], size = (1, dimension)))

    #Get obj function values to train surrogate
    ysamples=np.array([objective(x) for x in x_dict_samples])
    #print(ysamples)
   
    for i in range(num_calls):
        #Fit the surrogate model.
        #k = int(len(xsamples) * 0.5)
        #mean, std = qesgpr_fit_predict(xsamples, ysamples, noise_variance=1e-6, k=k)
        #Use fit2 and predict2 functions to use quantum filters for both data, x, xtest
        #fit prior
        k = int(len(xsamples) * 0.5)
        #L, x_star, y_star = qesgpr_fit2(xsamples, ysamples, noise_var, k)
        #L = qesgpr_fit(xsamples, ysamples, noise_var, k)
        #print(x_star.shape, y_star.shape)
        #yhat, _ = qesgpr_predict2(L, x_star, xsamples, y_star, noise_var, k)
        #yhat, _ = qesgpr_predict(L,xsamples, ysamples, noise_var, k)
        mean, std, inducing_indices = qk_predict_qbo(xsamples, xsamples, ysamples, k)
        
        #predict from posterior
        best = np.max(ysamples)
        #print(best)
        #xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (300, dimension))
        #try:
        #    xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (100, dimension))
        #except:
        #    xtest = np.random.uniform(maxmin_bounds[0], maxmin_bounds[1],size = (100, dimension))
        xtest = []
        for i in range(num_test):
            x_arr, x_dict = generate_x(ranges)
            xtest.append(x_arr)
        xtest = np.array(xtest)
            
        #mean, std = qesgpr_predict2(L,x_star, xtest, y_star, noise_var, k)
        #mean, std = qesgpr_predict(L, xtest, yhat, noise_var, k)
        mean, std, inducing_indices = qk_predict_qbo(xsamples, xtest, ysamples, k)
        mean = np.mean(mean)
        std = np.mean(std)

        #Find new x using acquisition function.
        new_x= ei_acquisition(best, mean, std, xtest)
        #new_x= ucb_acquisition(mean, std, xtest)
        new_y= objective(convert_x(new_x, taskid))
        #print(new_y)

        #update
        xsamples=np.vstack((xsamples, new_x))
        ysamples=np.hstack((ysamples, new_y))
        fitness.append(ysamples[np.argmax(ysamples)])
    print("qkgpr-qubo: ", ysamples[np.argmax(ysamples)])
    return [ysamples[np.argmax(ysamples)], xsamples[np.argmax(ysamples)]], fitness

In [ ]:
def qkgpr_hpo_optimize(maxmin_bounds,num_calls, f, ranges, num_start, num_test, k):
    noise_var=1e-9
    
    #generate intial samples to warm start optimizer, given by num_start
    #xsamples=np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (num_start, dimension))
   # print(xsamples)
    xsamples= []
    x_dict_samples = []
    fitness = []
    taskid = f.config_space.sample_configuration(1).get_dictionary()['OpenML_task_id']
    dimension = len(f.config_space.sample_configuration(1).get_dictionary().keys()) -1
    #try:
    for i in range(num_start):
        x_arr, x_dict = generate_x(ranges)
        xsamples.append(x_arr)
        x_dict_samples.append(x_dict)
   # print(xsamples)
    xsamples = np.array(xsamples)        
   # except:
        #print('except loop failed')
        #xsamples.append(np.random.uniform(maxmin_bounds[0], maxmin_bounds[1], size = (1, dimension)))

    #Get obj function values to train surrogate
    ysamples=np.array([objective(x) for x in x_dict_samples])
    #print(ysamples)
   
    for i in range(num_calls):
        #Fit the surrogate model.
        #k = int(len(xsamples) * 0.5)
        #mean, std = qesgpr_fit_predict(xsamples, ysamples, noise_variance=1e-6, k=k)
        #Use fit2 and predict2 functions to use quantum filters for both data, x, xtest
        #fit prior
        k = int(len(xsamples) * 0.5)
        #L, x_star, y_star = qesgpr_fit2(xsamples, ysamples, noise_var, k)
        #L = qesgpr_fit(xsamples, ysamples, noise_var, k)
        #print(x_star.shape, y_star.shape)
        #yhat, _ = qesgpr_predict2(L, x_star, xsamples, y_star, noise_var, k)
        #yhat, _ = qesgpr_predict(L,xsamples, ysamples, noise_var, k)
        mean, std, inducing_indices = qk_predict(xsamples, xsamples, ysamples, k)
        
        #predict from posterior
        best = np.max(mean)
        #print(np.max(ysamples))
        #xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (300, dimension))
        #try:
        #    xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (100, dimension))
        #except:
        #    xtest = np.random.uniform(maxmin_bounds[0], maxmin_bounds[1],size = (100, dimension))
        xtest = []
        for i in range(num_test):
            x_arr, x_dict = generate_x(ranges)
            xtest.append(x_arr)
        xtest = np.array(xtest)
            
        #mean, std = qesgpr_predict2(L,x_star, xtest, y_star, noise_var, k)
        #mean, std = qesgpr_predict(L, xtest, yhat, noise_var, k)
        mean, std, inducing_indices = qk_predict(xsamples, xtest, ysamples, k)
        mean = np.mean(mean)
        std = np.mean(std)

        #Find new x using acquisition function.
        new_x= ei_acquisition(best, mean, std, xtest)
        #new_x= ucb_acquisition(mean, std, xtest)
        new_y= objective(convert_x(new_x, taskid))
        #print(new_y)

        #update
        xsamples=np.vstack((xsamples, new_x))
        ysamples=np.hstack((ysamples, new_y))
        fitness.append(ysamples[np.argmax(ysamples)])
    print("qkgpr: ", ysamples[np.argmax(ysamples)])
    return [ysamples[np.argmax(ysamples)], xsamples[np.argmax(ysamples)]], fitness

In [ ]:
def rff_hpo_optimize(maxmin_bounds,num_calls, f, ranges, num_start, num_test, k):
    noise_var=1e-9
    
    #generate intial samples to warm start optimizer, given by num_start
    #xsamples=np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (num_start, dimension))
   # print(xsamples)
    xsamples= []
    x_dict_samples = []
    fitness = []
    taskid = f.config_space.sample_configuration(1).get_dictionary()['OpenML_task_id']
    dimension = len(f.config_space.sample_configuration(1).get_dictionary().keys()) -1
    #try:
    for i in range(num_start):
        x_arr, x_dict = generate_x(ranges)
        xsamples.append(x_arr)
        x_dict_samples.append(x_dict)
   # print(xsamples)
    xsamples = np.array(xsamples)        
   # except:
        #print('except loop failed')
        #xsamples.append(np.random.uniform(maxmin_bounds[0], maxmin_bounds[1], size = (1, dimension)))

    #Get obj function values to train surrogate
    ysamples=np.array([objective(x) for x in x_dict_samples])
    #print(ysamples)
   
    for i in range(num_calls):
        #Fit the surrogate model.
        #k = int(len(xsamples) * 0.5)
        #mean, std = qesgpr_fit_predict(xsamples, ysamples, noise_variance=1e-6, k=k)
        #Use fit2 and predict2 functions to use quantum filters for both data, x, xtest
        #fit prior
        k = int(len(xsamples) * 0.5)
        #L, x_star, y_star = qesgpr_fit2(xsamples, ysamples, noise_var, k)
        #L = qesgpr_fit(xsamples, ysamples, noise_var, k)
        #print(x_star.shape, y_star.shape)
        #yhat, _ = qesgpr_predict2(L, x_star, xsamples, y_star, noise_var, k)
        #yhat, _ = qesgpr_predict(L,xsamples, ysamples, noise_var, k)
        mean, std, inducing_indices = rff_predict(xsamples, xsamples, ysamples, output_dim=50, k=k)
        
        #predict from posterior
        best = np.max(mean)
        #print(np.max(ysamples))
        #xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (300, dimension))
        #try:
        #    xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (100, dimension))
        #except:
        #    xtest = np.random.uniform(maxmin_bounds[0], maxmin_bounds[1],size = (100, dimension))
        xtest = []
        for i in range(num_test):
            x_arr, x_dict = generate_x(ranges)
            xtest.append(x_arr)
        xtest = np.array(xtest)
            
        #mean, std = qesgpr_predict2(L,x_star, xtest, y_star, noise_var, k)
        #mean, std = qesgpr_predict(L, xtest, yhat, noise_var, k)
        mean, std, inducing_indices = rff_predict(xsamples, xtest, ysamples, output_dim=50, k=k)
        mean = np.mean(mean)
        std = np.mean(std)

        #Find new x using acquisition function.
        new_x= ei_acquisition(best, mean, std, xtest)
        #new_x= ucb_acquisition(mean, std, xtest)
        new_y= objective(convert_x(new_x, taskid))
        #print(new_y)

        #update
        xsamples=np.vstack((xsamples, new_x))
        ysamples=np.hstack((ysamples, new_y))
        fitness.append(ysamples[np.argmax(ysamples)])
    print("rff: ", ysamples[np.argmax(ysamples)])
    return [ysamples[np.argmax(ysamples)], xsamples[np.argmax(ysamples)]], fitness

In [ ]:
def qrff_hpo_optimize(maxmin_bounds,num_calls, f, ranges, num_start, num_test, k):
    noise_var=1e-9
    
    #generate intial samples to warm start optimizer, given by num_start
    #xsamples=np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (num_start, dimension))
   # print(xsamples)
    xsamples= []
    x_dict_samples = []
    fitness = []
    taskid = f.config_space.sample_configuration(1).get_dictionary()['OpenML_task_id']
    dimension = len(f.config_space.sample_configuration(1).get_dictionary().keys()) -1
    #try:
    for i in range(num_start):
        x_arr, x_dict = generate_x(ranges)
        xsamples.append(x_arr)
        x_dict_samples.append(x_dict)
   # print(xsamples)
    xsamples = np.array(xsamples)        
   # except:
        #print('except loop failed')
        #xsamples.append(np.random.uniform(maxmin_bounds[0], maxmin_bounds[1], size = (1, dimension)))

    #Get obj function values to train surrogate
    ysamples=np.array([objective(x) for x in x_dict_samples])
    #print(ysamples)
   
    for i in range(num_calls):
        #Fit the surrogate model.
        #k = int(len(xsamples) * 0.5)
        #mean, std = qesgpr_fit_predict(xsamples, ysamples, noise_variance=1e-6, k=k)
        #Use fit2 and predict2 functions to use quantum filters for both data, x, xtest
        #fit prior
        k = int(len(xsamples) * 0.5)
        #L, x_star, y_star = qesgpr_fit2(xsamples, ysamples, noise_var, k)
        #L = qesgpr_fit(xsamples, ysamples, noise_var, k)
        #print(x_star.shape, y_star.shape)
        #yhat, _ = qesgpr_predict2(L, x_star, xsamples, y_star, noise_var, k)
        #yhat, _ = qesgpr_predict(L,xsamples, ysamples, noise_var, k)
        mean, std, inducing_indices = rff_predict_qbo(xsamples, xsamples, ysamples, output_dim=50, k=k)
        
        #predict from posterior
        best = np.max(mean)
        #print(np.max(ysamples))
        #xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (300, dimension))
        #try:
        #    xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (100, dimension))
        #except:
        #    xtest = np.random.uniform(maxmin_bounds[0], maxmin_bounds[1],size = (100, dimension))
        xtest = []
        for i in range(num_test):
            x_arr, x_dict = generate_x(ranges)
            xtest.append(x_arr)
        xtest = np.array(xtest)
            
        #mean, std = qesgpr_predict2(L,x_star, xtest, y_star, noise_var, k)
        #mean, std = qesgpr_predict(L, xtest, yhat, noise_var, k)
        mean, std, inducing_indices = rff_predict_qbo(xsamples, xtest, ysamples, output_dim=50, k=k)
        mean = np.mean(mean)
        std = np.mean(std)

        #Find new x using acquisition function.
        new_x= ei_acquisition(best, mean, std, xtest)
        #new_x= ucb_acquisition(mean, std, xtest)
        new_y= objective(convert_x(new_x, taskid))
        #print(new_y)

        #update
        xsamples=np.vstack((xsamples, new_x))
        ysamples=np.hstack((ysamples, new_y))
        fitness.append(ysamples[np.argmax(ysamples)])
    print("rff: ", ysamples[np.argmax(ysamples)])
    return [ysamples[np.argmax(ysamples)], xsamples[np.argmax(ysamples)]], fitness

In [ ]:
def sd_hpo_optimize(maxmin_bounds,num_calls, f, ranges, num_start, num_test, k):
    noise_var=1e-9
    
    #generate intial samples to warm start optimizer, given by num_start
    #xsamples=np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (num_start, dimension))
   # print(xsamples)
    xsamples= []
    x_dict_samples = []
    fitness = []
    taskid = f.config_space.sample_configuration(1).get_dictionary()['OpenML_task_id']
    dimension = len(f.config_space.sample_configuration(1).get_dictionary().keys()) -1
    #try:
    for i in range(num_start):
        x_arr, x_dict = generate_x(ranges)
        xsamples.append(x_arr)
        x_dict_samples.append(x_dict)
   # print(xsamples)
    xsamples = np.array(xsamples)        
   # except:
        #print('except loop failed')
        #xsamples.append(np.random.uniform(maxmin_bounds[0], maxmin_bounds[1], size = (1, dimension)))

    #Get obj function values to train surrogate
    ysamples=np.array([objective(x) for x in x_dict_samples])
    #print(ysamples)
   
    for i in range(num_calls):
        #Fit the surrogate model.
        #k = int(len(xsamples) * 0.5)
        #mean, std = qesgpr_fit_predict(xsamples, ysamples, noise_variance=1e-6, k=k)
        #Use fit2 and predict2 functions to use quantum filters for both data, x, xtest
        #fit prior
        k = int(len(xsamples) * 0.5)
        #L, x_star, y_star = qesgpr_fit2(xsamples, ysamples, noise_var, k)
        #L = qesgpr_fit(xsamples, ysamples, noise_var, k)
        #print(x_star.shape, y_star.shape)
        #yhat, _ = qesgpr_predict2(L, x_star, xsamples, y_star, noise_var, k)
        #yhat, _ = qesgpr_predict(L,xsamples, ysamples, noise_var, k)
        mean, std, inducing_indices = sd_predict(xsamples, xsamples, ysamples, k)
        
        #predict from posterior
        best = np.max(ysamples)
        #print(best)
        #xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (300, dimension))
        #try:
        #    xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (100, dimension))
        #except:
        #    xtest = np.random.uniform(maxmin_bounds[0], maxmin_bounds[1],size = (100, dimension))
        xtest = []
        for i in range(num_test):
            x_arr, x_dict = generate_x(ranges)
            xtest.append(x_arr)
        xtest = np.array(xtest)
            
        #mean, std = qesgpr_predict2(L,x_star, xtest, y_star, noise_var, k)
        #mean, std = qesgpr_predict(L, xtest, yhat, noise_var, k)
        mean, std, inducing_indices = sd_predict(xsamples, xtest, ysamples, k)
        mean = np.mean(mean)
        std = np.mean(std)

        #Find new x using acquisition function.
        new_x= ei_acquisition(best, mean, std, xtest)
        #new_x= ucb_acquisition(mean, std, xtest)
        new_y= objective(convert_x(new_x, taskid))
        #print(new_y)

        #update
        xsamples=np.vstack((xsamples, new_x))
        ysamples=np.hstack((ysamples, new_y))
        fitness.append(ysamples[np.argmax(ysamples)])
    print("sd: ", ysamples[np.argmax(ysamples)])
    return [ysamples[np.argmax(ysamples)], xsamples[np.argmax(ysamples)]], fitness

In [ ]:
maxmin_bounds = [ -5., 5]
num_calls = 10
f = bench
ranges =ranges
num_start=10
num_test = 10
dimension= 8
k = num_start

[y_star, x_star], fitness = sd_hpo_optimize(maxmin_bounds,num_calls, f, ranges, num_start, num_test,  k)


In [ ]:
def qbo_hpo_optimize(maxmin_bounds,num_calls, f, ranges, num_start, num_test, k):
    noise_var=1e-9
    
    #generate intial samples to warm start optimizer, given by num_start
    #xsamples=np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (num_start, dimension))
   # print(xsamples)
    xsamples= []
    x_dict_samples = []
    fitness = []
    taskid = f.config_space.sample_configuration(1).get_dictionary()['OpenML_task_id']
    dimension = len(f.config_space.sample_configuration(1).get_dictionary().keys()) -1
    #try:
    for i in range(num_start):
        x_arr, x_dict = generate_x(ranges)
        xsamples.append(x_arr)
        x_dict_samples.append(x_dict)
   # print(xsamples)
    xsamples = np.array(xsamples)        
   # except:
        #print('except loop failed')
        #xsamples.append(np.random.uniform(maxmin_bounds[0], maxmin_bounds[1], size = (1, dimension)))

    #Get obj function values to train surrogate
    ysamples=np.array([objective(x) for x in x_dict_samples])
    #print(ysamples)
   
    for i in range(num_calls):
        #Fit the surrogate model.
        #k = int(len(xsamples) * 0.5)
        #mean, std = qesgpr_fit_predict(xsamples, ysamples, noise_variance=1e-6, k=k)
        #Use fit2 and predict2 functions to use quantum filters for both data, x, xtest
        #fit prior
        k = int(len(xsamples) * 0.5)
        #L, x_star, y_star = qesgpr_fit2(xsamples, ysamples, noise_var, k)
        #L = qesgpr_fit(xsamples, ysamples, noise_var, k)
        #print(x_star.shape, y_star.shape)
        #yhat, _ = qesgpr_predict2(L, x_star, xsamples, y_star, noise_var, k)
        #yhat, _ = qesgpr_predict(L,xsamples, ysamples, noise_var, k)
        mean, std, inducing_indices = qbo_predict(xsamples, xsamples, ysamples, k)
        
        #predict from posterior
        best = np.max(mean)
        #print(best)
        #xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (300, dimension))
        #try:
        #    xtest = np.random.uniform(f.min_bounds[0], f.max_bounds[0], size = (100, dimension))
        #except:
        #    xtest = np.random.uniform(maxmin_bounds[0], maxmin_bounds[1],size = (100, dimension))
        xtest = []
        for i in range(num_test):
            x_arr, x_dict = generate_x(ranges)
            xtest.append(x_arr)
        xtest = np.array(xtest)
            
        #mean, std = qesgpr_predict2(L,x_star, xtest, y_star, noise_var, k)
        #mean, std = qesgpr_predict(L, xtest, yhat, noise_var, k)
        mean, std, inducing_indices = qbo_predict(xsamples, xtest, ysamples, k)
        mean = np.mean(mean)
        std = np.mean(std)

        #Find new x using acquisition function.
        new_x= ei_acquisition(best, mean, std, xtest)
        #new_x= ucb_acquisition(mean, std, xtest)
        new_y= objective(convert_x(new_x, taskid))
        #print(new_y)

        #update
        xsamples=np.vstack((xsamples, new_x))
        ysamples=np.hstack((ysamples, new_y))
        fitness.append(ysamples[np.argmax(ysamples)])
    print("qbo: ", ysamples[np.argmax(ysamples)])
    return [ysamples[np.argmax(ysamples)], xsamples[np.argmax(ysamples)]], fitness

In [ ]:
import sparse_GP as sGP

def vgpr_hpo_optimize(maxmin_bounds, num_calls, f, ranges, num_start, num_test, k):
    
    
    #generate intial samples to warm start optimizer, given by num_start
    xsamples= []
    x_dict_samples = []
    fitness = []
    
    taskid = f.config_space.sample_configuration(1).get_dictionary()['OpenML_task_id']
    dimension = len(f.config_space.sample_configuration(1).get_dictionary().keys()) -1
    
    #try:
    for i in range(num_start):
        x_arr, x_dict = generate_x(ranges)
        xsamples.append(x_arr)
        x_dict_samples.append(x_dict)
    #print(xsamples)
    xsamples = np.array(xsamples)   

    #Get obj function values to train surrogate
    ysamples=np.array([objective(x) for x in x_dict_samples])
    #print(ysamples)
    #print(ysamples)
    
    for i in range(num_calls):
        #Fit the surrogate function.
        
        
        k = int(len(xsamples) * 0.5)
        num_start = len(xsamples)
        
        L = None
        
        while L == None:
            try:
                (L, sigma, K, C, inv_C, Xs, Ys, LB_best, elapsed_time) = sGP.train(0.2, noise_var,
                                                                   xsamples, ysamples, num_start, k,
                                                                   1)
            except:
                pass

           # Make predictions
        N_Star = len(xsamples)               # No. points where we make predictions
        Y_star_mean = np.zeros(num_test )     # Initialise GP mean predictions
        Y_star_std = np.zeros(num_test ) 

        for indx, x_star_ in enumerate(xsamples):
            try:
                Y_star_mean[indx], Y_star_std[indx] = sGP.predict(Xs, x_star_, L, sigma,
                                                        Ys, K, C, inv_C, k)
            except:
                pass

        #predict from posterior
        best = np.max(Y_star_mean)
        #print(best)
        #fitness.append(best)
        
        #predict mean, std from xtest
        
        xtest = []
        for i in range(num_test):
            x_arr, x_dict = generate_x(ranges)
            xtest.append(x_arr)
        xtest = np.array(xtest)
        
        for indx, x_star_ in enumerate(xtest):
            try:
                Y_star_mean[indx], Y_star_std[indx] = sGP.predict(Xs, x_star_, L, sigma,
                                                        Ys, K, C, inv_C, k)
                
            except:
                pass

        #print(mean, std)

        #Find new x using acquisition function.
        new_x= ei_acquisition(best, Y_star_mean, Y_star_std, xtest)
        #new_x= ucb_acquisition(mean, std, xtest)
        #print(new_x)
        new_y= objective(convert_x(new_x, taskid))
        #new_y= f(new_x)
        #print(new_y)

        #update
        xsamples=np.vstack((xsamples, new_x))
        ysamples=np.hstack((ysamples, new_y))
        fitness.append(ysamples[np.argmax(ysamples)])
    print("vgpr: ",ysamples[np.argmax(ysamples)])
    return [ysamples[np.argmax(ysamples)], xsamples[np.argmax(ysamples)]], fitness

In [ ]:
[y_star, x_star], fitness = vgpr_hpo_optimize(maxmin_bounds, num_calls, f, ranges, num_start, num_test, k)

In [ ]:
def bo_hpo_optimize(maxmin_bounds, num_calls, f, ranges, num_start, num_test):
    surrogate = GaussianProcessRegression(kernel_function=rbf_kernel_)
    
    #generate intial samples to warm start optimizer, given by num_start
    xsamples= []
    x_dict_samples = []
    fitness = []
    
    taskid = f.config_space.sample_configuration(1).get_dictionary()['OpenML_task_id']
    dimension = len(f.config_space.sample_configuration(1).get_dictionary().keys()) -1
    
    #try:
    for i in range(num_start):
        x_arr, x_dict = generate_x(ranges)
        xsamples.append(x_arr)
        x_dict_samples.append(x_dict)
    #print(xsamples)
    xsamples = np.array(xsamples)   

    #Get obj function values to train surrogate
    ysamples=np.array([objective(x) for x in x_dict_samples])
    #print(ysamples)
    #print(ysamples)
    
    for i in range(num_calls):
        #Fit the surrogate function.
        surrogate.fit(xsamples, ysamples)
        yhat, _ = surrogate.predict(xsamples)
    
        best = np.max(yhat)
       # print(best)
        #fitness.append(best)
        
        #predict mean, std from xtest
        
        xtest = []
        for i in range(num_test):
            x_arr, x_dict = generate_x(ranges)
            xtest.append(x_arr)
        xtest = np.array(xtest)
        
        mean, std = surrogate.predict(xtest)
        #print(mean, std)

        #Find new x using acquisition function.
        new_x= ei_acquisition(best, mean, std, xtest)
        #new_x= ucb_acquisition(mean, std, xtest)
        #print(new_x)
        new_y= objective(convert_x(new_x, taskid))
        #new_y= f(new_x)
        #print(new_y)

        #update
        xsamples=np.vstack((xsamples, new_x))
        ysamples=np.hstack((ysamples, new_y))
        fitness.append(ysamples[np.argmax(ysamples)])
    print("bo: ",ysamples[np.argmax(ysamples)])
    return [ysamples[np.argmax(ysamples)], xsamples[np.argmax(ysamples)]], fitness

In [ ]:
maxmin_bounds = [ -5., 5]
num_calls = 75
f = bench
ranges =ranges
num_start=10
num_test = 100
dimension= 8
k = num_start

#[y_star, x_star], fitness = qkgpr_hpo_optimize(maxmin_bounds,num_calls, f, ranges, num_start, num_test, k)


In [ ]:
n_experiments = 20
num_calls = 50
experiment_results_ = {}
num_test=10
num_start =10

for instance in bench.instances:
    
    

    experiment_results_[instance] = {}

    f = bench
    qbo_fitness_all = []
    bo_fitness_all = []
    rff_fitness_all = []
    qrff_fitness_all = []
    sd_fitness_all = []
    qkgpr_fitness_all = []
    qkgpr_qubo_fitness_all = []
    
    vgpr_fitness_all = []
    
    
    for s in range(n_experiments):
        print(" instance: ", instance," experiment: ", s, )
       # _, qbo_fitness = qbo_hpo_optimize(maxmin_bounds,num_calls, f, ranges, num_start, num_test,  k)
       # qbo_fitness_all.append(qbo_fitness)
        
       # _, bo_fitness = bo_hpo_optimize(maxmin_bounds, num_calls, f, ranges, num_start, num_test)
       # bo_fitness_all.append(bo_fitness)
        
        _, vgpr_fitness = vgpr_hpo_optimize(maxmin_bounds,num_calls, f, ranges, num_start, num_test,  k)
        vgpr_fitness_all.append(vgpr_fitness)
        
       # _, rff_fitness = rff_hpo_optimize(maxmin_bounds,num_calls, f, ranges, num_start, num_test,  k)
       # rff_fitness_all.append(rff_fitness)
        
       # _, qrff_fitness = qrff_hpo_optimize(maxmin_bounds,num_calls, f, ranges, num_start, num_test,  k)
        #qrff_fitness_all.append(qrff_fitness)
        
        
       # _, sd_fitness = sd_hpo_optimize(maxmin_bounds,num_calls, f, ranges, num_start, num_test,  k)
       # sd_fitness_all.append(sd_fitness)
        
       # _, qkgpr_fitness = qkgpr_hpo_optimize(maxmin_bounds,num_calls, f, ranges, num_start, num_test,  k)
       # qkgpr_fitness_all.append(qkgpr_fitness)
        
       # _, qkgpr_qubo_fitness = qkgpr_qubo_hpo_optimize(maxmin_bounds,num_calls, f, ranges, num_start, num_test,  k)
       # qkgpr_qubo_fitness_all.append(qkgpr_qubo_fitness)
        
        
        
        
    #print("QBO mean: ", np.array(qbo_fitness_all).mean())
    #print("BO mean: ", np.array(bo_fitness_all).mean())
    #print("RFF mean: ", np.array(rff_fitness_all).mean())
    #print("Q-RFF mean: ", np.array(qrff_fitness_all).mean())
    #print("SD mean: ", np.array(sd_fitness_all).mean())
    #print("QKGPR mean: ", np.array(qkgpr_fitness_all).mean())
    #print("QKGPR-QUBO mean: ", np.array(qkgpr_qubo_fitness_all).mean())
    
    print("VGPR mean: ", np.array(vgpr_fitness_all).mean())
    
    #experiment_results_[instance]["qbo"] = qbo_fitness_all
    #experiment_results_[instance]["bo"] = bo_fitness_all
    #experiment_results_[instance]["rff"] = rff_fitness_all
    #experiment_results_[instance]["qrff"] = qrff_fitness_all
    #experiment_results_[instance]["sd"] = sd_fitness_all
    #experiment_results_[instance]["qkgpr"] = qkgpr_fitness_all
    #experiment_results_[instance]["qkgpr_qubo"] = qkgpr_qubo_fitness_all
            
    
    experiment_results_[instance]["vgpr"] = vgpr_fitness_all
   

In [ ]:
import pickle
with open('vgpr_hpo_20trials_50g_110123_val_acc.pkl', 'wb') as f_:
    pickle.dump(experiment_results_, f_)
    
#with open('all_tests_3d_100test_10_numstart_100calls_20trials_101423.pkl', 'rb') as f_:
  #  experiment_results_ = pickle.load(f_)
    
    

In [ ]:
import scipy

out = []
for i in experiment_results_.keys():
    print(i)
    for j in experiment_results_[i].keys():
        print(j, " ", np.array(experiment_results_[i][j])[:, -1].mean(axis=0))
        out.append(np.array(experiment_results_[i][j])[:, -1].mean(axis=0))
print(np.array(out).mean())
    #print(scipy.stats.ttest_ind(np.array(experiment_results_[i]['qbo'])[:,-1],np.array(experiment_results_[i]['bo'])[:,-1]))

In [ ]:
import pandas as pd
out =[]

#fs = []

for i in list(experiment_results_.keys()):
    fs = []
    print(i)
    for j in experiment_results_[i].keys():
        print(j)
        #fs.append(j)
      
        fs.append(format(np.array(experiment_results_[i][j])[:,-1].mean(), 'f'))
    #fs.append(scipy.stats.ttest_ind(np.array(experiment_results_[i]['qbo'])[:,-1],np.array(experiment_results_[i]['bo'])[:,-1])[1])

            
    out.append(fs)
#df2 = pd.DataFrame(out, index=list(experiment_results_.keys()), columns = experiment_results_['3945'].keys()) 

print(out)

In [ ]:
df2 = pd.DataFrame(out, index=list(experiment_results_.keys()), columns = experiment_results_['3945'].keys()) 

In [ ]:
df2.astype(np.float).mean()
#scipy.stats.ttest_ind(df2['qkgpr_qubo'].values.astype(np.float), df2['bo'].values.astype(np.float), alternative='greater')[1]

In [ ]:
names = ['QBO', 'BO', 'RFF', 'SD', 'QKGPR-BO', 'QUBO-QKGPR-BO']
for i in df2.columns:
    df2[i] = pd.to_numeric(df2[i],errors='coerce')
    
df2.columns=names    
df2['p-value']= scipy.stats.ttest_ind(df2['QUBO-QKGPR-BO'].values.astype(np.float), df2['BO'].values.astype(np.float), alternative='greater')[1]

df_mean = df2.mean(axis=0)
df_mean = pd.DataFrame(df_mean).T
df_mean
#df_mean = pd.DataFrame(df_mean)
print(df_mean.to_latex())


In [ ]:
#len(df['BO'].values)
df_mean['p-value'] = scipy.stats.ttest_ind(df['QBO'].values, df['BO'].values, alternative='greater')[1]

In [ ]:
print(df_mean.to_latex())

In [ ]:
print(df.to_latex())

# Eigendecomposition analysis and data visualization

In [ ]:
#Generate full and sparse covariance matrices
import numpy as np
from optproblems.continuous import *

num_experiments = 20
num_calls =100
problem_dimension = 1
num_start=20
k = int(num_start * .5)


from optproblems.continuous import Weierstrass, Schaffer7, LunacekTwoRastrigins, Ackley, Rastrigin, Branin, FletcherPowell, GoldsteinPrice, Griewank, Vincent,Himmelblau,SixHumpCamelback
import pickle


#func = Rastrigin(num_variables=problem_dimension)

def func(x):
    """Latent function."""
    return 1.0 * np.cos(x * 4 * np.pi) + \
           0.3 * np.sin(x * 3 * np.pi) + \
           0.5 * np.cos(x * 2 * np.pi)
l2 = 0.2

r_min, r_max = -5, 5.
    
maxmin_bounds = (r_min, r_max)

xsamples=np.random.uniform(r_min, r_max, size = (num_start, problem_dimension))


#Get obj function values to train surrogate
ysamples=np.array([func(x) for x in xsamples])
print(ysamples)


k = int(len(xsamples) * 0.5)
L, x_star, y_star, _ = qesgpr_fit2(xsamples, ysamples, noise_var, k)

K = rbf_kernel(xsamples, xsamples,l2=l2)
print(K.shape)

K_star = rbf_kernel(x_star, x_star,l2=l2)
print(K_star.shape)

In [ ]:
import numpy as np

K_eig = np.linalg.eig(K)
sorted(K_eig[0])

In [ ]:
K_star_eig = np.linalg.eig(K_star)
K_star_eig

In [ ]:
K_eig_real = K_eig[0].real

In [ ]:
K_star_eig_real = K_star_eig[0].real

In [ ]:
np.set_printoptions(suppress=True)
K_eig_real

In [ ]:
K_star_eig_real

In [ ]:
import matplotlib.pyplot as plt
plt.hist(K_eig_real[:20])
plt.hist(K_star_eig_real[:20])
 
# plotting second histogram

 
# Showing the plot using plt.show()
plt.show()

In [ ]:
K_star_eig_real = np.append(K_star_eig_real, np.zeros(500))
K_star_eig_real 

In [ ]:
plt.plot(K_eig_real[:20], "r--",label='K eigenvalues')
plt.plot(K_star_eig_real[:20], "b--", label='K* eigenvalues') 
plt.title("Comparison of largest Eigenvalues from Eigendecompostion, K and K*")
plt.xlabel("index")
plt.ylabel("value")
plt.legend()
plt.show()

In [ ]:
print(np.linalg.cond(K))

In [ ]:
print(np.linalg.cond(K_star))

In [ ]:
K_svd = np.linalg.svd(K)

In [ ]:
K_svd

In [ ]:
K_star_svd = np.linalg.svd(K_star)

In [ ]:
K_star_svd

In [ ]:
import pandas as pd

svd_df = pd.DataFrame()
svd_star_df = pd.DataFrame()

delta_zero =1.


svd_df['K singular values'] = K_svd[1]
#svd_df.hist()

delta_svd_df = svd_df.loc[(svd_df['K singular values'] >= delta_zero)]
delta_svd_df.hist()

svd_star_df['K* singular values'] = K_star_svd[1]
delta_svd_star_df = svd_star_df.loc[(svd_star_df['K* singular values'] >= delta_zero)]
delta_svd_star_df.hist()

#svd_star_df['K* singular values'] = K_star_svd[1]
#svd_star_df.hist()

In [ ]:
from scipy.stats import shapiro, ttest_ind

test_k = delta_svd_df.values
test_k_star = delta_svd_star_df.values
#A test statistic close to 1 indicates normality
#print(shapiro(K_svd[0]))

print(shapiro(test_k))
print(shapiro(test_k_star))
# test with large p-value reject the null that the distributions are unequal
print(ttest_ind(test_k, test_k_star, alternative='two-sided'))

## 1d visualization

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("seaborn-darkgrid")
import random
import numpy as np

seed = 1111
random.seed(seed)

np.random.seed(seed)
noise_var=1e-9

def func(x):
    """Latent function."""
    return 1.0 * np.cos(x * 4 * np.pi) + \
           0.3 * np.sin(x * 2 * np.pi) + \
           0.8 * np.cos(x **2 * 1.5 * np.pi) 
          



# Number of training examples
n = 50

# Number of inducing variables
m = 25
c=5
output_dim =50

#def func(x):
#    return np.sin(x) + 0.1*np.random.randn(len(x))

# Noise
sigma_y = 0.2
k = m

# Noisy training data
X = np.linspace(-1.0, 1.0, n).reshape(-1, 1)
y = func(X) + sigma_y * np.random.randn(n).reshape(-1, 1)

#Extend to multidimensional
X_c = np.random.uniform(low=-1. , high= 1. , size= (n,c))
y_c= func(X_c) + sigma_y * np.random.randn(n).reshape(-1, 1)

# Test data
X_test = np.linspace(-1.0, 1.0, n).reshape(-1, 1)
f_true = func(X_test).reshape(-1, 1)

X_test_c = np.random.uniform(low=-1. , high= 1. , size= (n,c))
f_true_c = func(X_test_c).reshape(-1, 1)

#X_test_rand = np.linspace(-1.0, 1.0, n).reshape(-1, 1)

print(X.shape)

In [ ]:

#rff_predict(X_train, X_test, output_dim=200, k=k, kernel=rbf_kernel(X_train,X))
rff_y_pred,rff_s, inducing_indices = rff_predict(X, X_test, y, output_dim, k)
sd_y_pred,sd_s, _ = sd_predict(X, X_test, y, k)

qbo_yhat, qbo_s, inducing_qbo = qbo_predict(X, X_test, y, k)
#qk_yhat, qk_s, inducing_qbo = qk_predict(X, X_test, y, k)



gp = GaussianProcessRegression(kernel_function=rbf_kernel_)
gp.fit(X, y)

#X_test = np.array([[1.5], [2.5]])
yhat_full, sigma = gp.predict(X_test)

ci = qbo_yhat+qbo_s.reshape(-1,1)


In [ ]:
plt.plot(X_test, qbo_yhat,linestyle='dashed', label='Prediction from Sparse GPR', c='g')
plt.fill_between(np.array(X_test).reshape(-1,1).flatten(), np.array(qbo_yhat+(2*qbo_s).reshape(-1,1)).flatten(), np.array(qbo_yhat-(2*qbo_s).reshape(-1,1)).flatten())
#plt.plot(X_test, rff_y_pred,linestyle='dashed', label='Prediction from Fourier Features GPR', c='y')
#plt.fill_between(X_t_star[truncated_aspect1:-truncated_aspect1], (yhat[truncated_aspect1:-truncated_aspect1]-qe_ci[truncated_aspect1:-truncated_aspect1]), (yhat[truncated_aspect1:-truncated_aspect1]+qe_ci[truncated_aspect1:-truncated_aspect1]), color='b', alpha=.1)
#plt.plot(X_test, Y_star_mean, 'orange', label='Prediction from Variational Sparse GP')
plt.plot(X_test, yhat_full,linestyle='dashed', label='Prediction from Full GPR', c='r')
#plt.plot(X_test, y_pygps[0],linestyle='dashed', label='Prediction from FITC', c='blue')
#plt.plot(X, y, label='Training examples', c='r')
plt.title('Model Predictions')
plt.plot(X_test, f_true, 'black', label='Objective Function', c='k')
plt.xlabel('x')
plt.ylabel('y')
#plt.xlim(-1.5, 1.5)
plt.legend()

In [ ]:
np.array(qbo_yhat + qbo_s.reshape(-1,1)).shape

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharex=True)
ax1.plot(X_test, qbo_yhat, label='Prediction from QUBO Sparse GPR', c='g')
ax1.fill_between(np.array(X_test).reshape(-1,1).flatten(), np.array(qbo_yhat+qbo_s.reshape(-1,1)).flatten(), np.array(qbo_yhat-qbo_s.reshape(-1,1)).flatten())
ax1.plot(X_test, f_true, 'yellow', label='Objective Function', c='y')
ax2.plot(X_test, sd_y_pred, label='Prediction from SD Sparse GPR', c='r')
ax2.plot(X_test, f_true, 'yellow', label='Objective Function', c='y')
#ax2.plot(X_test, f_true, 'black', label='Objective Function', c='k')
#ax3.plot(X_test, yhat_full,linestyle='dashed', label='Prediction from Full GPR', c='r')
#ax3.plot(X_test, f_true, 'black', label='Objective Function', c='k')


In [ ]:
#L, X_star, y_star, K_idx = gpr_subset_fit(X, y, noise_variance=0.2, k=k)

#print(L, X_star, y_star, K_idx)
#y_sd_pred, _ = qesgpr_predict_vanilla(L, X_test[K_idx], y_star, noise_variance=0.2, k=k)
y_sd_pred,s, _ = sd_predict(X, X_test, y, k)

y_sd_pred

In [ ]:
plt.scatter(X, y, label='Training examples', marker='*', color='blue', alpha=0.5)
plt.plot(X_test, f_true, label='Objective Function', c='k', lw=0.5)
plt.title('Noisy Training Data')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()

In [ ]:
#yhat, std, X_t_star = qesgpr_predict3(L, x_star, X_test_rand, y_star, noise_var, m)
#qesgpr_predict_vanilla(L, Xtest, y, noise_variance, k)

#k = int(len(X) * 0.5)
#L, x_star, y_star, K_idx = qesgpr_fit2(X, y, noise_var, m)
#L = qesgpr_fit(X, y, noise_var, k)
#yhat, std = qesgpr_predict_vanilla(L,x_star, y_star, noise_var, m)

#qe_ci = yhat * std/np.sqrt(len(yhat))


yhat, s, inducing_qbo = qbo_predict(X, X_test, y, k)


yhat

In [ ]:
plt.figure(figsize=(9,6))
plt.plot(X_test, f_true, label='Objective Function', c='k', lw=1.5)
plt.scatter(X, y, label='Training examples', marker='*', color='r', alpha=0.5)
plt.scatter(X[inducing_qbo], y[inducing_qbo], label='Inducing variables',marker='+', c='b')
plt.title('Training data with selected inducing variables')
plt.xlabel('x')
plt.ylabel('y')
#plt.xlim(-1.5, 1.5)
plt.legend()

plt.show();

In [ ]:
from sklearn.gaussian_process.kernels import RBF

#vanilla_kernel = RBF(length_scale=0.2)

#full_surrogate = GaussianProcessRegressor()

#full_surrogate.fit(X, y)
#print(full_surrogate.log_marginal_likelihood())

#yhat_full = full_surrogate.predict(X_test)
#L = qesgpr_fit(X, y, noise_var, k)

#yhat_full, _ =qesgpr_predict_vanilla(L, X_test, y, noise_var, k)

gp = GaussianProcessRegression(kernel_function=rbf_kernel_)
gp.fit(X, y)

#X_test = np.array([[1.5], [2.5]])
yhat_full, sigma = gp.predict(X_test)

yhat_full

In [ ]:
#import GPy

#Z = np.random.rand(m,1)
#sparse_gpy = GPy.models.SparseGPRegression(X,y,Z=Z)
#sparse_gpy.optimize('bfgs')
#mu, var = sparse_gpy._raw_predict(X_test) #

#mu


In [ ]:
import pyGPs

pygps_m = pyGPs.GPR_FITC()
pygps_m.setData(X, y, value_per_axis=m)
pygps_m.optimize(numIterations=5)

y_pygps = pygps_m.predict(X_test)
y_pygps
#pygps_m.plot()



In [ ]:
import sparse_GP as sGP




(L, sigma, K, C, inv_C, Xs, Ys, LB_best, elapsed_time) = sGP.train(0.2, 0.2,
                                                                   X, y, n, m,
                                                                   5)



print("Condition of K: ", np.linalg.cond(K) )

# Print results
print('Maximum lower bound:', LB_best)
print('Hyperparameters:', L, sigma)
print('Elapsed time:', elapsed_time)

   # Make predictions
N_Star = len(X_test)               # No. points where we make predictions
Y_star_mean = np.zeros(N_Star)     # Initialise GP mean predictions
Y_star_std = np.zeros(N_Star) 

for indx, x_star_ in enumerate(X_test):
    Y_star_mean[indx], Y_star_std[indx] = sGP.predict(Xs, x_star_, L, sigma,
                                                Ys, K, C, inv_C, m)
    
Y_star_mean

#plt.figure()
#plt.plot(X_test, Y_star_mean, 'black', label='Sparse GP')
#plt.plot(X_test, Y_star_mean + 3 * Y_star_std, 'black')
#plt.plot(X_test, Y_star_mean - 3 * Y_star_std, 'black')
#plt.plot(X, y, '.', color='blue', label='Full dataset')
#plt.plot(Xs, Ys, 'o', markeredgecolor='black', markerfacecolor='red',
#         markeredgewidth=1.5, markersize=10, label='Sparse dataset')
#plt.xlabel('x')
#plt.legend()
#plt.show()

In [ ]:

plt.plot(X_test, yhat,linestyle='dashed', label='Prediction from Sparse GPR', c='g')
plt.plot(X_test, rff_y_pred,linestyle='dashed', label='Prediction from Fourier Features GPR', c='y')
#plt.fill_between(X_t_star[truncated_aspect1:-truncated_aspect1], (yhat[truncated_aspect1:-truncated_aspect1]-qe_ci[truncated_aspect1:-truncated_aspect1]), (yhat[truncated_aspect1:-truncated_aspect1]+qe_ci[truncated_aspect1:-truncated_aspect1]), color='b', alpha=.1)
#plt.plot(X_test, Y_star_mean, 'orange', label='Prediction from Variational Sparse GP')
plt.plot(X_test, yhat_full,linestyle='dashed', label='Prediction from Full GPR', c='r')
#plt.plot(X_test, y_pygps[0],linestyle='dashed', label='Prediction from FITC', c='blue')
#plt.plot(X, y, label='Training examples', c='r')
plt.title('Model Predictions')
plt.plot(X_test, f_true, 'black', label='Objective Function', c='k')
plt.xlabel('x')
plt.ylabel('y')
#plt.xlim(-1.5, 1.5)
plt.legend()

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import wasserstein_distance

In [ ]:
print("Sparse GPR mean squared Error", mean_squared_error (yhat, f_true, squared=False))
print("Sparse GPR R2 Score",r2_score(yhat, f_true))
print("Wasserstein distance", wasserstein_distance(yhat.flatten(), yhat_full.flatten()))

#print(yhat, y_star)

In [ ]:

print("FITC GPR mean squared Error",mean_squared_error (y_pygps[0], f_true, squared=False))
print("FITC GPR R2 Score",r2_score(y_pygps[0], f_true))
print("Wasserstein distance", wasserstein_distance(y_pygps[0].flatten(), yhat_full.flatten()))


In [ ]:

print("unfiltered")
print("Full GPR mean squared Error",mean_squared_error (yhat_full, f_true, squared=False))
print("Full GPR R2 Score",r2_score(yhat_full, f_true))
print("Wasserstein distance", wasserstein_distance(yhat_full.flatten(), yhat_full.flatten()))

In [ ]:


print("Variational Sparse GPR mean squared Error",mean_squared_error (Y_star_mean, f_true, squared=False))
print("Variational GPR R2 Score", r2_score(Y_star_mean, f_true))
print("Wasserstein distance", wasserstein_distance(Y_star_mean.flatten(), yhat_full.flatten()))


In [ ]:

print("Fourier features GPR mean squared Error",mean_squared_error (rff_y_pred, f_true, squared=False))
print("Fourier features GPR  R2 Score", r2_score(rff_y_pred, f_true,))
print("Wasserstein distance", wasserstein_distance(rff_y_pred.flatten(), yhat_full.flatten()))



In [ ]:
print("SD GPR mean squared Error",mean_squared_error (y_sd_pred, f_true, squared=False))
print("SD GPR  R2 Score", r2_score(y_sd_pred, f_true,))
print("Wasserstein distance", wasserstein_distance(y_sd_pred.flatten(), yhat_full.flatten()))



In [ ]:


n = 100


np.random.seed(seed)
out = []
m_sizes =[2, 5,10, 20, 30, 40, 50]

X = np.linspace(-1.0, 1.0, n).reshape(-1, 1)
y = func(X) + sigma_y * np.random.randn(n).reshape(-1, 1)


#Extend to multidimensional
#X = np.random.uniform(low=-1. , high= 1. , size= (n,c))
#eedy = func(X) + sigma_y * np.random.randn(n).reshape(-1, 1)

# Test data
X_test = np.linspace(-1.0, 1.0, n).reshape(-1, 1)
f_true = func(X_test).reshape(-1, 1)
for m in m_sizes:
    print(m)
    
    m_list = []
    
    rff_l = []
    qbo_l = []
    sd_l = []
    
    for _ in range(3):
    
        #rff_predict(X_train, X_test, output_dim=200, k=k, kernel=rbf_kernel(X_train,X))
        rff_y_pred,rff_s, inducing_indices = rff_predict(X, X_test, y, output_dim, k=m)
        sd_y_pred,sd_s, _ = sd_predict(X, X_test, y, k=m)

        qbo_yhat, qbo_s, inducing_qbo = qbo_predict(X, X_test, y, k=m)
        qbo_l.append(mean_squared_error (qbo_yhat, f_true, squared=False))
        rff_l.append(mean_squared_error (rff_y_pred, f_true, squared=False))
        sd_l.append(mean_squared_error (sd_y_pred, f_true, squared=False))
      
        
    m_list.append((np.mean(np.array(qbo_l)),np.std(np.array(qbo_l))))
    m_list.append((np.mean(np.array(sd_l)),np.std(np.array(sd_l))))
    m_list.append((np.mean(np.array(sd_l)),np.std(np.array(sd_l))))
    
    out.append(m_list)
        
    



    

In [ ]:
pd.DataFrame(out)

In [ ]:
import pandas as pd

mse_df = pd.DataFrame(, columns=['QBO', 'SD', 'RFF'])
mse_df


plt.plot(m_sizes, mse_df['QBO'],linestyle='dashed', label = 'QBO', c='g')

plt.plot(m_sizes, mse_df['SD'],linestyle='dashed',label = 'SD', c='r')

plt.plot(m_sizes, mse_df['RFF'],linestyle='dashed', label = 'RFF',c='b')

#plt.fill_between(X_t_star[truncated_aspect1:-truncated_aspect1], (yhat[truncated_aspect1:-truncated_aspect1]-qe_ci[truncated_aspect1:-truncated_aspect1]), (yhat[truncated_aspect1:-truncated_aspect1]+qe_ci[truncated_aspect1:-truncated_aspect1]), color='b', alpha=.1)
#plt.plot(X_test, Y_star_mean, 'orange', label='Prediction from Variational Sparse GP')
#plt.plot(X_test, yhat_full,linestyle='dashed', label='Prediction from Full GPR', c='r')
#plt.plot(X_test, y_pygps[0],linestyle='dashed', label='Prediction from FITC', c='blue')
#plt.plot(X, y, label='Training examples', c='r')
plt.title('MSE vs ratio of k/n')

plt.xlabel('ratio of k/n')


plt.legend()
plt.ylabel('mse')

plt.legend()

# More data visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.spatial.distance import cdist


def kernel(x1, x2, var_f, l):
    """Squared exponential kernel"""
    return var_f * np.exp((-cdist(x1, x2)**2) / (2*l**2))


n = 20  # number of test points
x_min = -5
x_max = 5
x_star = np.linspace(x_min*1.4, x_max*1.4, n).reshape(-1,1) # points we're going to make predictions at


jitter = 1e-10  # small number to ensure numerical stability (eigenvalues of K can decay rapidly)
def sample(mu, var, epsilon, N):
    """Generate N samples from a multivariate Gaussian \mathcal{N}(mu, var)"""
    L = np.linalg.cholesky(var + jitter*np.eye(var.shape[0])) # cholesky decomposition (square root) of covariance matrix
    f_post = mu + L @ np.random.normal(size=(n, N))
    return f_post


l = 2 # lengthscale hyper-parameter
var_f = 1  # signal variance hyper-parameter

Kss = kernel(x_star, x_star, var_f, l)  # prior covariance

# draw samples from the prior 
Ns = 10 # number of samples
f_prior = sample(0, Kss, jitter, N=Ns)

# compute standard deviation for plotting
std = np.sqrt(np.diag(Kss))

fig = plt.figure(figsize=(12, 12))
plt.subplots_adjust(wspace=0.3, hspace=0.5)

# plot the sampled values as functions of x_star
plt.subplot(2, 2, 1)
plt.plot(x_star, f_prior)
plt.title('%i samples from the GP prior' % Ns)
plt.fill_between(x_star.flatten(), 0-2*std, 0+2*std, label='$\pm$2 standard deviations of posterior', color="#dddddd")
plt.legend()

# visualise the covariance function
plt.subplot(2, 2, 2)
plt.title("Prior covariance $K(X_*, X_*)$")
plt.contourf(Kss)
plt.show()

In [ ]:

def f(x):
    """Latent function."""
    return 1.0 * np.cos(x * 4 * np.pi) + \
           0.3 * np.sin(x * 2 * np.pi) + \
           0.8 * np.cos(x **2 * 1.5 * np.pi) 



N = 40  # number of training points
x_train = np.random.uniform(x_min, x_max, size=(N, 1))  # training inputs
# x_train = np.linspace(x_min, x_max, N).reshape(N, 1)  # training inputs
y_train = f(x_train)  # training outputs (f) - these are noise free observations of the underlying function f

n =  40 # number of test points
x_min = -1
x_max = 1
x_star = np.linspace(x_min, x_max, n).reshape(-1,1) 

# plt.title('Underlying function $f(x)=sin(x)$ and dummy data set ')
plt.plot(x_star, f(x_star), color='lawngreen', label='Underlying function')
plt.scatter(x_train, y_train, color='b', label='Training data points')
plt.legend()
plt.show()

In [ ]:
plt.style.use('fast')

print(plt.style.available)

def gp_regression(X, y, k, x_star, s_f, l):
    # calculate mean
    Lxx = np.linalg.cholesky(k(X, X,s_f, l))
    a = np.linalg.solve(Lxx.T, np.linalg.solve(Lxx, y))
    mu = k(X, x_star, s_f, l).T @ a
    
    # calculate variance
    v = np.linalg.solve(Lxx, k(X, x_star, s_f, l))
    var = k(x_star, x_star,s_f, l) - v.T @ v

    return mu, var


l = 0.1 # lengthscale
var_f = 0.9 # signal variance
mu, var = gp_regression(x_train, y_train, kernel, x_star, var_f, l)



def plot_posterior(x_train, y_train, x_star, mu, var, num_samples=50, title="GP posterior"):
    std = np.sqrt(np.diag(var))  # square root the variance to get standard deviation
    f_post = sample(mu, var, jitter, N=num_samples)  # draw samples from posterior
    
    fig = plt.figure(figsize=(15, 15))
    plt.subplots_adjust(wspace=0.2, hspace=0.5)

    # plot underlying function, training data, posterior mean and +/- 2 standard deviations
    plt.subplot(2, 2, 1)
    plt.title(title)
    plt.fill_between(x_star.flatten(), mu.flatten()-2*std, mu.flatten()+2*std, label='$\pm$2 standard deviations of posterior', color="#dddddd")
    plt.plot(x_star, f(x_star), 'b-', label='Underlying function')
    plt.plot(x_star, mu, 'r-', label='Mean of posterior')  # plot mean of posterior
    plt.plot(x_train, y_train, 'kx', ms=8 ,label='Training input-target pairs')
    plt.legend()

    # plot samples from posterior
    plt.subplot(2, 2, 2)
    plt.title('%i samples from GP posterior' % num_samples)
    plt.plot(x_star, f_post)  # plot samples from posterior
    plt.plot(x_train, y_train, 'kx', ms=8 ,label='Training input-target pairs $\{(x_i, y_i | i = 1,...,n)\}$')
    plt.show()

plot_posterior(x_train, y_train, x_star, mu, var, num_samples=25, title="Posterior - noise free observations")

In [ ]:
def g_regression(X, y, x_star, s_f, l):
    
    K = rbf_kernel(X, X, l2=0.2)
    # calculate mean
    Lxx = np.linalg.cholesky(K)
    a = np.linalg.solve(Lxx.T, np.linalg.solve(Lxx, y))
   # mu = k(X, x_star, s_f, l).T @ a
    mu = rbf_kernel(X, x_star,l2=0.2)
    
    # calculate variance
    v = np.linalg.solve(Lxx, rbf_kernel(X, x_star,l2=0.2))
    
    var = rbf_kernel(x_star, x_star,l2=0.2) - v.T @ v

    return mu, var

def qbo_predict(X_train, X_test, y_train, k=20):

#set up prior -Here we filter down the initial samples as well as xtest
    K = rbf_kernel(X_train, X_train,l2=0.2)
    #print(K, y_train)
    bqm = dimod.BinaryQuadraticModel.empty(dimod.BINARY)
    for i in range(len(K)):
        for j in range(len(K)):
            if i == j:
                #bqm.add_variable(i, -np.abs(y[i]))
                #bqm.add_variable(i, np.abs(K[i, i]))
                bqm.add_variable(i, -np.abs(y_train[i]))
                
            elif i > j:
                bqm.add_interaction(i, j, 10 *-np.abs(K[i, j] ))
                #bqm.add_interaction(i, j, -np.abs(np.linalg.norm(X[i] - X[j])))
                #print(np.abs(stats.pearsonr(X[i] ,X[j]))
                #bqm.add_interaction(i, j, np.abs(stats.pearsonr(X[i] ,X[j])))
    bqm.update(dimod.generators.combinations(list(range(len(K))), k, strength=1000))            
    sampler = neal.SimulatedAnnealingSampler()
    sample_ = sampler.sample(bqm)
    sample= sample_.first.sample
    
    #filter data by coefficients
    K_idx = np.where(np.array(list(sample.values())) ==1)
    #print(len(K_idx[0]))
    #print(K_idx)
    
    X_train_inducing, y_train_inducing = X_train[K_idx], y_train[K_idx]
    
    gp = GaussianProcessRegression(kernel_function=rbf_kernel_)
    gp.fit(X_train_inducing, y_train_inducing)

    y_pred, sigma = gp.predict(X_test[K_idx])
    #y_pred, sigma = g_regression(X_train_inducing, y_train_inducing, X_test[K_idx],s_f=1, l=0.2)
    
    return y_pred, sigma, K_idx[0]


    
    
mu, s, inducing_idx = qbo_predict(x_train, x_star, y_train, k=20)


print(inducing_idx)

#l = 1 # lengthscale
#var_f = 1  # signal variance
mu, var = gp_regression(x_train[inducing_idx], y_train[inducing_idx], kernel, x_star, var_f, l)

plot_posterior(x_train[inducing_idx], y_train[inducing_idx], x_star, mu, var, num_samples=50, title="Posterior - noise free observations")


# Kernel viz

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split



X, y = sklearn.datasets.make_friedman1(n_samples=75, n_features=5,  noise=5., random_state=None)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)



plt.rcParams.update({'font.size': 8})
for i in plt.style.available:
    plt.style.use(i)
    print(i)
    K = rbf_kernel(X_train, X_train, l2=0.2)
    
    
    
    
    K_star = rbf_kernel(X_train[inducing_idx], X_train[inducing_idx],  l2=0.1)

    ig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
    plt.subplots_adjust(wspace=0.2, hspace=0.5)
    #ax1.subplot(4, 4, 1)
    ax1.contourf(K)
    ax1.set_title('RBF Kernel-Full')
    #ax2.subplot(4, 4, 2)
    ax2.contourf(K_star)
    ax2.set_title('RBF Kernel-Sparse')

    K = qkr.get_kernel(X_train)
    K_star = qkr.get_kernel(X_train[inducing_idx])


    #ax3.subplots_adjust(wspace=0.2, hspace=0.5)
    #ax3.subplot(4, 4, 3)
    ax3.contourf(K)
    ax3.set_title('Quantum Kernel-Full')
   # ax2.subplot(4, 4, 2)
    ax4.contourf(K_star)
    ax4.set_title('Quantum Kernel-Sparse')


    plt.show()





In [ ]:
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

def func(x):
    """Latent function."""
    return np.sum(1.0 * np.cos(x * 4 * np.pi) + \
           0.3 * np.sin(x * 3 * np.pi) + \
           0.5 * np.cos(x * 2 * np.pi))
l = 0.2

r_min, r_max = -5, 5.
    
maxmin_bounds = (r_min, r_max)

xsamples=np.random.uniform(r_min, r_max, size = (100, 3))


#Get obj function values to train surrogate
ysamples=np.array([func(x) for x in xsamples])
#print(ysamples)
# Compute the k

# Compute the kernel matrix
#K  = qkr.get_kernel(xsamples)

X_train, X_test, y_train, y_test = train_test_split(xsamples, ysamples, test_size=0.5, random_state=42)


qbo_ypred, qbo_s, inducing_idx = qbo_predict(X_train, X_test, y_train, k=25)

c_map = 'gist_ncar'

K = rbf_kernel(X_train, X_train, l2=l)
K_star = rbf_kernel(X_train[inducing_idx], X_train[inducing_idx],  l2=l)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
#ax1.subplot(4, 4, 1)
sns.heatmap(K , ax=ax1, cmap=c_map,annot=False)
ax1.set_title('RBF Kernel-Full')
#ax2.subplot(4, 4, 2)
ax2 = sns.heatmap(K_star , ax=ax2, cmap=c_map,annot=False)
ax2.set_title('RBF Kernel-Sparse')

K_q = qkr.get_kernel(X_train)
K_q_star = qkr.get_kernel(X_train[inducing_idx])


#ax3.subplots_adjust(wspace=0.2, hspace=0.5)
#ax3.subplot(4, 4, 3)
ax3 = sns.heatmap(K_q , ax=ax3, cmap=c_map,annot=False)
ax3.set_title('Quantum Kernel-Full')
#ax2.subplot(4, 4, 2)
ax4 = sns.heatmap(K_q_star,ax=ax4, cmap=c_map,annot=False)
ax4.set_title('Quantum Kernel-Sparse')


plt.show()

print(np.linalg.cond(K))
print(np.linalg.cond(K_star))
print(np.linalg.cond(K_q))
print(np.linalg.cond(K_q_star))


In [ ]:
q_l = []
s_l = []

for i in [5,10,15,20,25]:
    
    q = []
    s = []
    
    for _ in range(10):
    
        qbo_ypred, qbo_s, q_inducing_idx = qbo_predict(X_train, X_test, y_train, k=i)

        s_ypred, s_s, s_inducing_idx = sd_predict(X_train, X_test, y_train, k=i)

        c_map = 'gist_ncar'

        K = rbf_kernel(X_train, X_train, l2=0.2)
        K_qstar = rbf_kernel(X_train[q_inducing_idx], X_train[q_inducing_idx],  l2=0.1)
        K_s_star = rbf_kernel(X_train[s_inducing_idx], X_train[s_inducing_idx],  l2=0.1)

        #print(np.linalg.cond(K))
        q.append(np.linalg.cond(K_qstar))
        s.append(np.linalg.cond(K_s_star))
        
    q_l.append(np.mean(np.array(q)))
    s_l.append(np.mean(np.array(s)))

    
df = pd.DataFrame()
df['QBO'] = q_l
df['SD'] =s_l

df.plot()

In [30]:
import numpy as np

def nystrom_approximation(kernel_matrix, m):
    """
    Approximate the kernel matrix using the Nystrom method.

    Parameters:
    - kernel_matrix: The full kernel matrix of shape (n, n)
    - m: The number of columns to be used for the approximation

    Returns:
    - Approximated kernel matrix of shape (n, n)
    """

    n = kernel_matrix.shape[0]
    if m > n:
        raise ValueError("m should be less than or equal to n")

    # Randomly sample m indices from the kernel matrix
    idx = np.random.choice(n, m, replace=False)
    print(idx)

    # Extract the submatrices
    W = kernel_matrix[idx][:, idx]
    C = kernel_matrix[:, idx]

    # Compute the pseudo-inverse of W
    W_inv = np.linalg.pinv(W)

    # Approximate the kernel matrix
    approx_kernel = C.dot(W_inv).dot(C.T)

    return approx_kernel

# Example usage:

# Create a random kernel matrix
X = np.random.rand(100, 100)
kernel_matrix = X @ X.T

# Approximate the kernel matrix using Nyström method with m=20
m = 20
approx_kernel = nystrom_approximation(kernel_matrix, m)

print(np.linalg.norm(approx_kernel- kernel_matrix))


[56 86 91 57 55 73 36 28  4 66 39 80 17 70 40 99 13  1 95 38]
88.8231060058837


In [31]:
def q_approximation(kernel_matrix, X, y, m):
    """
    Approximate the kernel matrix using the Nystrom method.

    Parameters:
    - kernel_matrix: The full kernel matrix of shape (n, n)
    - m: The number of columns to be used for the approximation

    Returns:
    - Approximated kernel matrix of shape (n, n)
    """
    r = 10
    n = kernel_matrix.shape[0]
    if m > n:
        raise ValueError("m should be less than or equal to n")
    #print(X.shape[0])

    
    #print(K, y_train)
    bqm = dimod.BinaryQuadraticModel.empty(dimod.BINARY)
    for i in range(len(kernel_matrix)):
        for j in range(len(kernel_matrix)):
            if i == j:
                bqm.add_variable(i, -np.abs(y[i]))
                #bqm.add_variable(i, -np.abs(kernel_matrix[i, i]))
                #bqm.add_variable(i, -np.abs(kernel_matrix[i,j]))
                
            elif i > j:
                bqm.add_interaction(i, j, 1000*np.abs(kernel_matrix[i,j] ))
                #bqm.add_interaction(i, j, -np.abs(np.linalg.norm(X[i] - X[j])))
                #print(np.abs(stats.pearsonr(X[i] ,X[j]))
                #bqm.add_interaction(i, j, np.abs(stats.pearsonr(X[i] ,X[j])))
   # print(bqm)            
    bqm.update(dimod.generators.combinations(list(range(len(kernel_matrix))), m, strength=1000))            
    sampler = neal.SimulatedAnnealingSampler()
    sample_ = sampler.sample(bqm)
    sample= sample_.first.sample
    
    #filter data by coefficients
    idx = np.where(np.array(list(sample.values())) ==1)[0]
    
   
    print(idx)
    #print(len(K_idx[0]))
    

    # Extract the submatrices
    W = kernel_matrix[idx][:, idx]
    C = kernel_matrix[:, idx]

    # Compute the pseudo-inverse of W
    W_inv = np.linalg.pinv(W)

    # Approximate the kernel matrix
    approx_kernel = C.dot(W_inv).dot(C.T)

    return approx_kernel

# Create a random kernel matrix
X = np.random.rand(100, 100)
y = np.random.rand(100, 1)
print(y.shape)

kernel_matrix = X @ X.T

# Approximate the kernel matrix using Nyström method with m=20
m = 20
approx_kernel = q_approximation(kernel_matrix, X, y, m)

print(np.linalg.norm(approx_kernel- kernel_matrix))





(100, 1)
[51 57 93]
269.4884027458053


In [32]:
import sklearn
from sklearn.datasets import make_spd_matrix
np.random.seed(0)
#X = np.random.randn(100, 2)

X = make_spd_matrix(n_dim=20, random_state=None)

def func(x):
    """Latent function."""
    return np.sum(1.0 * np.cos(x * 4 * np.pi) + \
           0.3 * np.sin(x * 3 * np.pi) + \
           0.5 * np.cos(x * 2 * np.pi))
l2 = 0.2

r_min, r_max = -5, 5.
    
maxmin_bounds = (r_min, r_max)

xsamples=np.random.uniform(r_min, r_max, size = (50, 3))


#Get obj function values to train surrogate
ysamples=np.array([func(x) for x in xsamples])
#print(ysamples)
# Compute the k

# Compute the kernel matrix
K  = qkr.get_kernel(xsamples)
#print(sorted(np.linalg.eigvalsh(K))[::-1])

nlist_mean= []
qlist_mean = []
nlist_sd= []
qlist_sd= []

x_range = np.linspace(2, 10, 4)
for i in x_range:
    
    nk = []
    qk_ = []
    
    for _ in range(10):
    
        m = int(i)
        print(m)
        #m=50
        #kernel_matrix = rbf_kernel(X, X, l2=i)
        n_k= nystrom_approximation(K, m)

        q_k= q_approximation(K, xsamples, ysamples, m)
        #print(np.linalg.norm(q_k- kernel_matrix))
        
        nk.append(np.linalg.norm(K-n_k))
        qk_.append(np.linalg.norm(K-q_k))

    nlist_mean.append(np.array(nk).mean())
    qlist_mean.append(np.array(qk_).mean())
    
    nlist_sd.append(np.array(nk).std())
    qlist_sd.append(np.array(qk_).std())
    
#df =pd.DataFrame()

#df['QBO'] = qlist_mean
#df['SD'] = nlist_mean

yerr0 = np.array(qlist_mean) - 0.3 *np.array(qlist_sd)
yerr1 = np.array(qlist_mean) + 0.3 *np.array(qlist_sd)

yerr2 = np.array(nlist_mean) - 0.3 *np.array(nlist_sd)
yerr3 = np.array(nlist_mean) + 0.3 *np.array(nlist_sd)

fig, ax = plt.subplots()

ax.plot(x_range, qlist_mean, color='C0', label="QBO")
plt.fill_between(x_range, yerr0, yerr1, color='C0', alpha=0.5)

ax.plot(x_range, nlist_mean, color='C1', label="SD")
plt.fill_between(x_range, yerr2, yerr3, color='C1', alpha=0.5)
plt.legend()
    
plt.show() 
    
    

2
[46 21]
[32 43]
2
[43 13]
[34 36]
2
[22 16]
[21 38]
2
[ 0 42]
[10 38]
2
[ 5 22]
[36 40]
2
[ 8 24]
[15 31]
2
[18 36]
[ 8 29]
2
[49  1]
[ 3 21]
2
[8 5]
[22 25]
2
[44 33]
[28 46]
4
[21  6  9  0]
[ 9 32 45 46]
4
[39  6 40 41]
[19 35 38 47]
4
[39 27  3 46]
[27 32 41 44]
4
[27 28 19 35]
[11 22 38 40]
4
[43  0 45 15]
[ 6 31 44 47]
4
[21  9 41 23]
[ 3 12 20 43]
4
[15 34 43 30]
[ 8 31 36 41]
4
[ 3 46 28  2]
[10 11 39 47]
4
[ 6 45 16 37]
[11 14 22 46]
4
[32 23 34 28]
[ 8 10 16 38]
7
[24 49 42 11  3  1 34]
[ 5 11 12 13 28 43 47]
7
[14 16  2 47 30  7 17]
[ 3  9 16 21 39 41 49]
7
[31 30  9 49 19 44 26]
[ 5 10 14 20 41 42 44]
7
[17 13 39 32 43 21 27]
[ 4  5 16 18 26 36 46]
7
[43  7 47 24 20 26  5]
[ 2 15 19 32 37 40 47]
7
[14 44 12 22 20 33 16]
[ 3 18 24 27 29 32 41]
7
[46 36 12 43 24 13 21]
[ 2  5 20 26 32 34 41]
7
[15 10 39 44 32  2 47]
[ 3 30 35 37 38 41 43]
7
[44  1 45 12 36  8 24]
[ 4  9 29 37 39 41 49]
7
[14 23 34 13 22  5 41]
[12 27 31 33 38 46 47]
10
[ 3 25 39 21 27  7 47 42  8 32]
[ 6 20 

NameError: name 'plt' is not defined

In [33]:
import numpy as np

# Define kernel function (RBF Kernel)
def rbf_kernel(x, y, lengthscale=1.0):
    return np.exp(-0.5 * np.sum((x - y) ** 2) / lengthscale ** 2)

# Nystrom approximation
def nystrom_approximation(X, kernel_func, m):
    n = X.shape[0]
    X_sample = X[np.random.choice(n, m, replace=False)]
    
    K_mm = np.zeros((m, m))
    for i in range(m):
        for j in range(m):
            K_mm[i, j] = kernel_func(X_sample[i], X_sample[j])
            
    K_nm = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            K_nm[i, j] = kernel_func(X[i], X_sample[j])
            
    W_inv = np.linalg.pinv(K_mm)
    
    return K_nm @ W_inv @ K_nm.T

# GPR with Nystrom approximation
def nystrom_gpr(X_train, y_train, X_test, kernel_func, m, sigma_noise=1e-3):
    K_approx = nystrom_approximation(X_train, kernel_func, m)
    Ky = K_approx + sigma_noise**2 * np.eye(K_approx.shape[0])
    alpha = np.linalg.solve(Ky, y_train)

    K_star = np.zeros((X_test.shape[0], X_train.shape[0]))
    for i in range(X_test.shape[0]):
        for j in range(X_train.shape[0]):
            K_star[i, j] = kernel_func(X_test[i], X_train[j])
            
    y_pred = K_star @ alpha
    return y_pred

# Generate synthetic data (for testing)
np.random.seed(42)
X_train = np.sort(np.random.rand(100, 1) * 10, axis=0)
y_train = func(X_train)

X_test = np.linspace(0, 10, 500).reshape(-1, 1)
y_test = func(X_test)

# Predict using Nystrom GPR
y_pred = nystrom_gpr(X_train, y_train, X_test, rbf_kernel, m=20)

print(X_train.ravel().shape)
print(y_train.shape)

# Plot results
import matplotlib.pyplot as plt
plt.scatter(X_train.ravel(), y_train, s=50, c='k', marker='x', label='Training data')
plt.plot(X_test, y_pred, 'r-', label='Nystrom GPR')
plt.plot(X_test, y_test, 'b-', label='Ground truth')
plt.legend()
plt.title("Gaussian Process Regression with Nystrom Approximation")
plt.xlabel("X")
plt.ylabel("y")
plt.show()


ValueError: solve: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (m,m),(m,n)->(m,n) requires 2)

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

def rbf_kernel(x, y, l2):
    sqdist = np.sum(x**2,1).reshape(-1,1) + \
             np.sum(y**2,1) - 2*np.dot(x, y.T)
    return np.exp(-.5 * (1/l2) * sqdist)

# Compute the kernel matrix
def compute_kernel_matrix(X, kernel_func):
    n_samples = X.shape[0]
    K = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        for j in range(n_samples):
            K[i, j] = kernel_func(X[i], X[j])
    return K

# Generate some sample data (for demonstration purposes)
np.random.seed(0)
#X = np.random.randn(100, 2)

#X = sklearn.datasets.make_spd_matrix(n_dim=10, random_state=None)
xsamples=np.random.uniform(r_min, r_max, size = (100, 3))




#Get obj function values to train surrogate
ysamples=np.array([func(x) for x in xsamples])
#file = open("./stilted_data_diabetes.pkl",'rb')
#[x_train_pqk, x_test_pqk, x_train, x_test, y_train, y_test] = pickle.load(file)
#X_train = np.array(x_train_pqk).reshape(-1,(np.array(x_train_pqk).shape[1]* np.array(x_train_pqk).shape[2]) )
#X_test = np.array(x_test_pqk).reshape(-1,(np.array(x_test_pqk).shape[1]* np.array(x_test_pqk).shape[2]) )
#print(ysamples)
# Compute the k

# Compute the kernel matrix
#K  = qkr.get_kernel(xsamples)

X_train, X_test, y_train, y_test = train_test_split(xsamples, ysamples, test_size=0.5, random_state=42)


qbo_ypred, qbo_s, inducing_idx = qbo_predict(X_train, X_test, y_train, k=25)

c_map = 'gist_ncar'

ls = 5.
qk



# Compute the kernel matrix
#K  = qkr.get_kernel(X)
#K_r  = rbf_kernel(X, X, l2=0.2)
print(K_r)
# Compute the eigenvalues
eigenvalues_q = np.linalg.eigvalsh(K_q)

# Sort eigenvalues in descending order
sorted_eigenvalues_q = np.sort(eigenvalues_q)[::-1]

print(np.mean(sorted_eigenvalues_q[:-1] / sorted_eigenvalues_q[1:]))

eigenvalues_qs = np.linalg.eigvalsh(K_q_star)


# Sort eigenvalues in descending order
sorted_eigenvalues_qs = np.sort(eigenvalues_qs)[::-1]
print(np.mean(sorted_eigenvalues_qs[:-1] / sorted_eigenvalues_qs[1:]))


eigenvalues_r = np.linalg.eigvalsh(K_r)

# Sort eigenvalues in descending order
sorted_eigenvalues_r = np.sort(eigenvalues_r)[::-1]
print(np.mean(sorted_eigenvalues_r[:-1] / sorted_eigenvalues_r[1:]))

eigenvalues_rs = np.linalg.eigvalsh(K_star_r)

# Sort eigenvalues in descending order
sorted_eigenvalues_rs = np.sort(eigenvalues_rs)[::-1]
print(np.mean(sorted_eigenvalues_rs[:-1] / sorted_eigenvalues_rs[1:]))


# Plot the eigenvalues
plt.plot(sorted_eigenvalues_r, marker='o', label="RBF")
plt.plot(sorted_eigenvalues_rs, marker='o', label="Sparse RBF")
plt.plot(sorted_eigenvalues_q, marker='o', label="QK")
plt.plot(sorted_eigenvalues_qs, marker='o', label="Sparse QK")
plt.xlabel('Index')
plt.ylabel('Eigenvalue')
plt.title('Decay of Eigenvalues of the Kernel Matrix')
plt.grid(True)
plt.legend()
plt.show()


NameError: name 'r_min' is not defined

In [ ]:
#K = rbf_kernel(X, X, l2=0.2)
#K_star = rbf_kernel(X_train[inducing_idx], X_train[inducing_idx],  l2=0.1)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
#ax1.subplot(4, 4, 1)
sns.heatmap(K_r , ax=ax1, cmap=c_map,annot=False)
ax1.set_title('RBF Kernel-Full')
#ax2.subplot(4, 4, 2)
ax2 = sns.heatmap(K_star_r , ax=ax2, cmap=c_map,annot=False)
ax2.set_title('RBF Kernel-Sparse')

#K_q = qkr.get_kernel(X_train)
#K_q_star = qkr.get_kernel(X_train[inducing_idx])


#ax3.subplots_adjust(wspace=0.2, hspace=0.5)
#ax3.subplot(4, 4, 3)
ax3 = sns.heatmap(K_q , ax=ax3, cmap=c_map,annot=False)
ax3.set_title('Quantum Kernel-Full')
#ax2.subplot(4, 4, 2)
ax4 = sns.heatmap(K_q_star,ax=ax4, cmap=c_map,annot=False)
ax4.set_title('Quantum Kernel-Sparse')


plt.show()

print(np.linalg.cond(K_r))
print(np.linalg.cond(K_star_r))
print(np.linalg.cond(K_q))
print(np.linalg.cond(K_q_star))

In [ ]:
# Install Qiskit if you haven't already
# !pip install qiskit

import qiskit
from qiskit import QuantumCircuit, Aer, transpile
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import ZZFeatureMap
import numpy as np
import matplotlib.pyplot as plt

# Create a quantum circuit that prepares entangled states
def create_entangled_circuit(n_qubits):
    qc = QuantumCircuit(n_qubits)
    for i in range(n_qubits):
        qc.h(i)
    for i in range(n_qubits - 1):
        qc.ry(i, i + 1)
    for i in range(n_qubits):
        qc.h(i)
    for i in range(n_qubits - 1):
        qc.rx(i, i + 1)
    for i in range(n_qubits - 1):
        qc.cnot(i, i + 1)
    return qc

n_qubits = 5
qc = create_entangled_circuit(n_qubits)
qc.draw('mpl')

# Quantum kernel: Use the overlap of two quantum states
def quantum_kernel(circuit1, circuit2):
    # Combine circuits
    combined_circuit = circuit1 + circuit2.inverse()
    
    # Run the combined circuit on a simulator
    backend = Aer.get_backend('statevector_simulator')
    t_qc = transpile(combined_circuit, backend)
    qobj = qiskit.assemble(t_qc)
    result = backend.run(qobj).result()
    
    # Compute overlap
    statevector = result.get_statevector()
    overlap = np.abs(np.dot(statevector, statevector.conj()))
    
    return overlap

# Generate data samples using quantum circuits
samples = [create_entangled_circuit(n_qubits) for _ in range(10)]

# Compute kernel matrix
kernel_matrix = np.zeros((len(samples), len(samples)))
for i in range(len(samples)):
    for j in range(len(samples)):
        kernel_matrix[i, j] = quantum_kernel(samples[i], samples[j])

# Compute eigenvalues
eigenvalues = np.linalg.eigvalsh(kernel_matrix)


# Sort and plot eigenvalues
sorted_eigenvalues = np.sort(eigenvalues)[::-1]
print(np.sum(np.diff(sorted_eigenvalues)))
plt.plot(sorted_eigenvalues, marker='o')
plt.xlabel('Index')
plt.ylabel('Eigenvalue')
plt.title('Eigenvalue Decay of Quantum Kernel Matrix')
plt.grid(True)
plt.show()
qc.draw('mpl')


In [ ]:
kernel_matrix

In [ ]:
import numpy as np
import qiskit
from qiskit import Aer, transpile, assemble
from qiskit.circuit import ParameterVector
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# 1. Define a parameterized quantum circuit
def parameterized_circuit(n_qubits, params):
    assert len(params) == n_qubits
    
    qc = qiskit.QuantumCircuit(n_qubits)
    for i, theta in enumerate(params):
        qc.rx(theta, i)
    for i in range(n_qubits-1):
        qc.cx(i, i+1)
    for i, theta in enumerate(params):
        qc.ry(theta, i)
    return qc

# 2. Quantum kernel
def quantum_kernel(circuit1, circuit2):
    combined_circuit = circuit1 + circuit2.inverse()
    combined_circuit.measure_all()
    backend = Aer.get_backend('qasm_simulator')
    t_qc = transpile(combined_circuit, backend)
    qobj = assemble(t_qc, shots=8192)
    result = backend.run(qobj).result()
    counts = result.get_counts()
    # Extract the probability of the all-zero bitstring
    all_zero_prob = counts.get('0'*circuit1.num_qubits, 0) / sum(counts.values())
    return all_zero_prob

# 3. Compute kernel matrix
def compute_kernel_matrix(samples):
    kernel_matrix = np.zeros((len(samples), len(samples)))
    for i in range(len(samples)):
        for j in range(len(samples)):
            kernel_matrix[i, j] = quantum_kernel(samples[i], samples[j])
    return kernel_matrix

# 4. Define the cost function
def eigenvalue_decay_cost(params, n_qubits, samples):
    parameterized_samples = [parameterized_circuit(n_qubits, params) for _ in samples]
    kernel_matrix = compute_kernel_matrix(parameterized_samples)
    print(kernel_matrix)
    eigenvalues = np.linalg.eigvalsh(kernel_matrix)
    sorted_eigenvalues = np.sort(eigenvalues)[::-1]
    cost = -np.sum(np.diff(sorted_eigenvalues))
    return cost

# Random data samples
n_qubits = 4
n_samples = 10
samples = [np.random.randn(n_qubits) for _ in range(n_samples)]

# 5. Optimize the cost function
initial_params = np.random.rand(n_qubits)
result = minimize(eigenvalue_decay_cost, initial_params, args=(n_qubits, samples), method='Powell')
optimized_params = result.x

print("Optimized Parameters:", optimized_params)

optimized_samples = [parameterized_circuit(n_qubits, optimized_params) for _ in samples]
kernel_matrix = compute_kernel_matrix(optimized_samples)
eigenvalues = np.linalg.eigvalsh(kernel_matrix)
sorted_eigenvalues = np.sort(eigenvalues)[::-1]

plt.plot(sorted_eigenvalues, marker='o')
plt.xlabel('Index')
plt.ylabel('Eigenvalue')
plt.title('Eigenvalue Decay of Optimized Quantum Kernel Matrix')
plt.grid(True)
plt.show()


In [34]:
feature_dim = 2  # assuming 2-dimensional data for simplicity
feature_map = ZZFeatureMap(feature_dimension=feature_dim)

# Define quantum instance
backend = QuantumInstance(Aer.get_backend('statevector_simulator'))

# Example usage
#X_train = np.array([[1., 1.], [2., 2.], [3., 3.]])
#y_train = np.array([1., 3., 2.])

qkr = QuantumKernelRegression(feature_map=feature_map, backend=backend)

In [ ]:
K = qkr.get_kernel(X_train)
plt.contourf(K)

for i in plt.style.available:

    plt.style.use(i)
    print(i)
    K = qkr.get_kernel(X_train)
    K_star = qkr.get_kernel(X_train[inducing_idx])

    fig = plt.figure(figsize=(15, 15))
    plt.subplots_adjust(wspace=0.2, hspace=0.5)
    plt.subplot(2, 2, 1)
    plt.contourf(K)
    plt.subplot(2, 2, 2)
    plt.contourf(K_star)
    plt.show()


In [ ]:
# visualize trianing splits

In [ ]:
from sklearn.datasets import 

In [ ]:
X, y = sklearn.datasets.make_friedman1(n_samples=50, n_features=5,  noise=5., random_state=None)

qbo = []
rff = []
sd = []
qk = []
qkq = []

for m in reversed([5, 10 , 15,20, 25]):
    
    
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1)
    
     
    
    #for _ in range(5):
    
    qbo_avg , rff_avg, sd_avg, qk_avg, qkq_avg = [], [], [] ,[] ,[]

    qbo_y, qbo_s, qbo_i = qbo_predict(X_train, X_test, y_train, k=m)
    sd_y, sd_s, sd_i = sd_predict(X_train, X_test, y_train, k=m)
    rff_y, rff_s, rff_i = rff_predict(X_train, X_test, y_train, output_dim =500, k=m)
    qk_y, qk_s, qk_i = qk_predict(X_train, X_test, y_train, k=m)
    qkq_y, qkq_s, qkq_i = qk_predict_qbo(X_train, X_test, y_train, k=m)

    #    for j, z in zip([qbo_avg, rff_avg, sd_avg, qk_avg, qkq_avg], [qbo_y, rff_y, sd_y, qk_y, qkq_y]):
    #        j.append(mean_squared_error(y_test, z))
    for j, z in zip([qbo_avg, rff_avg, sd_avg, qk_avg, qkq_avg], [qbo_y, rff_y, sd_y, qk_y, qkq_y]):
            j.append(mean_squared_error(y_test, z))
    qbo.append(np.mean(np.array(qbo_avg)))
    rff.append(np.mean(np.array(rff_avg)))
    sd.append(np.mean(np.array(sd_avg)))
    qk.append(np.mean(np.array(qk_avg)))
    qkq.append(np.mean(np.array(qkq_avg)))
    
    
df =pd.DataFrame()

df['QBO'] =qbo
df['RFF'] = rff
df['SD'] =sd
df['QKGPR'] = qk
df['Q-QKGPR'] = qkq

df.plot()

In [ ]:

    #print(K, y_train)
    bqm = dimod.BinaryQuadraticModel.empty(dimod.BINARY)
    for i in range(len(K)):
        for j in range(len(K)):
            if i == j:
                #bqm.add_variable(i, -np.abs(y[i]))
                #bqm.add_variable(i, np.abs(K[i, i]))
                bqm.add_variable(i, -np.abs(y_train[i]))
                
            elif i > j:
                bqm.add_interaction(i, j, 10 *np.abs(K[i, j] ))
                #bqm.add_interaction(i, j, -np.abs(np.linalg.norm(X[i] - X[j])))
                #print(np.abs(stats.pearsonr(X[i] ,X[j]))
                #bqm.add_interaction(i, j, np.abs(stats.pearsonr(X[i] ,X[j])))
    bqm.update(dimod.generators.combinations(list(range(len(K))), k, strength=1000))            
    sample_ = sampler.sample(bqm)
    sample= sample_.first.sample
    

# Test Stilted dataset

In [ ]:
file = open("./stilted_data_random_regression.pkl",'rb')
[x_train_pqk, x_test_pqk, x_train, x_test, y_train_new, y_test_new] = pickle.load(file)

In [ ]:
x_train_pqk_reshaped = np.array(x_train_pqk).reshape(-1,(np.array(x_train_pqk).shape[1]* np.array(x_train_pqk).shape[2]) )
x_test_pqk_reshaped = np.array(x_test_pqk).reshape(-1,(np.array(x_test_pqk).shape[1]* np.array(x_test_pqk).shape[2]) )

In [ ]:
m =5

qbo_out = []

sd_out = []
rff_out= []
rff_outq= []

qk_out = []
qkq_out = []

vgpr_out =[]

for _ in range(5):
    print(_)

    qbo_y, qbo_s, qbo_i = qbo_predict(x_train_pqk_reshaped, x_test_pqk_reshaped, y_train_new, k=m)
    sd_y, sd_s, sd_i = sd_predict(x_train_pqk_reshaped, x_test_pqk_reshaped, y_train_new, k=m)
    rff_y, rff_s, rff_i = rff_predict(x_train_pqk_reshaped, x_test_pqk_reshaped, y_train_new, output_dim =500, k=m)
    rff_yq, rff_sq, rff_iq = rff_predict_qbo(x_train_pqk_reshaped, x_test_pqk_reshaped, y_train_new, output_dim =500, k=m)
    qk_y, qk_s, qk_i = qk_predict(x_train_pqk_reshaped, x_test_pqk_reshaped, y_train_new, k=m)
    qkq_y, qkq_s, qkq_i = qk_predict_qbo(x_train_pqk_reshaped, x_test_pqk_reshaped, y_train_new, k=m)
    
    vgpr_y, vgpr_s = vgpr_predict(x_train_pqk_reshaped, x_test_pqk_reshaped, y_train_new, k=m)
    
    
    qbo_out.append(mean_squared_error(y_test_new,qbo_y))
    sd_out.append(mean_squared_error(y_test_new,sd_y))
    
    rff_out.append(mean_squared_error(y_test_new,rff_y))
    rff_outq.append(mean_squared_error(y_test_new,rff_yq))
    qk_out.append(mean_squared_error(y_test_new,qk_y))
    qkq_out.append(mean_squared_error(y_test_new,qkq_y))
    
    vgpr_out.append(mean_squared_error(y_test_new,vgpr_y))
    


print("qbo: ", (np.mean(np.array(qbo_out))))
print("sd: ", (np.mean(np.array(sd_out))))
print("rff: ", (np.mean(np.array(rff_out))))
print("rff-q: ", (np.mean(np.array(rff_outq))))
print("qk: ", (np.mean(np.array(qk_out))))
print("qkq: ", (np.mean(np.array(qkq_out))))
print('vgpr: ',(np.mean(np.array(vgpr_out))))

In [ ]:
m =5

qbo_out2 = []

sd_out2 = []
rff_out2 = []
rff_out2q = []
qk_out2  = []
qkq_out2 = []

vgpr_out2 =[]


for _ in range(5):
    print(_)
    qbo_y, qbo_s, qbo_i = qbo_predict(x_train, x_test, y_train_new, k=m)
    sd_y, sd_s, sd_i = sd_predict(x_train, x_test, y_train_new, k=m)
    rff_y, rff_s, rff_i = rff_predict(x_train, x_test, y_train_new, output_dim =500, k=m)
    rff_yq, rff_sq, rff_iq = rff_predict_qbo(x_train, x_test, y_train_new, output_dim =500, k=m)
    qk_y, qk_s, qk_i = qk_predict(x_train, x_test, y_train_new, k=m)
    qkq_y, qkq_s, qkq_i = qk_predict_qbo(x_train, x_test, y_train_new, k=m)
    vgpr_y, vgpr_s = vgpr_predict(x_train_pqk_reshaped, x_test_pqk_reshaped, y_train_new, k=m)
    
    qbo_out2.append(mean_squared_error(y_test_new,qbo_y))
    sd_out2.append(mean_squared_error(y_test_new,sd_y))
    
    rff_out2.append(mean_squared_error(y_test_new,rff_y))
    rff_out2q.append(mean_squared_error(y_test_new,rff_yq))
    qk_out2.append(mean_squared_error(y_test_new,qk_y))
    qkq_out2.append(mean_squared_error(y_test_new,qkq_y))
    
    vgpr_out2.append(mean_squared_error(y_test_new,vgpr_y))
    



print("qbo: ", (np.mean(np.array(qbo_out2))))
print("sd: ", (np.mean(np.array(sd_out2))))
print("rff: ", (np.mean(np.array(rff_out2))))
print("rff-q: ", (np.mean(np.array(rff_out2q))))
print("qk: ", (np.mean(np.array(qk_out2))))
print("qkq: ", (np.mean(np.array(qkq_out2))))
print('vgpr: ',(np.mean(np.array(vgpr_out2))))




In [ ]:
import pandas as pd
plt.style.use('seaborn-v0_8-whitegrid')

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(11, 4))
all_data  = pd.DataFrame({ 'QBO' : qbo_out, 'SD' :sd_out, 'VGPR' :vgpr_out, 'Q-RFF' : rff_outq,'RFF' : rff_out, 'QKGPR' : qk_out, 'Q-QKGPR' :qkq_out})

all_data2 = [qbo_out2, sd_out2, vgpr_out2, rff_out2q, rff_out2, qk_out2, qkq_out2]

label = ["", "QBO","SD", "VGPR", "Q-RFF","RFF","QKGPR","Q-QKGPR"]

violin = axs[0].violinplot(all_data,
                  showmeans=True,
                  showmedians=False)

axs[0].set_xticklabels(label)
axs[0].set_title('Stilted (PQK-encoded) Data')
                 
for pc, c in zip(violin["bodies"], ['magenta', 'brown','blue', 'orange','green', 'red']):
    pc.set_facecolor(c)
    pc.set_edgecolor("black")
    pc.set_alpha(0.5)
                 




violin2 = axs[1].violinplot(all_data2,
                  showmeans=True,
                  showmedians=False)
axs[1].set_xticklabels(label)
axs[1].set_title('Original (Classical) Data')

for pc, c in zip(violin2["bodies"], ['magenta', 'brown','blue', 'orange','green', 'red']):
    pc.set_facecolor(c)
    pc.set_edgecolor("black")
    pc.set_alpha(0.5)





# Visualize Bayesian posterior

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

f = Ackley(num_variables=2)


np.random.seed(0)
xs = np.random.rand(100)*10 - 5  # Random x values between -5 and 5
ys = np.random.rand(100)*10 - 5  # Random y values between -5 and 5

zs = np.array([f((x, y)) for x, y in zip(xs, ys)])  # z values computed from x and y


# Create a grid for interpolation
grid_x, grid_y = np.meshgrid(np.linspace(min(xs), max(xs), 100),
                             np.linspace(min(ys), max(ys), 100))

# Interpolate the z values to get a 2D grid of z-values
grid_z = griddata((xs, ys), zs, (grid_x, grid_y), method='cubic')

# Create the contour plot
plt.contourf(grid_x, grid_y, grid_z, 20, cmap='RdGy')
plt.colorbar()

plt.title("Ackley Function")
plt.xlabel("x0")
plt.ylabel("x1")

plt.show()

In [ ]:
np.random.seed(0)
xs = np.random.rand(100)*10 - 5  # Random x values between -5 and 5
ys = np.random.rand(100)*10 - 5  # Random y values between -5 and 5
zs = np.sin(np.sqrt(xs**2 + ys**2))  # z values computed from x and y
print(zs.shape)

# Create a grid for interpolation
grid_x, grid_y = np.meshgrid(np.linspace(min(xs), max(xs), 100),
                             np.linspace(min(ys), max(ys), 100))

# Interpolate the z values to get a 2D grid of z-values
grid_z = griddata((xs, ys), zs, (grid_x, grid_y), method='cubic')

# Create the contour plot
plt.contourf(grid_x, grid_y, grid_z, 20, cmap='RdGy')
plt.colorbar()
plt.scatter(xs, ys, color='red', marker='o')  # plotting the original points
plt.title("Contour Plot")
plt.xlabel("X-axis")
plt.ylabel("Y-axis")

plt.show()

##### Pennylane Q-kernel

In [ ]:
import pennylane as qml
from pennylane.templates import AngleEmbedding, StronglyEntanglingLayers
from pennylane.operation import Tensor

n_qubits = len(X[0])
n_qubits

In [ ]:
dev_kernel = qml.device("lightning.qubit", wires=n_qubits)

projector = np.zeros((2**n_qubits, 2**n_qubits))
projector[0, 0] = 1

@qml.qnode(dev_kernel, interface="autograd")
def kernel(x1, x2):
    """The quantum kernel."""
    AngleEmbedding(x1, wires=range(n_qubits))
    qml.adjoint(AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))

In [ ]:
def kernel_matrix(A, B):
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""
    return np.array([[kernel(a, b) for b in B] for a in A])

In [ ]:
K = kernel_matrix(X[:20], X[:20])
N = len(X)
K
eigenvalues = np.linalg.eigvalsh(K)
sorted_eigenvalues = np.sort(eigenvalues)[::-1]
sorted_eigenvalues

In [ ]:
print(np.linalg.cond(K))

In [ ]:
noise_var = 10
i_k =noise_var*np.eye(N)

L = np.linalg.cholesky(K + i_k)
#L = np.linalg.cholesky(K)

In [ ]:
yhat_qk, std = qesgpr_predict_vanilla(L, X_test, y, noise_var, m)

In [ ]:
print("Quantum-Kernel GPR mean squared Error", mean_squared_error(yhat_qk,f_true,squared=False))
print("Quantum-Kernel GPR R2 Score", r2_score(yhat_qk,f_true))
yhat_qk

In [ ]:

plt.plot(x_star, yhat,linestyle='dashed', label='Prediction from Sparse GPR', c='g')
#plt.fill_between(X_t_star[truncated_aspect1:-truncated_aspect1], (yhat[truncated_aspect1:-truncated_aspect1]-qe_ci[truncated_aspect1:-truncated_aspect1]), (yhat[truncated_aspect1:-truncated_aspect1]+qe_ci[truncated_aspect1:-truncated_aspect1]), color='b', alpha=.1)
plt.plot(X_test, Y_star_mean, 'orange', label='Prediction from Variational Sparse GP')
plt.plot(X_test, yhat_full,linestyle='dashed', label='Prediction from Full GPR', c='r')

plt.plot(X_test, yhat_qk,linestyle='dashed', label='Prediction from Full GPR', c='y')
#plt.plot(X, y, label='Training examples', c='r')
plt.title('Model Predictions')
plt.plot(X_test, f_true, 'black', label='Objective Function', c='k')
plt.xlabel('x')
plt.ylabel('y')
#plt.xlim(-1.5, 1.5)
plt.legend()

# Test Regression Data Sets


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define the range for n and m
n_values = np.arange(10, 101)
m_values = np.arange(5, 101)

# Calculate the values of the functions nm^2 and n^2 + m^2
nm2_values = [n * m**2 for n, m in zip(n_values, m_values)]
n2_m2_values = [n**2 + m**2 for n, m in zip(n_values, m_values)]
m3_values = [m**3 for n, m in zip(n_values, m_values)]
n3_values = [n**3 for n, m in zip(n_values, m_values)]

# Plot the functions
plt.figure(figsize=(10, 6))
plt.plot(n_values, nm2_values, label='nm^2', color='blue')
plt.plot(n_values, n2_m2_values, label='n^2 + m^2', color='red')
plt.plot(n_values, m3_values, label='m^3', color='green') 
plt.plot(n_values, n3_values, label='n^3', color='purple' )

plt.xlabel('Value of n (and m)')
plt.ylabel('Function Value')
plt.title('Comparison of Asymptotic Growth')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
from qiskit import Aer
from qiskit.aqua.algorithms import HHL, NumPyLSsolver
from qiskit.aqua.components.eigs import EigsQPE
from qiskit.aqua.components.reciprocals import LookupRotation
from qiskit.aqua.operators import MatrixOperator
from qiskit.aqua.components.initial_states import Custom

# Define matrix A and vector b for Ax = b
matrix = np.array([[1, -1/3], [-1/3, 1]])
vector = np.array([1, 0])

# Decompose matrix for HHL
qpe = EigsQPE(MatrixOperator(matrix=matrix), 
              num_ancillae=3,
              num_time_slices=50)

# Create the HHL algorithm instance
hhl = HHL(qpe, 
          state_in=Custom(qpe.get_register_sizes()[0], state_vector=vector), 
          reciprocal=LookupRotation(negative_evals=True))

# Run the algorithm directly on the backend
backend = Aer.get_backend('statevector_simulator')
result = hhl.run(backend)  # Note the direct use of the backend here

# Compare with classical solution
result_ref = NumPyLSsolver(matrix, vector).run()

print("Classical solution:", result_ref['solution'])
print("Quantum solution:", result['solution'])


In [ ]:
print(qiskit.__version__)